In [ ]:
# SINGLE-CELL END-TO-END PIPELINE FOR ICRISAT-STYLE DATASET
# ----------------------------------------------------------
# This cell implements:
# - Loading and basic EDA
# - Cleaning and outlier handling
# - Crop share and diversification computation
# - District-level similarity network + metrics
# - Crop-crop co-occurrence network + metrics
# - Supervised ML dataset construction for selected crops
# - STL-based leakage-free temporal features
# - Time-aware expanding-window CV
# - Model training and evaluation (Naive, Ridge, RF, GB)
# - Plots and saving outputs
# ----------------------------------------------------------

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from statsmodels.tsa.seasonal import STL

# Global configuration
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Path to data (adjust as needed in Colab)
DATA_PATH = "/content/drive/MyDrive/Shiny/ICRISAT/ICRISAT-District Level Data.csv"

# Output paths
OUTPUT_DIR = "/content/drive/MyDrive/Shiny/ICRISAT/Results"

# Ensure output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

CLEANED_DATA_PATH = os.path.join(OUTPUT_DIR, "icrisat_cleaned_with_features.csv")
MODEL_PERF_PATH = os.path.join(OUTPUT_DIR, "model_performance_summary.csv")
SUP_DATA_PREFIX = os.path.join(OUTPUT_DIR, "supervised_panel_")  # crop name will be appended

# Target crops to attempt modeling (if present)
TARGET_CROPS = ["RICE", "WHEAT", "MAIZE", "GROUNDNUT", "COTTON", "SUGARCANE"]

# -------------------------
# Helper functions
# -------------------------

def load_data(path):
    """Load dataset and perform initial type conversions."""
    if not os.path.exists(path):
        raise FileNotFoundError(f"Data file not found at: {path}")
    df = pd.read_csv(path)
    # Convert -1 to NaN in numeric columns
    num_cols = df.select_dtypes(include=[np.number]).columns
    df[num_cols] = df[num_cols].replace(-1, np.nan)
    # Ensure Year is integer
    if "Year" in df.columns:
        df["Year"] = df["Year"].astype(int)
    else:
        raise KeyError("Column 'Year' not found in dataset.")
    return df


def detect_crop_columns(df):
    """Detect area, production, yield columns and derive crop names."""
    area_cols = [c for c in df.columns if c.endswith("AREA (1000 ha)")]
    prod_cols = [c for c in df.columns if c.endswith("PRODUCTION (1000 tons)")]
    yield_cols = [c for c in df.columns if c.endswith("YIELD (Kg per ha)")]
    crops = set()
    for col in area_cols + prod_cols + yield_cols:
        if " AREA (1000 ha)" in col:
            crop = col.replace(" AREA (1000 ha)", "").strip()
        elif " PRODUCTION (1000 tons)" in col:
            crop = col.replace(" PRODUCTION (1000 tons)", "").strip()
        elif " YIELD (Kg per ha)" in col:
            crop = col.replace(" YIELD (Kg per ha)", "").strip()
        else:
            continue
        crops.add(crop)
    crops = sorted(list(crops))
    return crops, area_cols, prod_cols, yield_cols


def verify_keys(df):
    """Verify uniqueness of (Dist Code, Year)."""
    if "Dist Code" not in df.columns:
        raise KeyError("Column 'Dist Code' not found in dataset.")
    dup_mask = df.duplicated(subset=["Dist Code", "Year"])
    num_dups = dup_mask.sum()
    print(f"Duplicate (Dist Code, Year) rows: {num_dups}")
    if num_dups == 0:
        print("Key (Dist Code, Year) is unique.")
    else:
        print("WARNING: (Dist Code, Year) is not unique; duplicates exist.")


def basic_eda(df, crops, yield_cols):
    """Basic exploratory statistics and plots."""
    print("\n===== BASIC EDA =====")
    print(f"Rows: {len(df)}, Columns: {len(df.columns)}")
    print(f"Time range: {df['Year'].min()} - {df['Year'].max()}")
    print(f"Unique States: {df['State Name'].nunique() if 'State Name' in df.columns else 'N/A'}")
    print(f"Unique Districts: {df['Dist Name'].nunique() if 'Dist Name' in df.columns else 'N/A'}")
    print(f"Detected crops (count={len(crops)}): {crops[:20]}{'...' if len(crops) > 20 else ''}")

    # Missingness summary for yields
    print("\nYield missingness (% of NaN):")
    miss_data = []
    for yc in yield_cols:
        miss_pct = df[yc].isna().mean() * 100
        miss_data.append((yc, miss_pct))
    miss_df = pd.DataFrame(miss_data, columns=["YieldColumn", "MissingPct"]).sort_values("MissingPct", ascending=False)
    print(miss_df.head(20))

    # Summary stats for some key crops if present
    def summarize_crop(crop):
        y_col = f"{crop} YIELD (Kg per ha)"
        if y_col in df.columns:
            print(f"\nSummary for {crop} yield:")
            print(df[y_col].describe())

    for crop in ["RICE", "WHEAT", "MAIZE", "GROUNDNUT", "COTTON", "SUGARCANE"]:
        summarize_crop(crop)

    # Plots: average yields over years for RICE and WHEAT (if available)
    try:
        plt.figure(figsize=(8, 5))
        plotted = False
        for crop, color in [("RICE", "tab:blue"), ("WHEAT", "tab:orange")]:
            y_col = f"{crop} YIELD (Kg per ha)"
            if y_col in df.columns:
                avg_y = df.groupby("Year")[y_col].mean()
                plt.plot(avg_y.index, avg_y.values, label=crop, alpha=0.8)
                plotted = True
        if plotted:
            plt.xlabel("Year")
            plt.ylabel("Average Yield (kg/ha)")
            plt.title("Average Yield Over Time (National)")
            plt.legend()
            plt.tight_layout()
            plt.savefig(os.path.join(OUTPUT_DIR, "avg_yield_rice_wheat_over_time.pdf"), bbox_inches="tight")
        plt.close()
    except Exception as e:
        print(f"Failed to create/save avg yield plot: {e}")

    # Histogram/KDE for one major crop
    for crop in ["RICE", "WHEAT", "MAIZE"]:
        y_col = f"{crop} YIELD (Kg per ha)"
        if y_col in df.columns:
            try:
                plt.figure(figsize=(7, 4))
                sns.histplot(df[y_col].dropna(), kde=True, bins=40)
                plt.xlabel(f"{crop} Yield (kg/ha)")
                plt.title(f"Distribution of {crop} Yield")
                plt.tight_layout()
                fname = f"{crop.lower()}_yield_distribution.pdf"
                plt.savefig(os.path.join(OUTPUT_DIR, fname), bbox_inches="tight")
                plt.close()
            except Exception as e:
                print(f"Failed to create/save distribution plot for {crop}: {e}")
            break


def clean_data(df, crops, area_cols, prod_cols, yield_cols):
    """Clean data: handle invalid yields, clip outliers, basic quality report."""
    print("\n===== CLEANING DATA =====")

    # Map crop -> col names
    crop_cols = {}
    for crop in crops:
        area_col = f"{crop} AREA (1000 ha)"
        prod_col = f"{crop} PRODUCTION (1000 tons)"
        yield_col = f"{crop} YIELD (Kg per ha)"
        crop_cols[crop] = {
            "area": area_col if area_col in df.columns else None,
            "prod": prod_col if prod_col in df.columns else None,
            "yield": yield_col if yield_col in df.columns else None,
        }

    # Invalidate yields and production when area <= 0
    for crop, cols in crop_cols.items():
        a, p, y = cols["area"], cols["prod"], cols["yield"]
        if a is None:
            continue
        area_series = df[a]
        invalid_mask = area_series <= 0
        if y is not None:
            df.loc[invalid_mask, y] = np.nan
        if p is not None:
            df.loc[invalid_mask, p] = np.nan

    # Clip yield outliers
    for crop, cols in crop_cols.items():
        y = cols["yield"]
        if y is None or y not in df.columns:
            continue
        series = df[y]
        if series.notna().sum() < 50:
            continue  # too few values for robust percentiles
        p1, p99 = np.nanpercentile(series, [1, 99])
        old_max = series.max()
        df[y] = np.clip(series, p1, p99)
        new_max = df[y].max()
        print(f"Clipped {crop} yield: old max={old_max:.2f}, new max={new_max:.2f}")

    # Data quality: non-missing fraction per crop yield
    quality = []
    for crop, cols in crop_cols.items():
        y = cols["yield"]
        if y is not None and y in df.columns:
            frac = df[y].notna().mean()
            quality.append((crop, frac))
    qdf = pd.DataFrame(quality, columns=["Crop", "NonMissingFrac"]).sort_values("NonMissingFrac", ascending=False)
    print("\nTop 15 crops by non-missing yield fraction:")
    print(qdf.head(15))
    return df, crop_cols


def compute_diversification(df, area_cols):
    """Compute total area, area shares, Herfindahl and diversification index."""
    print("\n===== COMPUTING DIVERSIFICATION =====")
    if not area_cols:
        raise ValueError("No area columns detected; cannot compute diversification.")

    # Total cropped area
    df["TOTAL_AREA_1000_HA"] = df[area_cols].clip(lower=0).sum(axis=1, min_count=1)

    # Area shares
    share_cols = []
    for col in area_cols:
        share_col = col.replace(" AREA (1000 ha)", " AREA_SHARE")
        share_cols.append(share_col)
        denom = df["TOTAL_AREA_1000_HA"].replace(0, np.nan)
        df[share_col] = df[col].clip(lower=0) / denom
        df[share_col] = df[share_col].fillna(0.0)

    # Herfindahl and diversification index
    df["HERFINDAHL"] = (df[share_cols] ** 2).sum(axis=1)
    df["DIVERSIFICATION_INDEX"] = 1.0 - df["HERFINDAHL"]

    # Plot diversification over time for a few states if available
    if "State Name" in df.columns:
        try:
            top_states = df["State Name"].value_counts().index[:3]
            plt.figure(figsize=(8, 5))
            for st in top_states:
                sub = df[df["State Name"] == st].groupby("Year")["DIVERSIFICATION_INDEX"].mean()
                plt.plot(sub.index, sub.values, label=st)
            plt.xlabel("Year")
            plt.ylabel("Diversification Index (1 - Herfindahl)")
            plt.title("Average Diversification by State")
            plt.legend()
            plt.tight_layout()
            plt.savefig(os.path.join(OUTPUT_DIR, "diversification_by_state.pdf"), bbox_inches="tight")
            plt.close()
        except Exception as e:
            print(f"Failed to create/save diversification plot: {e}")

    return df


def build_district_network(df, area_cols, k_neighbors=5):
    """Build district-level similarity network based on crop-share vectors."""
    print("\n===== BUILDING DISTRICT SIMILARITY NETWORK =====")
    if not area_cols:
        raise ValueError("No area columns; cannot build district network.")

    # Identify share columns
    share_cols = [c for c in df.columns if c.endswith("AREA_SHARE")]
    if not share_cols:
        raise ValueError("No share columns found; ensure compute_diversification was run.")

    # Reference window: last 10 years (or less if data shorter)
    max_year = df["Year"].max()
    min_year = df["Year"].min()
    window = 10
    start_win = max(min_year, max_year - window + 1)
    df_win = df[(df["Year"] >= start_win) & (df["Year"] <= max_year)].copy()
    print(f"Network reference window: {start_win}-{max_year}")

    # Compute mean crop shares per district over window
    group_cols = ["Dist Code"]
    share_means = df_win.groupby(group_cols)[share_cols + ["TOTAL_AREA_1000_HA"]].mean()
    share_means = share_means.rename(columns={"TOTAL_AREA_1000_HA": "MEAN_TOTAL_AREA_1000_HA"})
    share_means = share_means[share_means["MEAN_TOTAL_AREA_1000_HA"] > 0]
    if share_means.empty:
        raise ValueError("No districts with positive total area in reference window.")

    dist_codes = share_means.index.to_list()
    X = share_means[share_cols].values
    cos_sim = cosine_similarity(X)

    # Build k-NN graph
    G = nx.Graph()
    # Add nodes with attributes
    meta_cols = ["Dist Name", "State Name"]
    meta_df = df.drop_duplicates(subset=["Dist Code"])[["Dist Code"] + [c for c in meta_cols if c in df.columns]].set_index("Dist Code")
    for dc in dist_codes:
        attrs = {}
        if dc in meta_df.index:
            for c in meta_cols:
                if c in meta_df.columns:
                    attrs[c] = meta_df.loc[dc, c]
        G.add_node(dc, **attrs)

    # Add edges based on top-k cosine similarity
    n = len(dist_codes)
    for i in range(n):
        sims = cos_sim[i]
        idx_sorted = np.argsort(-sims)
        added = 0
        for j in idx_sorted:
            if i == j:
                continue
            if sims[j] <= 0:
                continue
            u = dist_codes[i]
            v = dist_codes[j]
            w = float(sims[j])
            if G.has_edge(u, v):
                continue
            G.add_edge(u, v, weight=w)
            added += 1
            if added >= k_neighbors:
                break

    print(f"District network: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")

    # Network metrics
    strength = dict(G.degree(weight="weight"))
    betw = nx.betweenness_centrality(G, normalized=True, weight=None)
    try:
        eig = nx.eigenvector_centrality_numpy(G, weight="weight")
    except Exception as e:
        print(f"Eigenvector centrality failed ({e}); using degree as fallback.")
        eig = {n: strength.get(n, 0.0) for n in G.nodes()}
    clust = nx.clustering(G, weight="weight")

    metrics_df = pd.DataFrame({
        "Dist Code": list(G.nodes()),
        "NET_STRENGTH": [strength.get(n, 0.0) for n in G.nodes()],
        "NET_BETWEENNESS": [betw.get(n, 0.0) for n in G.nodes()],
        "NET_EIGENVECTOR": [eig.get(n, 0.0) for n in G.nodes()],
        "NET_CLUSTERING": [clust.get(n, 0.0) for n in G.nodes()],
    })

    # Merge back to main df
    df = df.merge(metrics_df, on="Dist Code", how="left")

    # Simple network visualization
    try:
        plt.figure(figsize=(8, 8))
        pos = nx.spring_layout(G, seed=RANDOM_STATE)
        node_sizes = np.array([strength.get(n, 0.0) for n in G.nodes()])
        node_sizes = 100 + 400 * (node_sizes - node_sizes.min()) / (node_sizes.max() - node_sizes.min() + 1e-6)
        nx.draw_networkx_nodes(G, pos, node_size=node_sizes, alpha=0.7)
        nx.draw_networkx_edges(G, pos, alpha=0.3)
        plt.axis("off")
        plt.title("District Similarity Network (Strength-scaled Nodes)")
        plt.tight_layout()
        plt.savefig(os.path.join(OUTPUT_DIR, "district_network_overview.pdf"), bbox_inches="tight")
        plt.close()
    except Exception as e:
        print(f"Network plot failed: {e}")

    return df, G, share_means, share_cols


def build_crop_network(share_means, share_cols):
    """Build crop-crop co-occurrence network from district-level mean shares."""
    print("\n===== BUILDING CROP-COOCCURRENCE NETWORK =====")
    if share_means is None or share_means.empty:
        print("No share means provided; skipping crop network.")
        return None, None

    # Presence matrix: district x crop
    share_array = share_means[share_cols].values
    presence = (share_array > 0).astype(int)
    if presence.size == 0:
        print("No non-zero shares; skipping crop network.")
        return None, None

    # Co-occurrence matrix
    coocc = presence.T @ presence  # crops x crops
    crop_names = [c.replace(" AREA_SHARE", "").strip() for c in share_cols]

    Gc = nx.Graph()
    for i, ci in enumerate(crop_names):
        Gc.add_node(ci)
    n_crops = len(crop_names)
    for i in range(n_crops):
        for j in range(i + 1, n_crops):
            w = int(coocc[i, j])
            if w > 0:
                Gc.add_edge(crop_names[i], crop_names[j], weight=float(w))

    print(f"Crop network: {Gc.number_of_nodes()} nodes, {Gc.number_of_edges()} edges")

    # Metrics
    degree = dict(Gc.degree(weight="weight"))
    try:
        eig = nx.eigenvector_centrality_numpy(Gc, weight="weight")
    except Exception as e:
        print(f"Crop eigenvector centrality failed ({e}); using degree as fallback.")
        eig = {n: degree.get(n, 0.0) for n in Gc.nodes()}

    crop_metrics = pd.DataFrame({
        "Crop": list(Gc.nodes()),
        "CROP_DEGREE": [degree.get(n, 0.0) for n in Gc.nodes()],
        "CROP_EIGENVECTOR": [eig.get(n, 0.0) for n in Gc.nodes()],
    }).set_index("Crop")

    return Gc, crop_metrics


def add_stl_features_to_group(group, target_col, min_len=5):
    """Add STL-based trend and residual features to a district-crop group."""
    group = group.sort_values("Year").copy()
    y = group[target_col].values
    trend = np.zeros_like(y, dtype=float)
    resid = np.zeros_like(y, dtype=float)

    for i in range(len(y)):
        series = y[:i+1]
        if np.isnan(series).all():
            trend[i] = np.nan
            resid[i] = np.nan
            continue
        valid_idx = ~np.isnan(series)
        series_valid = series[valid_idx]
        if len(series_valid) < min_len:
            trend[i] = series_valid[-1]
            resid[i] = 0.0
        else:
            period = min(10, max(2, len(series_valid) // 2))
            try:
                stl = STL(series_valid, period=period, robust=True)
                res = stl.fit()
                trend[i] = res.trend[-1]
                resid[i] = res.resid[-1]
            except Exception:
                trend[i] = series_valid[-1]
                resid[i] = 0.0

    group["STL_TREND"] = trend
    group["STL_RESID"] = resid
    return group


def construct_supervised_dataset_for_crop(df, crop, crop_cols, crop_metrics=None):
    """Construct supervised dataset for yield prediction for a given crop."""
    print(f"\n===== BUILDING SUPERVISED DATA FOR {crop} =====")

    cols = crop_cols.get(crop, {})
    area_col = cols.get("area")
    prod_col = cols.get("prod")
    yield_col = cols.get("yield")

    if yield_col is None or yield_col not in df.columns:
        print(f"Yield column for {crop} not found; skipping.")
        return None

    # Basic columns
    base_cols = ["Dist Code", "Year"]
    if "State Name" in df.columns:
        base_cols.append("State Name")
    if "Dist Name" in df.columns:
        base_cols.append("Dist Name")

    feat_cols = []
    for c in ["TOTAL_AREA_1000_HA", "DIVERSIFICATION_INDEX",
              "NET_STRENGTH", "NET_BETWEENNESS", "NET_EIGENVECTOR", "NET_CLUSTERING"]:
        if c in df.columns:
            feat_cols.append(c)

    use_cols = base_cols.copy()
    if area_col is not None and area_col in df.columns:
        use_cols.append(area_col)
    if prod_col is not None and prod_col in df.columns:
        use_cols.append(prod_col)
    use_cols.append(yield_col)
    use_cols += feat_cols

    use_cols = list(dict.fromkeys(use_cols))

    sub = df[use_cols].copy()
    sub = sub[sub[yield_col].notna()]
    if sub.empty:
        print(f"No non-missing yield data for {crop}; skipping.")
        return None

    # Add crop-level metrics (constant per crop)
    if crop_metrics is not None and crop in crop_metrics.index:
        sub["CROP_DEGREE"] = crop_metrics.loc[crop, "CROP_DEGREE"]
        sub["CROP_EIGENVECTOR"] = crop_metrics.loc[crop, "CROP_EIGENVECTOR"]

    # Group by district and construct temporal features
    def process_group(g):
        g = g.sort_values("Year").copy()
        # Lag features for yield
        g[f"{crop}_Y_LAG1"] = g[yield_col].shift(1)
        g[f"{crop}_Y_LAG2"] = g[yield_col].shift(2)

        # Lag features for area & production if present
        if area_col is not None and area_col in g.columns:
            g[f"{crop}_A_LAG1"] = g[area_col].shift(1)
        if prod_col is not None and prod_col in g.columns:
            g[f"{crop}_P_LAG1"] = g[prod_col].shift(1)

        # Rolling statistics (using only past values)
        g[f"{crop}_Y_ROLL_MEAN3"] = g[yield_col].shift(1).rolling(window=3, min_periods=1).mean()
        g[f"{crop}_Y_ROLL_STD3"] = g[yield_col].shift(1).rolling(window=3, min_periods=2).std()
        g[f"{crop}_Y_ROLL_CV3"] = g[f"{crop}_Y_ROLL_STD3"] / (g[f"{crop}_Y_ROLL_MEAN3"].abs() + 1e-6)

        # Time index features
        year0 = g["Year"].min()
        g["YEAR_CENTERED"] = g["Year"] - year0
        g["YEAR_CENTERED2"] = g["YEAR_CENTERED"] ** 2

        # STL-based features
        g = add_stl_features_to_group(g, yield_col)

        # Target: next-year yield
        g["TARGET_YIELD_NEXT"] = g[yield_col].shift(-1)

        # Drop last year (where target is NaN)
        g = g[g["TARGET_YIELD_NEXT"].notna()]

        return g

    sup = sub.groupby("Dist Code", group_keys=False).apply(process_group)
    if sup.empty:
        print(f"Supervised dataset for {crop} is empty after processing; skipping.")
        return None

    # Drop rows with missing predictors or target
    sup_clean = sup.dropna()
    if sup_clean.empty:
        print(f"Supervised dataset for {crop} has no fully non-missing rows; skipping.")
        return None

    # Save supervised panel
    out_path = f"{SUP_DATA_PREFIX}{crop.lower()}.csv"
    sup_clean.to_csv(out_path, index=False)
    print(f"Saved supervised panel for {crop} to: {out_path}")

    return sup_clean, yield_col


def expanding_time_splits(years, n_splits=3, min_train_years=10, test_window_years=5):
    """Generate expanding-window time-series splits based on years."""
    unique_years = np.sort(np.unique(years))
    if len(unique_years) < min_train_years + 2:
        return []

    min_year = unique_years[0]
    max_year = unique_years[-1]
    splits = []
    for i in range(n_splits):
        train_end = min_year + min_train_years - 1 + i * test_window_years
        test_start = train_end + 1
        test_end = test_start + test_window_years - 1
        if test_start >= max_year:
            break
        train_end = min(train_end, max_year - 1)
        test_end = min(test_end, max_year)
        train_mask = (years <= train_end)
        test_mask = (years >= test_start) & (years <= test_end)
        train_idx = np.where(train_mask)[0]
        test_idx = np.where(test_mask)[0]
        if len(train_idx) == 0 or len(test_idx) == 0:
            continue
        splits.append((train_idx, test_idx))
    return splits


def compute_metrics(y_true, y_pred, mape_threshold=100.0):
    """Compute MAE, RMSE, MAPE (modified), R2."""
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mask = np.abs(y_true) > mape_threshold
    if mask.sum() > 0:
        mape = np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100.0
    else:
        mape = np.nan
    r2 = r2_score(y_true, y_pred)
    return mae, rmse, mape, r2


def evaluate_models_for_crop(sup_df, crop, yield_col):
    """Evaluate baseline and ML models for a given crop with time-aware CV."""
    print(f"\n===== MODELING {crop} =====")
    exclude_cols = ["Dist Code", "Year", "TARGET_YIELD_NEXT"]
    id_cols = [c for c in ["State Name", "Dist Name"] if c in sup_df.columns]
    exclude_cols += id_cols
    feature_cols = [c for c in sup_df.columns if c not in exclude_cols]
    feature_cols = [c for c in feature_cols if not c.endswith(yield_col)]

    X = sup_df[feature_cols].values.astype(float)
    y = sup_df["TARGET_YIELD_NEXT"].values.astype(float)
    years = sup_df["Year"].values.astype(int)

    splits = expanding_time_splits(years, n_splits=3, min_train_years=10, test_window_years=5)
    if not splits:
        print(f"Not enough temporal range to create CV folds for {crop}; skipping modeling.")
        return None

    models = {
        "Naive": None,
        "Ridge": Pipeline([
            ("scaler", RobustScaler()),
            ("model", Ridge(alpha=1.0, random_state=RANDOM_STATE))
        ]),
        "RandomForest": Pipeline([
            ("scaler", RobustScaler()),
            ("model", RandomForestRegressor(
                n_estimators=200,
                max_depth=7,
                random_state=RANDOM_STATE,
                n_jobs=-1
            ))
        ]),
        "GradientBoosting": Pipeline([
            ("scaler", RobustScaler()),
            ("model", GradientBoostingRegressor(
                n_estimators=300,
                learning_rate=0.05,
                max_depth=3,
                random_state=RANDOM_STATE
            ))
        ])
    }

    perf_records = []

    for model_name, model in models.items():
        fold_mae, fold_rmse, fold_mape, fold_r2 = [], [], [], []
        for fold_id, (train_idx, test_idx) in enumerate(splits):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            if model_name == "Naive":
                lag1_name = f"{crop}_Y_LAG1"
                if lag1_name in feature_cols:
                    lag1_idx = feature_cols.index(lag1_name)
                    y_pred = X_test[:, lag1_idx]
                    mean_train_y = np.nanmean(y_train)
                    y_pred = np.where(np.isnan(y_pred), mean_train_y, y_pred)
                else:
                    y_pred = np.repeat(np.mean(y_train), len(y_test))
            else:
                model.fit(X_train, y_train)
                y_pred = model.predict(X_test)

            mae, rmse, mape, r2 = compute_metrics(y_test, y_pred)
            fold_mae.append(mae)
            fold_rmse.append(rmse)
            fold_mape.append(mape)
            fold_r2.append(r2)

            print(f"{crop} | {model_name} | Fold {fold_id+1}: MAE={mae:.2f}, RMSE={rmse:.2f}, MAPE={mape:.2f}, R2={r2:.3f}")

        perf_records.append({
            "Crop": crop,
            "Model": model_name,
            "MAE_mean": np.nanmean(fold_mae),
            "MAE_std": np.nanstd(fold_mae),
            "RMSE_mean": np.nanmean(fold_rmse),
            "RMSE_std": np.nanstd(fold_rmse),
            "MAPE_mean": np.nanmean(fold_mape),
            "MAPE_std": np.nanstd(fold_mape),
            "R2_mean": np.nanmean(fold_r2),
            "R2_std": np.nanstd(fold_r2),
        })

    perf_df = pd.DataFrame(perf_records)
    print(f"\nPerformance summary for {crop}:")
    print(perf_df.sort_values(["MAE_mean"]))

    return perf_df


def diagnostic_plots_for_crop(sup_df, crop, yield_col, best_model_name):
    """Create diagnostic plots (true vs pred and time-series) for a selected crop."""
    print(f"\n===== DIAGNOSTIC PLOTS FOR {crop} ({best_model_name}) =====")

    exclude_cols = ["Dist Code", "Year", "TARGET_YIELD_NEXT"]
    id_cols = [c for c in ["State Name", "Dist Name"] if c in sup_df.columns]
    exclude_cols += id_cols
    feature_cols = [c for c in sup_df.columns if c not in exclude_cols]
    feature_cols = [c for c in feature_cols if not c.endswith(yield_col)]

    X = sup_df[feature_cols].values.astype(float)
    y = sup_df["TARGET_YIELD_NEXT"].values.astype(float)
    years = sup_df["Year"].values.astype(int)

    splits = expanding_time_splits(years, n_splits=3, min_train_years=10, test_window_years=5)
    if not splits:
        print("Not enough data for diagnostic plots.")
        return

    models = {
        "Ridge": Pipeline([
            ("scaler", RobustScaler()),
            ("model", Ridge(alpha=1.0, random_state=RANDOM_STATE))
        ]),
        "RandomForest": Pipeline([
            ("scaler", RobustScaler()),
            ("model", RandomForestRegressor(
                n_estimators=200,
                max_depth=7,
                random_state=RANDOM_STATE,
                n_jobs=-1
            ))
        ]),
        "GradientBoosting": Pipeline([
            ("scaler", RobustScaler()),
            ("model", GradientBoostingRegressor(
                n_estimators=300,
                learning_rate=0.05,
                max_depth=3,
                random_state=RANDOM_STATE
            ))
        ])
    }
    if best_model_name not in models:
        print(f"Best model {best_model_name} not in available models for diagnostics.")
        return

    model = models[best_model_name]

    train_idx, test_idx = splits[-1]
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # True vs predicted scatter
    try:
        plt.figure(figsize=(6, 6))
        plt.scatter(y_test, y_pred, alpha=0.5)
        lims = [min(y_test.min(), y_pred.min()), max(y_test.max(), y_pred.max())]
        plt.plot(lims, lims, "r--")
        plt.xlabel("True Yield (kg/ha)")
        plt.ylabel("Predicted Yield (kg/ha)")
        plt.title(f"{crop}: True vs Predicted ({best_model_name})")
        plt.tight_layout()
        fname_scatter = os.path.join(OUTPUT_DIR, f"{crop.lower()}_yield_true_vs_pred.pdf")
        plt.savefig(fname_scatter, bbox_inches="tight")
        plt.close()
    except Exception as e:
        print(f"Failed to create/save true-vs-pred plot for {crop}: {e}")

    # Time-series plot for a few example districts
    try:
        sup_df = sup_df.reset_index(drop=True)
        sup_df["PRED_TEMP"] = np.nan
        sup_df.loc[test_idx, "PRED_TEMP"] = y_pred

        if "Dist Code" not in sup_df.columns:
            return
        test_districts = sup_df.loc[test_idx, "Dist Code"].dropna().unique()
        if len(test_districts) == 0:
            return
        np.random.seed(RANDOM_STATE)
        sample_dists = np.random.choice(test_districts, size=min(3, len(test_districts)), replace=False)

        plt.figure(figsize=(9, 6))
        for d in sample_dists:
            g = sup_df[sup_df["Dist Code"] == d].sort_values("Year")
            plt.plot(g["Year"], g["TARGET_YIELD_NEXT"], "-o", label=f"True (Dist {d})", alpha=0.7)
            plt.plot(g["Year"], g["PRED_TEMP"], "--x", label=f"Pred (Dist {d})", alpha=0.7)
        plt.xlabel("Year")
        plt.ylabel("Next-Year Yield (kg/ha)")
        plt.title(f"{crop}: True vs Predicted Over Time ({best_model_name})")
        plt.legend(fontsize=8)
        plt.tight_layout()
        fname_ts = os.path.join(OUTPUT_DIR, f"{crop.lower()}_yield_timeseries_example.pdf")
        plt.savefig(fname_ts, bbox_inches="tight")
        plt.close()
    except Exception as e:
        print(f"Failed to create/save time-series plot for {crop}: {e}")


def main():
    # Ensure output directory exists (defensive)
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    # Load data
    df = load_data(DATA_PATH)

    # Verify key uniqueness
    verify_keys(df)

    # Detect crops and columns
    crops, area_cols, prod_cols, yield_cols = detect_crop_columns(df)

    # Basic EDA
    basic_eda(df, crops, yield_cols)

    # Cleaning and outlier handling
    df, crop_cols = clean_data(df, crops, area_cols, prod_cols, yield_cols)

    # Diversification
    df = compute_diversification(df, area_cols)

    # District network
    try:
        df, G_district, share_means, share_cols = build_district_network(df, area_cols, k_neighbors=5)
    except Exception as e:
        print(f"District network construction failed: {e}")
        G_district, share_means, share_cols = None, None, None

    # Crop network
    if share_means is not None and share_cols is not None:
        G_crop, crop_metrics = build_crop_network(share_means, share_cols)
    else:
        G_crop, crop_metrics = None, None

    # Save cleaned data with features
    df.to_csv(CLEANED_DATA_PATH, index=False)
    print(f"\nSaved cleaned dataset with features to: {CLEANED_DATA_PATH}")

    # Supervised datasets and modeling
    all_perf = []
    modeled_crops = []
    best_models_per_crop = {}
    sup_datasets = {}

    for crop in TARGET_CROPS:
        sup_res = construct_supervised_dataset_for_crop(df, crop, crop_cols, crop_metrics=crop_metrics)
        if sup_res is None:
            continue
        sup_df, yield_col = sup_res
        sup_datasets[crop] = (sup_df, yield_col)
        perf_df = evaluate_models_for_crop(sup_df, crop, yield_col)
        if perf_df is not None and not perf_df.empty:
            all_perf.append(perf_df)
            modeled_crops.append(crop)
            best_row = perf_df.loc[perf_df["MAE_mean"].idxmin()]
            best_model_name = best_row["Model"]
            best_models_per_crop[crop] = best_model_name

    if all_perf:
        perf_all_df = pd.concat(all_perf, ignore_index=True)
        perf_all_df.to_csv(MODEL_PERF_PATH, index=False)
        print(f"\nSaved model performance summary to: {MODEL_PERF_PATH}")
    else:
        perf_all_df = pd.DataFrame()
        print("\nNo models were successfully evaluated; performance summary not created.")

    # Diagnostic plots for one crop (e.g., first modeled crop)
    if modeled_crops:
        crop_for_plots = modeled_crops[0]
        best_model_name = best_models_per_crop.get(crop_for_plots, "RandomForest")
        sup_df_plot, yield_col_plot = sup_datasets[crop_for_plots]
        diagnostic_plots_for_crop(sup_df_plot, crop_for_plots, yield_col_plot, best_model_name)

    # Final console summary
    print("\n===== FINAL SUMMARY =====")
    print(f"Rows after cleaning: {len(df)}")
    print(f"Years: {df['Year'].min()} - {df['Year'].max()}")
    print(f"Number of districts: {df['Dist Code'].nunique() if 'Dist Code' in df.columns else 'N/A'}")
    print(f"Crops modeled: {modeled_crops}")
    if best_models_per_crop:
        print("Best model per crop (by MAE_mean):")
        for c, m in best_models_per_crop.items():
            print(f"  {c}: {m}")
    print(f"Cleaned dataset with features: {CLEANED_DATA_PATH}")
    print(f"Model performance summary: {MODEL_PERF_PATH if not perf_all_df.empty else 'N/A'}")
    print(f"Supervised panel CSV prefix: {SUP_DATA_PREFIX}")
    print("Key PDF plots saved in OUTPUT_DIR (yield trends, distributions, network overview, diagnostics).")


# Run main
if __name__ == "__main__":
    main()


Duplicate (Dist Code, Year) rows: 0
Key (Dist Code, Year) is unique.

===== BASIC EDA =====
Rows: 16146, Columns: 80
Time range: 1966 - 2017
Unique States: 20
Unique Districts: 311
Detected crops (count=29): ['BARLEY', 'CASTOR', 'CHICKPEA', 'COTTON', 'FINGER MILLET', 'FODDER', 'FRUITS', 'FRUITS AND VEGETABLES', 'GROUNDNUT', 'KHARIF SORGHUM', 'LINSEED', 'MAIZE', 'MINOR PULSES', 'OILSEEDS', 'ONION', 'PEARL MILLET', 'PIGEONPEA', 'POTATOES', 'RABI SORGHUM', 'RAPESEED AND MUSTARD']...

Yield missingness (% of NaN):
                         YieldColumn  MissingPct
20        OILSEEDS YIELD (Kg per ha)   10.361699
11    MINOR PULSES YIELD (Kg per ha)    8.794748
15       SAFFLOWER YIELD (Kg per ha)    2.991453
17         LINSEED YIELD (Kg per ha)    1.591726
21       SUGARCANE YIELD (Kg per ha)    1.492630
16          CASTOR YIELD (Kg per ha)    0.514059
5     PEARL MILLET YIELD (Kg per ha)    0.334448
13         SESAMUM YIELD (Kg per ha)    0.272513
9         CHICKPEA YIELD (Kg per ha)    0.2

/tmp/ipython-input-2696958754.py:242: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[share_col] = df[col].clip(lower=0) / denom
/tmp/ipython-input-2696958754.py:242: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[share_col] = df[col].clip(lower=0) / denom
/tmp/ipython-input-2696958754.py:242: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmen


===== BUILDING DISTRICT SIMILARITY NETWORK =====
Network reference window: 2008-2017
District network: 311 nodes, 1555 edges

===== BUILDING CROP-COOCCURRENCE NETWORK =====
Crop network: 29 nodes, 406 edges

Saved cleaned dataset with features to: /content/drive/MyDrive/Shiny/ICRISAT/Results/icrisat_cleaned_with_features.csv

===== BUILDING SUPERVISED DATA FOR RICE =====


/tmp/ipython-input-2696958754.py:539: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sup = sub.groupby("Dist Code", group_keys=False).apply(process_group)


Saved supervised panel for RICE to: /content/drive/MyDrive/Shiny/ICRISAT/Results/supervised_panel_rice.csv

===== MODELING RICE =====
RICE | Naive | Fold 1: MAE=324.00, RMSE=447.30, MAPE=39.32, R2=0.619
RICE | Naive | Fold 2: MAE=294.57, RMSE=408.97, MAPE=28.50, R2=0.702
RICE | Naive | Fold 3: MAE=261.94, RMSE=354.96, MAPE=22.04, R2=0.808
RICE | Ridge | Fold 1: MAE=370.17, RMSE=471.12, MAPE=52.54, R2=0.577
RICE | Ridge | Fold 2: MAE=267.07, RMSE=361.38, MAPE=30.33, R2=0.767
RICE | Ridge | Fold 3: MAE=212.04, RMSE=294.96, MAPE=19.37, R2=0.867


...

COTTON | RandomForest | Fold 1: MAE=52.70, RMSE=96.63, MAPE=24.54, R2=0.476
COTTON | RandomForest | Fold 2: MAE=86.45, RMSE=135.30, MAPE=33.36, R2=0.211
COTTON | RandomForest | Fold 3: MAE=73.51, RMSE=114.34, MAPE=24.60, R2=0.415
COTTON | GradientBoosting | Fold 1: MAE=55.39, RMSE=99.40, MAPE=26.54, R2=0.446
COTTON | GradientBoosting | Fold 2: MAE=87.69, RMSE=138.42, MAPE=34.47, R2=0.175
COTTON | GradientBoosting | Fold 3: MAE=75.93, RMSE=119.23, MAPE=25.17, R2=0.364

Performance summary for COTTON:
     Crop             Model   MAE_mean    MAE_std   RMSE_mean   RMSE_std  \
1  COTTON             Ridge  70.797446  14.231711  110.286841  17.758815   
2  COTTON      RandomForest  70.888036  13.905589  115.423294  15.804578   
3  COTTON  GradientBoosting  73.003030  13.346880  119.018462  15.930482   
0  COTTON             Naive  81.238336  19.597172  137.557850  22.737742   

   MAPE_mean  MAPE_std   R2_mean    R2_std  
1  26.960121  3.625313  0.421916  0.128314  
2  27.498192  4.146366  

/tmp/ipython-input-2696958754.py:539: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sup = sub.groupby("Dist Code", group_keys=False).apply(process_group)


Saved supervised panel for SUGARCANE to: /content/drive/MyDrive/Shiny/ICRISAT/Results/supervised_panel_sugarcane.csv

===== MODELING SUGARCANE =====
SUGARCANE | Naive | Fold 1: MAE=991.85, RMSE=1428.87, MAPE=25.42, R2=0.712
SUGARCANE | Naive | Fold 2: MAE=958.69, RMSE=1437.96, MAPE=25.06, R2=0.710
SUGARCANE | Naive | Fold 3: MAE=937.47, RMSE=1436.10, MAPE=25.00, R2=0.690
SUGARCANE | Ridge | Fold 1: MAE=941.71, RMSE=1329.05, MAPE=25.50, R2=0.751
SUGARCANE | Ridge | Fold 2: MAE=890.85, RMSE=1256.90, MAPE=29.87, R2=0.779
SUGARCANE | Ridge | Fold 3: MAE=822.24, RMSE=1191.91, MAPE=30.83, R2=0.786


/usr/local/lib/python3.12/dist-packages/scipy/_lib/_util.py:1233: LinAlgWarning: Ill-conditioned matrix (rcond=2.86218e-27): result may not be accurate.
  return f(*arrays, *other_args, **kwargs)
/usr/local/lib/python3.12/dist-packages/scipy/_lib/_util.py:1233: LinAlgWarning: Ill-conditioned matrix (rcond=1.94711e-26): result may not be accurate.
  return f(*arrays, *other_args, **kwargs)
/usr/local/lib/python3.12/dist-packages/scipy/_lib/_util.py:1233: LinAlgWarning: Ill-conditioned matrix (rcond=7.79147e-26): result may not be accurate.
  return f(*arrays, *other_args, **kwargs)


SUGARCANE | RandomForest | Fold 1: MAE=888.80, RMSE=1261.09, MAPE=24.84, R2=0.776
SUGARCANE | RandomForest | Fold 2: MAE=880.86, RMSE=1273.30, MAPE=25.71, R2=0.773
SUGARCANE | RandomForest | Fold 3: MAE=813.81, RMSE=1187.51, MAPE=26.16, R2=0.788
SUGARCANE | GradientBoosting | Fold 1: MAE=943.62, RMSE=1331.84, MAPE=24.91, R2=0.750
SUGARCANE | GradientBoosting | Fold 2: MAE=860.54, RMSE=1254.25, MAPE=25.18, R2=0.779
SUGARCANE | GradientBoosting | Fold 3: MAE=817.84, RMSE=1175.57, MAPE=25.99, R2=0.792

Performance summary for SUGARCANE:
        Crop             Model    MAE_mean    MAE_std    RMSE_mean   RMSE_std  \
2  SUGARCANE      RandomForest  861.156849  33.637094  1240.635356  37.895316   
3  SUGARCANE  GradientBoosting  874.000138  52.224834  1253.885854  63.800912   
1  SUGARCANE             Ridge  884.932768  48.953727  1259.285276  56.016105   
0  SUGARCANE             Naive  962.668924  22.377003  1434.307049   3.920844   

   MAPE_mean  MAPE_std   R2_mean    R2_std  
2  25.569

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, median_absolute_error

from statsmodels.tsa.seasonal import STL

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# Global config
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Paths (adjust as needed in Colab)
DATA_PATH = "/content/drive/MyDrive/Shiny/ICRISAT/ICRISAT-District Level Data.csv"
OUTPUT_DIR = "/content/drive/MyDrive/Shiny/ICRISAT/Results"

# Network and CV configuration
NETWORK_WINDOW_YEARS = 10
N_SPLITS = 3
MIN_TRAIN_YEARS = 5       # smaller to allow CV within 2008–2017 window
TEST_WINDOW_YEARS = 2


def ensure_output_dir(path):
    if not os.path.exists(path):
        os.makedirs(path, exist_ok=True)


def detect_crop_columns(df):
    area_cols = [c for c in df.columns if "AREA (1000 ha)" in c]
    prod_cols = [c for c in df.columns if "PRODUCTION (1000 tons)" in c]
    yield_cols = [c for c in df.columns if "YIELD (Kg per ha)" in c]
    crops = {}
    for ycol in yield_cols:
        crop_name = ycol.replace(" YIELD (Kg per ha)", "").strip()
        area_col = f"{crop_name} AREA (1000 ha)"
        prod_col = f"{crop_name} PRODUCTION (1000 tons)"
        crops[crop_name] = {
            "yield": ycol,
            "area": area_col if area_col in df.columns else None,
            "prod": prod_col if prod_col in df.columns else None,
        }
    return crops, area_cols, prod_cols, yield_cols


def load_and_basic_eda(path):
    df = pd.read_csv(path)

    # Replace -1 with NaN in numeric columns
    num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    df[num_cols] = df[num_cols].replace(-1, np.nan)

    # Ensure Year as int
    if "Year" not in df.columns:
        raise ValueError("Year column not found in dataset.")
    df["Year"] = df["Year"].astype(int)

    # Uniqueness of (Dist Code, Year)
    if "Dist Code" not in df.columns:
        raise ValueError("Dist Code column not found in dataset.")
    dup_count = df.duplicated(subset=["Dist Code", "Year"]).sum()
    print(f"Duplicate (Dist Code, Year) rows: {dup_count}")
    print("Key (Dist Code, Year) is unique." if dup_count == 0 else "WARNING: Key (Dist Code, Year) is not unique!")

    # Basic EDA
    print("\n===== BASIC EDA =====")
    print(f"Rows: {len(df)}, Columns: {df.shape[1]}")
    print(f"Time range: {df['Year'].min()} - {df['Year'].max()}")
    if "State Name" in df.columns:
        print(f"Unique States: {df['State Name'].nunique()}")
    if "Dist Name" in df.columns:
        print(f"Unique Districts: {df['Dist Name'].nunique()}")

    crops, area_cols, prod_cols, yield_cols = detect_crop_columns(df)
    crop_names = list(crops.keys())
    print(f"Detected crops (count={len(crop_names)}): {crop_names[:20]}...")

    # Yield missingness
    yield_missing = []
    for cname, info in crops.items():
        ycol = info["yield"]
        missing_pct = df[ycol].isna().mean() * 100.0
        yield_missing.append((ycol, missing_pct))
    miss_df = pd.DataFrame(yield_missing, columns=["YieldColumn", "MissingPct"]).sort_values("MissingPct", ascending=False)
    print("\nYield missingness (% of NaN):")
    print(miss_df.head(20))

    # Summaries for some key crops
    for key in ["RICE", "WHEAT", "MAIZE", "GROUNDNUT", "COTTON", "SUGARCANE"]:
        if key in crops:
            ycol = crops[key]["yield"]
            print(f"\nSummary for {key} yield:")
            print(df[ycol].describe())

    # Simple plots
    ensure_output_dir(OUTPUT_DIR)
    for key in ["RICE", "WHEAT"]:
        if key in crops:
            ycol = crops[key]["yield"]
            tmp = df.groupby("Year")[ycol].mean().reset_index()
            plt.figure(figsize=(8, 4))
            plt.plot(tmp["Year"], tmp[ycol], marker="o")
            plt.title(f"Average {key} Yield Over Time")
            plt.xlabel("Year")
            plt.ylabel("Yield (Kg per ha)")
            plt.tight_layout()
            fname = os.path.join(OUTPUT_DIR, f"avg_yield_{key.lower()}_over_time.pdf")
            plt.savefig(fname)
            plt.close()

    if "RICE" in crops:
        ycol = crops["RICE"]["yield"]
        plt.figure(figsize=(6, 4))
        sns.histplot(df[ycol].dropna(), bins=40, kde=True)
        plt.title("Distribution of RICE Yield")
        plt.xlabel("Yield (Kg per ha)")
        plt.tight_layout()
        fname = os.path.join(OUTPUT_DIR, "rice_yield_distribution.pdf")
        plt.savefig(fname)
        plt.close()

    return df, crops, area_cols, prod_cols, yield_cols


def clean_data(df, crops):
    print("\n===== CLEANING DATA =====")
    df = df.copy()

    # Treat 0 yield with positive area as missing
    for crop, info in crops.items():
        ycol = info["yield"]
        acol = info["area"]
        pcol = info["prod"]
        if acol is not None and ycol in df.columns:
            mask = (df[acol] > 0) & (df[ycol] == 0)
            if mask.any():
                df.loc[mask, ycol] = np.nan
                if pcol is not None and pcol in df.columns:
                    df.loc[mask, pcol] = np.nan

    # If area <=0, set yield and prod to NaN
    for crop, info in crops.items():
        ycol = info["yield"]
        acol = info["area"]
        pcol = info["prod"]
        if acol is not None and ycol in df.columns:
            mask = (df[acol].fillna(0) <= 0)
            df.loc[mask, ycol] = np.nan
            if pcol is not None and pcol in df.columns:
                df.loc[mask, pcol] = np.nan

    # Clip outliers in yields
    for crop, info in crops.items():
        ycol = info["yield"]
        if ycol in df.columns:
            series = df[ycol].dropna()
            if len(series) < 10:
                continue
            p1 = series.quantile(0.01)
            p99 = series.quantile(0.99)
            old_max = series.max()
            df[ycol] = df[ycol].clip(lower=p1, upper=p99)
            new_max = df[ycol].max()
            print(f"Clipped {crop} yield: old max={old_max:.2f}, new max={new_max:.2f}")

    # Data quality: non-missing fraction by crop
    stats = []
    for crop, info in crops.items():
        ycol = info["yield"]
        nonmiss_frac = 1.0 - df[ycol].isna().mean()
        stats.append((crop, nonmiss_frac))
    qual_df = pd.DataFrame(stats, columns=["Crop", "NonMissingFrac"]).sort_values("NonMissingFrac", ascending=False)
    print("\nTop 15 crops by non-missing yield fraction:")
    print(qual_df.head(15))

    return df


def compute_diversification(df, area_cols):
    print("\n===== COMPUTING DIVERSIFICATION =====")
    df = df.copy()

    # Total cropped area
    df["TOTAL_AREA_1000_HA"] = df[area_cols].clip(lower=0).sum(axis=1, min_count=1)

    # Construct shares via concat to avoid fragmentation
    share_cols = []
    share_data = {}
    denom = df["TOTAL_AREA_1000_HA"].replace(0, np.nan)
    for col in area_cols:
        share_col = col.replace(" AREA (1000 ha)", " AREA_SHARE")
        share_data[share_col] = df[col].clip(lower=0) / denom
        share_cols.append(share_col)
    share_df = pd.DataFrame(share_data, index=df.index)
    share_df = share_df.fillna(0.0)
    df = pd.concat([df, share_df], axis=1)

    # Herfindahl and diversification
    df["HERFINDAHL"] = (df[share_cols] ** 2).sum(axis=1)
    df["DIVERSIFICATION_INDEX"] = 1.0 - df["HERFINDAHL"]

    # Shannon entropy and crop count
    shares_np = df[share_cols].values
    with np.errstate(divide="ignore", invalid="ignore"):
        log_shares = np.where(shares_np > 0, np.log(shares_np), 0.0)
    shannon = -np.sum(shares_np * log_shares, axis=1)
    active_counts = (shares_np > 0).sum(axis=1)
    evenness = np.zeros_like(shannon)
    mask = active_counts > 1
    evenness[mask] = shannon[mask] / np.log(active_counts[mask])
    df["SHANNON_ENTROPY"] = shannon
    df["CROP_COUNT_ACTIVE"] = active_counts
    df["ENTROPY_EVENNESS"] = evenness

    # Example plot for diversification index for few random districts
    if "Dist Code" in df.columns:
        example_codes = df["Dist Code"].dropna().unique()
        example_codes = example_codes[:5]
        plt.figure(figsize=(8, 4))
        for dc in example_codes:
            sub = df[df["Dist Code"] == dc].sort_values("Year")
            plt.plot(sub["Year"], sub["DIVERSIFICATION_INDEX"], alpha=0.6, label=str(dc))
        plt.title("Diversification Index over Time (sample districts)")
        plt.xlabel("Year")
        plt.ylabel("1 - Herfindahl")
        plt.tight_layout()
        fname = os.path.join(OUTPUT_DIR, "diversification_index_over_time_sample_districts.pdf")
        plt.savefig(fname)
        plt.close()

    return df, share_cols


def build_district_network(df, share_cols, k=5):
    print("\n===== BUILDING DISTRICT SIMILARITY NETWORK =====")

    # Use most recent NETWORK_WINDOW_YEARS for network computation
    max_year = df["Year"].max()
    start_win = max(max_year - NETWORK_WINDOW_YEARS + 1, df["Year"].min())
    print(f"Network reference window: {start_win}-{max_year}")
    win_df = df[(df["Year"] >= start_win) & (df["Year"] <= max_year)].copy()

    # Compute mean share vector per district
    group_cols = ["Dist Code"]
    mean_shares = win_df.groupby(group_cols)[share_cols].mean()

    # Filter districts with nonzero total area in window
    keep_mask = mean_shares.sum(axis=1) > 0
    mean_shares = mean_shares[keep_mask]
    districts = mean_shares.index.tolist()

    # Normalize share vectors
    X = mean_shares.values
    norms = np.linalg.norm(X, axis=1, keepdims=True)
    norms[norms == 0] = 1.0
    X_norm = X / norms

    # Build k-NN cosine similarity graph
    G = nx.Graph()
    for dc in districts:
        G.add_node(dc)
    n = len(districts)
    for i in range(n):
        sims = X_norm @ X_norm[i, :].T
        sims[i] = -1
        idx_sorted = np.argsort(sims)[::-1]
        neighbors_idx = idx_sorted[:k]
        for j in neighbors_idx:
            if sims[j] <= 0:
                continue
            u = districts[i]
            v = districts[j]
            w = float(sims[j])
            if G.has_edge(u, v):
                if w > G[u][v]["weight"]:
                    G[u][v]["weight"] = w
            else:
                G.add_edge(u, v, weight=w)

    print(f"District network: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")

    # Network metrics
    strength = {}
    for node in G.nodes():
        strength[node] = sum(d["weight"] for _, _, d in G.edges(node, data=True))
    try:
        betweenness = nx.betweenness_centrality(G, weight="weight", normalized=True)
    except Exception:
        betweenness = {n: 0.0 for n in G.nodes()}
    try:
        eigen = nx.eigenvector_centrality(G, weight="weight", max_iter=1000)
    except Exception:
        eigen = {n: 0.0 for n in G.nodes()}
    clustering = nx.clustering(G, weight="weight")
    avg_neighbor_deg = nx.average_neighbor_degree(G, weight="weight")

    # Community detection
    try:
        from networkx.algorithms import community
        comms = list(community.greedy_modularity_communities(G, weight="weight"))
        comm_map = {}
        for cid, comm in enumerate(comms):
            for node in comm:
                comm_map[node] = cid
    except Exception:
        comm_map = {n: -1 for n in G.nodes()}

    metrics_rows = []
    for node in G.nodes():
        metrics_rows.append(
            {
                "Dist Code": node,
                "NET_STRENGTH": strength.get(node, 0.0),
                "NET_BETWEENNESS": betweenness.get(node, 0.0),
                "NET_EIGENVECTOR": eigen.get(node, 0.0),
                "NET_CLUSTERING": clustering.get(node, 0.0),
                "NET_AVG_NBR_DEGREE": avg_neighbor_deg.get(node, 0.0),
                "NET_COMMUNITY": comm_map.get(node, -1),
            }
        )
    metrics_df = pd.DataFrame(metrics_rows)

    # Merge back to all years
    df = df.merge(metrics_df, on="Dist Code", how="left")
    return df, G, start_win, max_year


def build_crop_network(df, area_cols, share_cols):
    print("\n===== BUILDING CROP-COOCCURRENCE NETWORK =====")
    G = nx.Graph()

    # Crop names from area_cols
    crops = []
    for col in area_cols:
        if col.endswith(" AREA (1000 ha)"):
            cname = col.replace(" AREA (1000 ha)", "").strip()
            crops.append(cname)
    crops = sorted(set(crops))
    for c in crops:
        G.add_node(c)

    if not share_cols:
        print("No share columns found for crop network.")
        return G, pd.DataFrame()

    # Use same reference window as district network for consistency
    max_year = df["Year"].max()
    start_win = max(max_year - NETWORK_WINDOW_YEARS + 1, df["Year"].min())
    win_df = df[(df["Year"] >= start_win) & (df["Year"] <= max_year)].copy()

    # For each district, mark crop present if mean share > 0
    share_cols_by_crop = {}
    for col in share_cols:
        cname = col.replace(" AREA_SHARE", "").replace(" AREA (1000 ha)", "").strip()
        share_cols_by_crop.setdefault(cname, []).append(col)

    present = {c: set() for c in crops}
    if "Dist Code" not in df.columns:
        return G, pd.DataFrame()

    for dc, sub in win_df.groupby("Dist Code"):
        for c in crops:
            cols = share_cols_by_crop.get(c, [])
            if not cols:
                continue
            mean_share = sub[cols].mean().sum()
            if mean_share > 0:
                present[c].add(dc)

    for i, ci in enumerate(crops):
        for cj in crops[i+1:]:
            common = len(present[ci].intersection(present[cj]))
            if common > 0:
                G.add_edge(ci, cj, weight=float(common))

    print(f"Crop network: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")

    # Simple crop metrics
    try:
        eigen = nx.eigenvector_centrality(G, weight="weight", max_iter=1000)
    except Exception:
        eigen = {n: 0.0 for n in G.nodes()}
    degree = dict(G.degree(weight="weight"))
    crop_metrics = pd.DataFrame(
        [
            {"Crop": c, "CROP_DEGREE": degree.get(c, 0.0), "CROP_EIGENVECTOR": eigen.get(c, 0.0)}
            for c in crops
        ]
    )
    return G, crop_metrics


def add_interaction_features(df):
    df = df.copy()
    if "DIVERSIFICATION_INDEX" in df.columns and "NET_EIGENVECTOR" in df.columns:
        df["DIV_NET_EIGEN"] = df["DIVERSIFICATION_INDEX"] * df["NET_EIGENVECTOR"]
    if "SHANNON_ENTROPY" in df.columns and "NET_STRENGTH" in df.columns:
        df["ENTROPY_NET_STRENGTH"] = df["SHANNON_ENTROPY"] * df["NET_STRENGTH"]
    if "TOTAL_AREA_1000_HA" in df.columns and "NET_STRENGTH" in df.columns:
        df["AREA_NET_STRENGTH"] = df["TOTAL_AREA_1000_HA"] * df["NET_STRENGTH"]
    return df


def rolling_stl_features(series, period=10, min_len=5):
    # Leakage-free rolling STL: at each time t, use data up to t only
    series = series.sort_index()
    trend = pd.Series(index=series.index, dtype=float)
    resid = pd.Series(index=series.index, dtype=float)
    for i, (idx, _) in enumerate(series.items()):  # .items() instead of deprecated .iteritems()
        sub = series.iloc[: i + 1]
        if sub.notna().sum() < min_len:
            trend.iloc[i] = sub.iloc[-1]
            resid.iloc[i] = 0.0
        else:
            try:
                stl = STL(sub, period=min(period, len(sub)), robust=True)
                res = stl.fit()
                trend.iloc[i] = res.trend.iloc[-1]
                resid.iloc[i] = res.resid.iloc[-1]
            except Exception:
                trend.iloc[i] = sub.iloc[-1]
                resid.iloc[i] = 0.0
    return trend, resid


def construct_supervised_dataset_for_crop(df, crop, crop_info, model_min_year=None):
    print(f"\n===== BUILDING SUPERVISED DATA FOR {crop} =====")
    ycol = crop_info["yield"]
    acol = crop_info["area"]
    pcol = crop_info["prod"]

    use_df = df.copy()
    if model_min_year is not None:
        use_df = use_df[use_df["Year"] >= model_min_year].copy()

    cols = ["Dist Code", "Year", "State Name", "Dist Name", ycol]
    if acol is not None:
        cols.append(acol)
    if pcol is not None:
        cols.append(pcol)
    for extra in ["DIVERSIFICATION_INDEX", "HERFINDAHL", "SHANNON_ENTROPY", "CROP_COUNT_ACTIVE", "ENTROPY_EVENNESS",
                  "TOTAL_AREA_1000_HA", "NET_STRENGTH", "NET_BETWEENNESS", "NET_EIGENVECTOR",
                  "NET_CLUSTERING", "NET_AVG_NBR_DEGREE", "NET_COMMUNITY",
                  "DIV_NET_EIGEN", "ENTROPY_NET_STRENGTH", "AREA_NET_STRENGTH"]:
        if extra in use_df.columns:
            cols.append(extra)

    sub = use_df[cols].dropna(subset=["Dist Code", "Year"])
    sub = sub.sort_values(["Dist Code", "Year"]).reset_index(drop=True)

    def process_group(g):
        g = g.sort_values("Year").copy()
        y = g[ycol]

        # Lag features
        g[f"{crop}_Y_LAG1"] = y.shift(1)
        g[f"{crop}_Y_LAG2"] = y.shift(2)

        # Rolling stats
        g[f"{crop}_Y_ROLL_MEAN3"] = y.rolling(window=3, min_periods=3).mean()
        g[f"{crop}_Y_ROLL_STD3"] = y.rolling(window=3, min_periods=3).std()
        g[f"{crop}_Y_ROLL_CV3"] = g[f"{crop}_Y_ROLL_STD3"] / (g[f"{crop}_Y_ROLL_MEAN3"].abs() + 1e-6)

        # Relative changes and shock flags
        g[f"{crop}_Y_DIFF1"] = y - y.shift(1)
        g[f"{crop}_Y_REL_CHANGE1"] = g[f"{crop}_Y_DIFF1"] / (y.shift(1).abs() + 1e-6)
        g[f"{crop}_Y_SHOCK_FLAG"] = (g[f"{crop}_Y_REL_CHANGE1"].abs() > 0.3).astype(float)

        if acol is not None and acol in g.columns:
            a = g[acol]
            g[f"{crop}_A_LAG1"] = a.shift(1)
            g[f"{crop}_A_DIFF1"] = a - a.shift(1)
            g[f"{crop}_A_REL_CHANGE1"] = g[f"{crop}_A_DIFF1"] / (a.shift(1).abs() + 1e-6)
        if pcol is not None and pcol in g.columns:
            p = g[pcol]
            g[f"{crop}_P_LAG1"] = p.shift(1)

        # Time index
        year0 = g["Year"].min()
        g["YEAR_CENTERED"] = g["Year"] - year0
        g["YEAR_CENTERED_SQ"] = g["YEAR_CENTERED"] ** 2

        # STL decomposition features on yield
        y_series = g.set_index("Year")[ycol].dropna()
        if len(y_series) > 0:
            trend, resid = rolling_stl_features(y_series, period=10, min_len=5)
            g = g.set_index("Year")
            g[f"{crop}_STL_TREND"] = trend
            g[f"{crop}_STL_RESID"] = resid
            g = g.reset_index()
        else:
            g[f"{crop}_STL_TREND"] = np.nan
            g[f"{crop}_STL_RESID"] = np.nan

        # Target: next-year yield
        g["TARGET_YIELD_NEXT"] = y.shift(-1)
        return g

    sup = sub.groupby("Dist Code", group_keys=False).apply(process_group)
    sup = sup.dropna(subset=["TARGET_YIELD_NEXT",
                             f"{crop}_Y_LAG1", f"{crop}_Y_LAG2",
                             f"{crop}_Y_ROLL_MEAN3", f"{crop}_Y_ROLL_STD3",
                             f"{crop}_Y_ROLL_CV3"])
    sup = sup.dropna(axis=0, how="any", subset=[ycol])

    out_path = os.path.join(OUTPUT_DIR, f"supervised_panel_{crop.lower()}.csv")
    sup.to_csv(out_path, index=False)
    print(f"Saved supervised panel for {crop} to: {out_path}")
    return sup, ycol


def time_series_cv_splits(years, n_splits=3, min_train_years=5, test_window_years=2):
    unique_years = np.sort(years.unique())
    if len(unique_years) < (min_train_years + test_window_years):
        raise ValueError("Not enough years to build the requested time-series CV splits.")

    splits = []
    start_idx = 0
    for split_id in range(n_splits):
        train_end_idx = min(start_idx + min_train_years + split_id * test_window_years - 1,
                            len(unique_years) - test_window_years - 1)
        train_end_year = unique_years[train_end_idx]
        test_start_idx = train_end_idx + 1
        test_end_idx = min(test_start_idx + test_window_years - 1, len(unique_years) - 1)
        test_start_year = unique_years[test_start_idx]
        test_end_year = unique_years[test_end_idx]

        train_mask = years <= train_end_year
        test_mask = (years >= test_start_year) & (years <= test_end_year)

        if train_mask.sum() == 0 or test_mask.sum() == 0:
            continue
        splits.append((np.where(train_mask)[0], np.where(test_mask)[0]))
    return splits


def compute_metrics(y_true, y_pred, mape_threshold=100.0):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    # MAPE only on sufficiently large true values
    mask = np.abs(y_true) >= mape_threshold
    if mask.any():
        mape = np.mean(np.abs((y_true[mask] - y_pred[mask]) /
                              np.clip(np.abs(y_true[mask]), 1e-6, None))) * 100.0
    else:
        mape = np.nan
    r2 = r2_score(y_true, y_pred)
    medae = median_absolute_error(y_true, y_pred)
    return mae, rmse, mape, r2, medae


def evaluate_models_for_crop(sup, crop, yield_col, include_network_features=True, feature_set_name="with_network"):
    print(f"\n===== MODELING {crop} | Feature set: {feature_set_name} =====")
    sup = sup.copy()

    # Define feature columns
    exclude_cols = ["TARGET_YIELD_NEXT", yield_col, "Dist Code", "State Name", "Dist Name"]
    feature_cols = [c for c in sup.columns if c not in exclude_cols]

    # Drop non-numeric features
    non_numeric = sup[feature_cols].select_dtypes(exclude=[np.number]).columns.tolist()
    feature_cols = [c for c in feature_cols if c not in non_numeric]

    # Remove network-based features if doing ablation
    if not include_network_features:
        feature_cols = [c for c in feature_cols if not (c.startswith("NET_") or c.startswith("CROP_") or c.startswith("DIV_NET"))]

    if not feature_cols:
        print(f"No usable numeric features for {crop} with feature set {feature_set_name}.")
        return pd.DataFrame()

    X = sup[feature_cols].values
    years = sup["Year"].values
    y = sup["TARGET_YIELD_NEXT"].values

    try:
        splits = time_series_cv_splits(pd.Series(years), n_splits=N_SPLITS,
                                       min_train_years=MIN_TRAIN_YEARS,
                                       test_window_years=TEST_WINDOW_YEARS)
    except ValueError as e:
        print(f"Time-series CV error for {crop} ({feature_set_name}): {e}")
        return pd.DataFrame()

    if not splits:
        print(f"Not enough temporal coverage for time-series CV for {crop}.")
        return pd.DataFrame()

    models = {
        "Naive": None,
        "RollingMean3": None,
        "Ridge": Pipeline([
            ("scaler", RobustScaler()),
            ("model", Ridge(alpha=5.0, random_state=RANDOM_STATE))
        ]),
        "RandomForest": Pipeline([
            ("scaler", RobustScaler()),
            ("model", RandomForestRegressor(
                n_estimators=200,
                max_depth=7,
                random_state=RANDOM_STATE,
                n_jobs=-1
            ))
        ]),
        "GradientBoosting": Pipeline([
            ("scaler", RobustScaler()),
            ("model", GradientBoostingRegressor(
                n_estimators=300,
                learning_rate=0.05,
                max_depth=3,
                random_state=RANDOM_STATE
            ))
        ]),
    }

    records = []
    for model_name, model in models.items():
        fold_metrics = []
        for fold_id, (train_idx, test_idx) in enumerate(splits, start=1):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]
            sup_test = sup.iloc[test_idx]

            if model_name == "Naive":
                # Predict next-year yield as current-year yield
                y_pred = sup_test[yield_col].values
            elif model_name == "RollingMean3":
                # Predict next-year as mean of last 3 years
                y_t = sup_test[yield_col].values
                y_lag1 = sup_test[f"{crop}_Y_LAG1"].values
                y_lag2 = sup_test[f"{crop}_Y_LAG2"].values
                stack = np.vstack([y_t, y_lag1, y_lag2])
                y_pred = np.nanmean(stack, axis=0)
            else:
                model.fit(X_train, y_train)
                y_pred = model.predict(X_test)

            mae, rmse, mape, r2, medae = compute_metrics(y_test, y_pred)
            fold_metrics.append((mae, rmse, mape, r2, medae))
            print(f"{crop} | {model_name} | {feature_set_name} | Fold {fold_id}: MAE={mae:.2f}, RMSE={rmse:.2f}, MAPE={mape:.2f}, MedAE={medae:.2f}, R2={r2:.3f}")

        if not fold_metrics:
            continue
        maes, rmses, mapes, r2s, medaes = zip(*fold_metrics)
        records.append({
            "Crop": crop,
            "Model": model_name,
            "FeatureSet": feature_set_name,
            "MAE_mean": float(np.mean(maes)),
            "MAE_std": float(np.std(maes)),
            "RMSE_mean": float(np.mean(rmses)),
            "RMSE_std": float(np.std(rmses)),
            "MAPE_mean": float(np.nanmean(mapes)),
            "MAPE_std": float(np.nanstd(mapes)),
            "MedAE_mean": float(np.mean(medaes)),
            "MedAE_std": float(np.std(medaes)),
            "R2_mean": float(np.mean(r2s)),
            "R2_std": float(np.std(r2s)),
        })

    perf_df = pd.DataFrame(records)
    if not perf_df.empty:
        print(f"\nPerformance summary for {crop} ({feature_set_name}):")
        print(perf_df.sort_values("MAE_mean").reset_index(drop=True))
    return perf_df


def plot_district_network(G):
    if G is None or G.number_of_nodes() == 0:
        return
    plt.figure(figsize=(8, 6))
    pos = nx.spring_layout(G, seed=RANDOM_STATE, k=0.15)
    degrees = dict(G.degree())
    node_sizes = np.array(list(degrees.values()), dtype=float)
    node_sizes = 100 + 300 * (node_sizes - node_sizes.min()) / (node_sizes.max() - node_sizes.min() + 1e-6)
    nx.draw_networkx_nodes(G, pos, node_size=node_sizes, alpha=0.7)
    nx.draw_networkx_edges(G, pos, width=0.3, alpha=0.3)
    plt.axis("off")
    plt.title("District Similarity Network (spring layout)")
    plt.tight_layout()
    fname = os.path.join(OUTPUT_DIR, "district_network_overview.pdf")
    plt.savefig(fname)
    plt.close()


def diagnostic_plots_for_crop(sup, crop, yield_col, best_model_name, include_network_features=True):
    print(f"\n===== DIAGNOSTIC PLOTS FOR {crop} ({best_model_name}, feature set={'with' if include_network_features else 'no'} network) =====")
    sup = sup.copy()

    exclude_cols = ["TARGET_YIELD_NEXT", yield_col, "Dist Code", "State Name", "Dist Name"]
    feature_cols = [c for c in sup.columns if c not in exclude_cols]
    non_numeric = sup[feature_cols].select_dtypes(exclude=[np.number]).columns.tolist()
    feature_cols = [c for c in feature_cols if c not in non_numeric]
    if not include_network_features:
        feature_cols = [c for c in feature_cols if not (c.startswith("NET_") or c.startswith("CROP_") or c.startswith("DIV_NET"))]
    if not feature_cols:
        print("No features available for diagnostics.")
        return

    X = sup[feature_cols].values
    years = sup["Year"].values
    y = sup["TARGET_YIELD_NEXT"].values
    try:
        splits = time_series_cv_splits(pd.Series(years), n_splits=N_SPLITS,
                                       min_train_years=MIN_TRAIN_YEARS,
                                       test_window_years=TEST_WINDOW_YEARS)
    except ValueError as e:
        print(f"Time-series CV error for diagnostics ({crop}): {e}")
        return

    if not splits:
        print("Not enough temporal coverage for diagnostics.")
        return

    # Use last split as "final validation"
    train_idx, test_idx = splits[-1]
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    sup_test = sup.iloc[test_idx]

    if best_model_name == "Naive":
        y_pred = sup_test[yield_col].values
    elif best_model_name == "RollingMean3":
        y_t = sup_test[yield_col].values
        y_lag1 = sup_test[f"{crop}_Y_LAG1"].values
        y_lag2 = sup_test[f"{crop}_Y_LAG2"].values
        stack = np.vstack([y_t, y_lag1, y_lag2])
        y_pred = np.nanmean(stack, axis=0)
    else:
        if best_model_name == "Ridge":
            model = Pipeline([
                ("scaler", RobustScaler()),
                ("model", Ridge(alpha=5.0, random_state=RANDOM_STATE))
            ])
        elif best_model_name == "RandomForest":
            model = Pipeline([
                ("scaler", RobustScaler()),
                ("model", RandomForestRegressor(
                    n_estimators=200,
                    max_depth=7,
                    random_state=RANDOM_STATE,
                    n_jobs=-1
                ))
            ])
        else:
            model = Pipeline([
                ("scaler", RobustScaler()),
                ("model", GradientBoostingRegressor(
                    n_estimators=300,
                    learning_rate=0.05,
                    max_depth=3,
                    random_state=RANDOM_STATE
                ))
            ])
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

    # Scatter true vs predicted
    plt.figure(figsize=(6, 6))
    plt.scatter(y_test, y_pred, alpha=0.4)
    lims = [min(y_test.min(), y_pred.min()), max(y_test.max(), y_pred.max())]
    plt.plot(lims, lims, "r--", linewidth=1)
    plt.xlabel("True Yield")
    plt.ylabel("Predicted Yield")
    plt.title(f"{crop}: True vs Predicted Yield ({best_model_name})")
    plt.tight_layout()
    fname = os.path.join(OUTPUT_DIR, f"{crop.lower()}_yield_true_vs_pred_{best_model_name}.pdf")
    plt.savefig(fname)
    plt.close()

    # Time-series for a few example districts
    example_dists = sup_test["Dist Code"].dropna().unique()[:3]
    for dc in example_dists:
        sub_idx = sup_test["Dist Code"] == dc
        sub_y_true = y_test[sub_idx]
        sub_y_pred = y_pred[sub_idx]
        sub_years = sup_test.loc[sub_idx, "Year"].values
        order = np.argsort(sub_years)
        plt.figure(figsize=(8, 4))
        plt.plot(sub_years[order], sub_y_true[order], marker="o", label="True")
        plt.plot(sub_years[order], sub_y_pred[order], marker="x", label="Predicted")
        plt.title(f"{crop} Yield over Time (Dist Code={dc})")
        plt.xlabel("Year")
        plt.ylabel("Yield")
        plt.legend()
        plt.tight_layout()
        fname = os.path.join(OUTPUT_DIR, f"{crop.lower()}_yield_timeseries_dist_{dc}_{best_model_name}.pdf")
        plt.savefig(fname)
        plt.close()


def main():
    ensure_output_dir(OUTPUT_DIR)

    df, crops, area_cols, prod_cols, yield_cols = load_and_basic_eda(DATA_PATH)
    df = clean_data(df, crops)
    df, share_cols = compute_diversification(df, area_cols)
    df, district_graph, net_start_year, net_end_year = build_district_network(df, share_cols, k=5)
    crop_graph, crop_metrics = build_crop_network(df, area_cols, share_cols)
    df = add_interaction_features(df)
    plot_district_network(district_graph)

    # Restrict modeling window to avoid temporal leakage from network features
    model_min_year = net_start_year

    # Major crops
    target_crops = ["RICE", "WHEAT", "MAIZE", "GROUNDNUT", "COTTON", "SUGARCANE"]
    all_perf = []
    best_models_per_crop = {}
    modeled_crops = []

    # Save cleaned dataset with features
    cleaned_path = os.path.join(OUTPUT_DIR, "icrisat_cleaned_with_features.csv")
    df.to_csv(cleaned_path, index=False)
    print(f"\nSaved cleaned dataset with features to: {cleaned_path}")

    for crop in target_crops:
        if crop not in crops:
            print(f"Crop {crop} not found in dataset; skipping.")
            continue

        sup, ycol = construct_supervised_dataset_for_crop(df, crop, crops[crop], model_min_year=model_min_year)
        if sup.empty:
            print(f"No supervised data for {crop}; skipping modeling.")
            continue

        modeled_crops.append(crop)

        # Evaluate without network features (ablation)
        perf_no_net = evaluate_models_for_crop(sup, crop, ycol,
                                               include_network_features=False,
                                               feature_set_name="no_network")
        # Evaluate with full feature set (including network)
        perf_with_net = evaluate_models_for_crop(sup, crop, ycol,
                                                 include_network_features=True,
                                                 feature_set_name="with_network")
        perf_crop = pd.concat([perf_no_net, perf_with_net], ignore_index=True)
        all_perf.append(perf_crop)

        # Select best model based on MAE_mean (across all feature sets)
        if not perf_crop.empty:
            best_row = perf_crop.sort_values("MAE_mean").iloc[0]
            best_models_per_crop[crop] = (best_row["Model"], best_row["FeatureSet"])
            print(f"Best model for {crop}: {best_row['Model']} (FeatureSet={best_row['FeatureSet']}), MAE_mean={best_row['MAE_mean']:.2f}")

        # Diagnostics for the best configuration
        if crop in best_models_per_crop:
            best_model_name, best_feat_set = best_models_per_crop[crop]
            include_net = (best_feat_set == "with_network")
            diagnostic_plots_for_crop(sup, crop, ycol, best_model_name, include_network_features=include_net)

    if all_perf:
        perf_all = pd.concat(all_perf, ignore_index=True)
        perf_path = os.path.join(OUTPUT_DIR, "model_performance_summary.csv")
        perf_all.to_csv(perf_path, index=False)
        print(f"\nSaved model performance summary to: {perf_path}")
    else:
        perf_all = pd.DataFrame()
        perf_path = ""

    # Final summary
    print("\n===== FINAL SUMMARY =====")
    print(f"Rows after cleaning: {len(df)}")
    print(f"Years: {df['Year'].min()} - {df['Year'].max()}")
    if 'Dist Name' in df.columns:
        print(f"Number of districts: {df['Dist Name'].nunique()}")
    else:
        print(f"Number of districts: {df['Dist Code'].nunique()}")
    print(f"Crops modeled: {modeled_crops}")
    if best_models_per_crop:
        print("Best model per crop (by MAE_mean across feature sets):")
        for crop, (model_name, feat_set) in best_models_per_crop.items():
            print(f"  {crop}: {model_name} (FeatureSet={feat_set})")
    print(f"Cleaned dataset with features: {cleaned_path}")
    if not perf_all.empty:
        print(f"Model performance summary: {perf_path}")
    print(f"Supervised panel CSV prefix: {os.path.join(OUTPUT_DIR, 'supervised_panel_')}")
    print("Key PDF plots saved in OUTPUT_DIR (yield trends, distributions, network overview, diagnostics).")


if __name__ == "__main__":
    main()


Duplicate (Dist Code, Year) rows: 0
Key (Dist Code, Year) is unique.

===== BASIC EDA =====
Rows: 16146, Columns: 80
Time range: 1966 - 2017
Unique States: 20
Unique Districts: 311
Detected crops (count=23): ['RICE', 'WHEAT', 'KHARIF SORGHUM', 'RABI SORGHUM', 'SORGHUM', 'PEARL MILLET', 'MAIZE', 'FINGER MILLET', 'BARLEY', 'CHICKPEA', 'PIGEONPEA', 'MINOR PULSES', 'GROUNDNUT', 'SESAMUM', 'RAPESEED AND MUSTARD', 'SAFFLOWER', 'CASTOR', 'LINSEED', 'SUNFLOWER', 'SOYABEAN']...

Yield missingness (% of NaN):
                         YieldColumn  MissingPct
20        OILSEEDS YIELD (Kg per ha)   10.361699
11    MINOR PULSES YIELD (Kg per ha)    8.794748
15       SAFFLOWER YIELD (Kg per ha)    2.991453
17         LINSEED YIELD (Kg per ha)    1.591726
21       SUGARCANE YIELD (Kg per ha)    1.492630
16          CASTOR YIELD (Kg per ha)    0.514059
5     PEARL MILLET YIELD (Kg per ha)    0.334448
13         SESAMUM YIELD (Kg per ha)    0.272513
9         CHICKPEA YIELD (Kg per ha)    0.260126
19   

/tmp/ipython-input-2510352981.py:510: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sup = sub.groupby("Dist Code", group_keys=False).apply(process_group)


Saved supervised panel for RICE to: /content/drive/MyDrive/Shiny/ICRISAT/Results/supervised_panel_rice.csv

===== MODELING RICE | Feature set: no_network =====
RICE | Naive | no_network | Fold 1: MAE=963.02, RMSE=964.12, MAPE=63.41, MedAE=963.02, R2=-3.421
RICE | Naive | no_network | Fold 2: MAE=963.02, RMSE=964.12, MAPE=63.41, MedAE=963.02, R2=-3.421
RICE | Naive | no_network | Fold 3: MAE=963.02, RMSE=964.12, MAPE=63.41, MedAE=963.02, R2=-3.421
RICE | RollingMean3 | no_network | Fold 1: MAE=542.54, RMSE=548.48, MAPE=34.70, MedAE=542.54, R2=-0.431
RICE | RollingMean3 | no_network | Fold 2: MAE=542.54, RMSE=548.48, MAPE=34.70, MedAE=542.54, R2=-0.431
RICE | RollingMean3 | no_network | Fold 3: MAE=542.54, RMSE=548.48, MAPE=34.70, MedAE=542.54, R2=-0.431
RICE | Ridge | no_network | Fold 1: MAE=462.00, RMSE=467.58, MAPE=32.19, MedAE=462.00, R2=-0.040
RICE | Ridge | no_network | Fold 2: MAE=462.00, RMSE=467.58, MAPE=32.19, MedAE=462.00, R2=-0.040
RICE | Ridge | no_network | Fold 3: MAE=462

/tmp/ipython-input-2510352981.py:510: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sup = sub.groupby("Dist Code", group_keys=False).apply(process_group)


Saved supervised panel for WHEAT to: /content/drive/MyDrive/Shiny/ICRISAT/Results/supervised_panel_wheat.csv

===== MODELING WHEAT | Feature set: no_network =====
WHEAT | Naive | no_network | Fold 1: MAE=126.29, RMSE=127.37, MAPE=10.41, MedAE=126.29, R2=-4.385
WHEAT | Naive | no_network | Fold 2: MAE=126.29, RMSE=127.37, MAPE=10.41, MedAE=126.29, R2=-4.385
WHEAT | Naive | no_network | Fold 3: MAE=126.29, RMSE=127.37, MAPE=10.41, MedAE=126.29, R2=-4.385
WHEAT | RollingMean3 | no_network | Fold 1: MAE=91.64, RMSE=113.41, MAPE=7.35, MedAE=91.64, R2=-3.269
WHEAT | RollingMean3 | no_network | Fold 2: MAE=91.64, RMSE=113.41, MAPE=7.35, MedAE=91.64, R2=-3.269
WHEAT | RollingMean3 | no_network | Fold 3: MAE=91.64, RMSE=113.41, MAPE=7.35, MedAE=91.64, R2=-3.269
WHEAT | Ridge | no_network | Fold 1: MAE=160.05, RMSE=166.05, MAPE=13.10, MedAE=160.05, R2=-8.152
WHEAT | Ridge | no_network | Fold 2: MAE=160.05, RMSE=166.05, MAPE=13.10, MedAE=160.05, R2=-8.152
WHEAT | Ridge | no_network | Fold 3: MAE=

...

Saved supervised panel for GROUNDNUT to: /content/drive/MyDrive/Shiny/ICRISAT/Results/supervised_panel_groundnut.csv

===== MODELING GROUNDNUT | Feature set: no_network =====
GROUNDNUT | Naive | no_network | Fold 1: MAE=314.83, RMSE=315.32, MAPE=16.85, MedAE=314.83, R2=-3.500
GROUNDNUT | Naive | no_network | Fold 2: MAE=314.83, RMSE=315.32, MAPE=16.85, MedAE=314.83, R2=-3.500
GROUNDNUT | Naive | no_network | Fold 3: MAE=314.83, RMSE=315.32, MAPE=16.85, MedAE=314.83, R2=-3.500
GROUNDNUT | RollingMean3 | no_network | Fold 1: MAE=182.43, RMSE=222.29, MAPE=9.27, MedAE=182.43, R2=-1.237
GROUNDNUT | RollingMean3 | no_network | Fold 2: MAE=182.43, RMSE=222.29, MAPE=9.27, MedAE=182.43, R2=-1.237
GROUNDNUT | RollingMean3 | no_network | Fold 3: MAE=182.43, RMSE=222.29, MAPE=9.27, MedAE=182.43, R2=-1.237
GROUNDNUT | Ridge | no_network | Fold 1: MAE=285.04, RMSE=340.49, MAPE=14.53, MedAE=285.04, R2=-4.248
GROUNDNUT | Ridge | no_network | Fold 2: MAE=285.04, RMSE=340.49, MAPE=14.53, MedAE=285.04, R

/tmp/ipython-input-2510352981.py:510: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sup = sub.groupby("Dist Code", group_keys=False).apply(process_group)


Saved supervised panel for COTTON to: /content/drive/MyDrive/Shiny/ICRISAT/Results/supervised_panel_cotton.csv

===== MODELING COTTON | Feature set: no_network =====
Time-series CV error for COTTON (no_network): Not enough years to build the requested time-series CV splits.

===== MODELING COTTON | Feature set: with_network =====
Time-series CV error for COTTON (with_network): Not enough years to build the requested time-series CV splits.

===== BUILDING SUPERVISED DATA FOR SUGARCANE =====


/tmp/ipython-input-2510352981.py:510: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sup = sub.groupby("Dist Code", group_keys=False).apply(process_group)


Saved supervised panel for SUGARCANE to: /content/drive/MyDrive/Shiny/ICRISAT/Results/supervised_panel_sugarcane.csv

===== MODELING SUGARCANE | Feature set: no_network =====
SUGARCANE | Naive | no_network | Fold 1: MAE=89.38, RMSE=89.38, MAPE=26.37, MedAE=89.38, R2=-3.000
SUGARCANE | Naive | no_network | Fold 2: MAE=89.38, RMSE=89.38, MAPE=26.37, MedAE=89.38, R2=-3.000
SUGARCANE | Naive | no_network | Fold 3: MAE=89.38, RMSE=89.38, MAPE=26.37, MedAE=89.38, R2=-3.000
SUGARCANE | RollingMean3 | no_network | Fold 1: MAE=70.93, RMSE=92.64, MAPE=23.21, MedAE=70.93, R2=-3.297
SUGARCANE | RollingMean3 | no_network | Fold 2: MAE=70.93, RMSE=92.64, MAPE=23.21, MedAE=70.93, R2=-3.297
SUGARCANE | RollingMean3 | no_network | Fold 3: MAE=70.93, RMSE=92.64, MAPE=23.21, MedAE=70.93, R2=-3.297
SUGARCANE | Ridge | no_network | Fold 1: MAE=272.56, RMSE=290.29, MAPE=84.25, MedAE=272.56, R2=-41.194
SUGARCANE | Ridge | no_network | Fold 2: MAE=272.56, RMSE=290.29, MAPE=84.25, MedAE=272.56, R2=-41.194
SUGA

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

from itertools import combinations
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# ----------------------------------------------------------------------
# CONFIG
# ----------------------------------------------------------------------

DATA_PATH = "/content/drive/MyDrive/Shiny/ICRISAT/ICRISAT-District Level Data.csv"
OUTPUT_DIR = "/content/drive/MyDrive/Shiny/ICRISAT/Results1"

os.makedirs(OUTPUT_DIR, exist_ok=True)

RANDOM_STATE = 42
PLOT_DPI = 150

# ----------------------------------------------------------------------
# UTILS
# ----------------------------------------------------------------------

def compute_regression_metrics(y_true, y_pred):
    """Return MAE, RMSE, MAPE (%), median AE, R2."""
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    # Safe MAPE (ignore zeros in denominator)
    denom = np.where(y_true == 0, np.nan, np.abs(y_true))
    mape = np.nanmean(np.abs(y_true - y_pred) / denom) * 100
    medae = np.median(np.abs(y_true - y_pred))
    try:
        r2 = r2_score(y_true, y_pred)
    except ValueError:
        r2 = np.nan
    return mae, rmse, mape, medae, r2

# ----------------------------------------------------------------------
# DATA LOADING & EDA
# ----------------------------------------------------------------------

def load_data(path):
    df = pd.read_csv(path)
    if "Year" not in df.columns:
        raise ValueError("Expected 'Year' column in data.")
    if "Dist Code" not in df.columns:
        raise ValueError("Expected 'Dist Code' column in data.")
    df["Year"] = df["Year"].astype(int)
    return df

def get_yield_columns_and_crops(df):
    """Detect yield columns and infer crop names from them."""
    yield_cols = [c for c in df.columns if "YIELD" in c.upper()]
    crop_to_yield = {}
    for col in yield_cols:
        # Example: "RICE YIELD (Kg per ha)" -> "RICE"
        up = col.upper()
        crop_part = up.split("YIELD")[0]
        crop_part = crop_part.replace("(KG PER HA)", "")
        crop_part = crop_part.strip()
        crop_name = " ".join(crop_part.split())
        crop_to_yield[crop_name] = col
    crops = sorted(crop_to_yield.keys())
    return yield_cols, crops, crop_to_yield

def infer_area_columns(df, crop_to_yield):
    """Map each crop to its likely AREA column."""
    crop_to_area = {}
    for crop in crop_to_yield.keys():
        up_crop = crop.upper()
        candidates = [c for c in df.columns if up_crop in c.upper() and "AREA" in c.upper()]
        if candidates:
            # pick the first candidate
            crop_to_area[crop] = candidates[0]
    return crop_to_area

def basic_eda(df):
    print("Duplicate (Dist Code, Year) rows:", df.duplicated(subset=["Dist Code", "Year"]).sum())
    if df.duplicated(subset=["Dist Code", "Year"]).sum() == 0:
        print("Key (Dist Code, Year) is unique.\n")

    print("===== BASIC EDA =====")
    print(f"Rows: {len(df):d}, Columns: {df.shape[1]:d}")
    print(f"Time range: {df['Year'].min()} - {df['Year'].max()}")
    if "State Name" in df.columns:
        print(f"Unique States: {df['State Name'].nunique()}")
    if "Dist Code" in df.columns:
        print(f"Unique Districts: {df['Dist Code'].nunique()}")

    yield_cols, crops, crop_to_yield = get_yield_columns_and_crops(df)
    print(f"Detected crops (count={len(crops)}): {crops[:20]}...")

    # Yield missingness
    miss = df[yield_cols].isna().mean().sort_values(ascending=False) * 100
    miss_df = pd.DataFrame({"YieldColumn": miss.index, "MissingPct": miss.values})
    print("\nYield missingness (% of NaN):")
    print(miss_df.head(20))

    # Summary for main crops if present
    main_crops = ["RICE", "WHEAT", "MAIZE", "GROUNDNUT", "COTTON", "SUGARCANE"]
    for mc in main_crops:
        if mc in crop_to_yield:
            col = crop_to_yield[mc]
            print(f"\nSummary for {mc} yield:")
            print(df[col].describe())

    return yield_cols, crops, crop_to_yield

# ----------------------------------------------------------------------
# CLEANING & DIVERSIFICATION
# ----------------------------------------------------------------------

def clip_yield_outliers(df, yield_cols, upper_q=0.99):
    print("\n===== CLEANING DATA =====")
    df = df.copy()
    for col in yield_cols:
        s = df[col]
        if s.notna().sum() == 0:
            continue
        old_max = s.max()
        q_high = s.quantile(upper_q)
        df[col] = s.clip(upper=q_high)
        new_max = df[col].max()
        if new_max < old_max:
            # Infer crop name for print
            crop_name = col.upper().split("YIELD")[0].strip()
            crop_name = " ".join(crop_name.split())
            print(f"Clipped {crop_name} yield: old max={old_max:.2f}, new max={new_max:.2f}")
    return df

def compute_diversification(df, area_cols):
    print("\n===== COMPUTING DIVERSIFICATION =====")
    if not area_cols:
        print("No crop-specific area columns found; skipping diversification.")
        return df

    df = df.copy()
    area = df[area_cols].clip(lower=0)
    denom = area.sum(axis=1).replace(0, np.nan)
    share_df = area.div(denom, axis=0)

    herf = (share_df ** 2).sum(axis=1)
    df["HERFINDAHL"] = herf
    df["DIVERSIFICATION_INDEX"] = 1.0 - herf
    return df

# ----------------------------------------------------------------------
# NETWORKS
# ----------------------------------------------------------------------

def build_district_similarity_network(df, area_cols, ref_start=2008, ref_end=2017, top_k=7):
    print("\n===== BUILDING DISTRICT SIMILARITY NETWORK =====")
    if not area_cols:
        print("No area columns found; cannot build district similarity network.")
        G = nx.Graph()
        return G, pd.DataFrame(columns=["Dist Code", "DIST_DEGREE", "DIST_STRENGTH", "DIST_BETWEENNESS"])

    sub = df[(df["Year"] >= ref_start) & (df["Year"] <= ref_end)].copy()
    if sub.empty:
        print("No rows in the reference window; cannot build district similarity network.")
        G = nx.Graph()
        return G, pd.DataFrame(columns=["Dist Code", "DIST_DEGREE", "DIST_STRENGTH", "DIST_BETWEENNESS"])

    area = sub[area_cols].clip(lower=0)
    denom = area.sum(axis=1).replace(0, np.nan)
    share = area.div(denom, axis=0)
    share["Dist Code"] = sub["Dist Code"].values

    dist_profile = share.groupby("Dist Code").mean().fillna(0.0)
    if dist_profile.empty:
        print("No district profiles; skipping district network.")
        G = nx.Graph()
        return G, pd.DataFrame(columns=["Dist Code", "DIST_DEGREE", "DIST_STRENGTH", "DIST_BETWEENNESS"])

    from sklearn.metrics import pairwise_distances

    # cosine similarity = 1 - cosine distance
    cos_dist = pairwise_distances(dist_profile.values, metric="cosine")
    cos_sim = 1.0 - cos_dist

    dist_codes = list(dist_profile.index)
    G = nx.Graph()
    for code in dist_codes:
        G.add_node(code)

    for i, u in enumerate(dist_codes):
        sims_i = cos_sim[i]
        # sort indices by similarity descending, skip self
        idx_sorted = np.argsort(-sims_i)
        neighbors = 0
        for j in idx_sorted:
            if j == i:
                continue
            v = dist_codes[j]
            sim_val = sims_i[j]
            if sim_val <= 0:
                break
            G.add_edge(u, v, weight=float(sim_val))
            neighbors += 1
            if neighbors >= top_k:
                break

    print(f"Network reference window: {ref_start}-{ref_end}")
    print(f"District network: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")

    degree = dict(G.degree())
    strength = {
        n: sum(data["weight"] for _, _, data in G.edges(n, data=True))
        for n in G.nodes()
    }
    betw = nx.betweenness_centrality(G, weight="weight", normalized=True)

    dist_net_feat = pd.DataFrame({
        "Dist Code": list(G.nodes()),
        "DIST_DEGREE": [degree[d] for d in G.nodes()],
        "DIST_STRENGTH": [strength[d] for d in G.nodes()],
        "DIST_BETWEENNESS": [betw[d] for d in G.nodes()],
    })
    return G, dist_net_feat

def build_crop_cooccurrence_network(df, crop_to_area):
    print("\n===== BUILDING CROP-COOCCURRENCE NETWORK =====")
    if not crop_to_area:
        print("No crop-area mapping; skipping crop network.")
        return nx.Graph()

    col_to_crop = {area_col: crop for crop, area_col in crop_to_area.items()}
    area_cols = list(col_to_crop.keys())
    area_df = df[area_cols].fillna(0)

    G = nx.Graph()
    for crop in crop_to_area.keys():
        G.add_node(crop)

    for _, row in area_df.iterrows():
        present_cols = [c for c in area_cols if row[c] > 0]
        if len(present_cols) < 2:
            continue
        crops_present = [col_to_crop[c] for c in present_cols]
        for c1, c2 in combinations(sorted(set(crops_present)), 2):
            if G.has_edge(c1, c2):
                G[c1][c2]["weight"] += 1
            else:
                G.add_edge(c1, c2, weight=1)

    print(f"Crop network: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")
    return G

# ----------------------------------------------------------------------
# SUPERVISED PANEL PER CROP
# ----------------------------------------------------------------------

def build_supervised_panel_for_crop(df, crop, yield_col, area_col=None, min_year=1970):
    cols = ["Dist Code", "Year", yield_col, "DIVERSIFICATION_INDEX", "HERFINDAHL",
            "DIST_DEGREE", "DIST_STRENGTH", "DIST_BETWEENNESS"]
    if "State Name" in df.columns:
        cols.append("State Name")
    if "District Name" in df.columns:
        cols.append("District Name")
    if area_col is not None and area_col in df.columns:
        cols.append(area_col)

    cols = [c for c in cols if c in df.columns]
    sub = df[cols].copy()
    sub = sub[sub["Year"] >= min_year].sort_values(["Dist Code", "Year"])

    def _process_group(g):
        g = g.sort_values("Year").copy()
        y = g[yield_col]
        g["Y_LAG1"] = y.shift(1)
        g["Y_LAG2"] = y.shift(2)
        g["Y_LAG3"] = y.shift(3)
        g["Y_ROLL_MEAN3"] = y.rolling(window=3, min_periods=3).mean().shift(1)
        return g

    # Will trigger a deprecation warning in future pandas, but safe now
    sub = sub.groupby("Dist Code", group_keys=False).apply(_process_group)
    sub = sub.rename(columns={yield_col: "TARGET_YIELD"})

    # Require at least lag1 and rolling mean to be available => drop early years
    sub = sub.dropna(subset=["TARGET_YIELD", "Y_LAG1", "Y_ROLL_MEAN3"])
    return sub

# ----------------------------------------------------------------------
# MODELING
# ----------------------------------------------------------------------

def evaluate_models_for_crop(sup, crop, feature_sets=("no_network", "with_network"), area_col_name=None,
                             random_state=RANDOM_STATE):
    print(f"\n===== MODELING {crop} =====")
    if sup.empty:
        print(f"No data for {crop}; skipping.")
        return pd.DataFrame()

    data = sup.copy().sort_values("Year").reset_index(drop=True)

    base_features = ["Year", "DIVERSIFICATION_INDEX", "HERFINDAHL",
                     "Y_LAG1", "Y_LAG2", "Y_LAG3", "Y_ROLL_MEAN3"]
    if area_col_name is not None and area_col_name in data.columns:
        base_features.append(area_col_name)

    network_features = ["DIST_DEGREE", "DIST_STRENGTH", "DIST_BETWEENNESS"]

    y_full = data["TARGET_YIELD"].values

    fold_rows = []

    for feature_set in feature_sets:
        if feature_set == "no_network":
            feature_cols = [c for c in base_features if c in data.columns]
        else:
            feature_cols = [c for c in base_features + network_features if c in data.columns]

        if len(feature_cols) == 0:
            print(f"{crop} | {feature_set}: no usable features, skipping.")
            continue

        X_all = data[feature_cols].values
        tscv = TimeSeriesSplit(n_splits=3)

        try:
            splits = list(tscv.split(X_all))
        except ValueError as e:
            print(f"Time-series CV error for {crop} ({feature_set}): {e}")
            continue

        for fold_idx, (train_idx, test_idx) in enumerate(splits, start=1):
            X_train, X_test = X_all[train_idx], X_all[test_idx]
            y_train, y_test = y_full[train_idx], y_full[test_idx]

            # 1) Naive baseline: use lag-1 yield
            y_pred_naive = data.loc[test_idx, "Y_LAG1"].values
            mae, rmse, mape, medae, r2 = compute_regression_metrics(y_test, y_pred_naive)
            print(f"{crop} | Naive | {feature_set} | Fold {fold_idx}: "
                  f"MAE={mae:.2f}, RMSE={rmse:.2f}, MAPE={mape:.2f}, MedAE={medae:.2f}, R2={r2:.3f}")
            fold_rows.append(dict(Crop=crop, Model="Naive", FeatureSet=feature_set,
                                  Fold=fold_idx, MAE=mae, RMSE=rmse, MAPE=mape,
                                  MedAE=medae, R2=r2))

            # 2) RollingMean3 baseline
            y_pred_rm3 = data.loc[test_idx, "Y_ROLL_MEAN3"].values
            mae, rmse, mape, medae, r2 = compute_regression_metrics(y_test, y_pred_rm3)
            print(f"{crop} | RollingMean3 | {feature_set} | Fold {fold_idx}: "
                  f"MAE={mae:.2f}, RMSE={rmse:.2f}, MAPE={mape:.2f}, MedAE={medae:.2f}, R2={r2:.3f}")
            fold_rows.append(dict(Crop=crop, Model="RollingMean3", FeatureSet=feature_set,
                                  Fold=fold_idx, MAE=mae, RMSE=rmse, MAPE=mape,
                                  MedAE=medae, R2=r2))

            # 3) Ridge regression
            ridge = Ridge(alpha=1.0)
            ridge.fit(X_train, y_train)
            y_pred = ridge.predict(X_test)
            mae, rmse, mape, medae, r2 = compute_regression_metrics(y_test, y_pred)
            print(f"{crop} | Ridge | {feature_set} | Fold {fold_idx}: "
                  f"MAE={mae:.2f}, RMSE={rmse:.2f}, MAPE={mape:.2f}, MedAE={medae:.2f}, R2={r2:.3f}")
            fold_rows.append(dict(Crop=crop, Model="Ridge", FeatureSet=feature_set,
                                  Fold=fold_idx, MAE=mae, RMSE=rmse, MAPE=mape,
                                  MedAE=medae, R2=r2))

            # 4) RandomForest
            rf = RandomForestRegressor(
                n_estimators=300,
                max_depth=None,
                min_samples_leaf=5,
                n_jobs=-1,
                random_state=random_state,
            )
            rf.fit(X_train, y_train)
            y_pred = rf.predict(X_test)
            mae, rmse, mape, medae, r2 = compute_regression_metrics(y_test, y_pred)
            print(f"{crop} | RandomForest | {feature_set} | Fold {fold_idx}: "
                  f"MAE={mae:.2f}, RMSE={rmse:.2f}, MAPE={mape:.2f}, MedAE={medae:.2f}, R2={r2:.3f}")
            fold_rows.append(dict(Crop=crop, Model="RandomForest", FeatureSet=feature_set,
                                  Fold=fold_idx, MAE=mae, RMSE=rmse, MAPE=mape,
                                  MedAE=medae, R2=r2))

            # 5) GradientBoosting
            gb = GradientBoostingRegressor(
                n_estimators=300,
                learning_rate=0.05,
                max_depth=3,
                random_state=random_state,
            )
            gb.fit(X_train, y_train)
            y_pred = gb.predict(X_test)
            mae, rmse, mape, medae, r2 = compute_regression_metrics(y_test, y_pred)
            print(f"{crop} | GradientBoosting | {feature_set} | Fold {fold_idx}: "
                  f"MAE={mae:.2f}, RMSE={rmse:.2f}, MAPE={mape:.2f}, MedAE={medae:.2f}, R2={r2:.3f}")
            fold_rows.append(dict(Crop=crop, Model="GradientBoosting", FeatureSet=feature_set,
                                  Fold=fold_idx, MAE=mae, RMSE=rmse, MAPE=mape,
                                  MedAE=medae, R2=r2))

    if not fold_rows:
        print(f"No fold-level metrics for {crop}; skipping performance summary.")
        return pd.DataFrame()

    fold_df = pd.DataFrame(fold_rows)
    summary = (
        fold_df.groupby(["Crop", "Model", "FeatureSet"], as_index=False)
        .agg(
            MAE_mean=("MAE", "mean"),
            MAE_std=("MAE", "std"),
            RMSE_mean=("RMSE", "mean"),
            RMSE_std=("RMSE", "std"),
            MAPE_mean=("MAPE", "mean"),
            MAPE_std=("MAPE", "std"),
            MedAE_mean=("MedAE", "mean"),
            MedAE_std=("MedAE", "std"),
            R2_mean=("R2", "mean"),
            R2_std=("R2", "std"),
        )
    )

    for fs in summary["FeatureSet"].unique():
        sub = summary[summary["FeatureSet"] == fs].copy().sort_values("MAE_mean")
        print(f"\nPerformance summary for {crop} ({fs}):")
        print(sub)

    return summary

def select_best_model(summary_df, crop, mae_tol_frac=0.01):
    """Select best model per crop:
       - start from min MAE_mean
       - allow mae_tol_frac tolerance
       - prefer positive R2_mean when available
       - then prefer FeatureSet='with_network'
       - then lowest MAE_mean, highest R2_mean
    """
    if summary_df.empty:
        return None
    dfc = summary_df[summary_df["Crop"] == crop].copy()
    if dfc.empty:
        return None
    best_mae = dfc["MAE_mean"].min()
    candidates = dfc[dfc["MAE_mean"] <= best_mae * (1.0 + mae_tol_frac)]

    # Prefer positive R2 where available
    pos = candidates[candidates["R2_mean"] > 0]
    if not pos.empty:
        candidates = pos

    # Prefer with_network when available
    with_net = candidates[candidates["FeatureSet"] == "with_network"]
    if not with_net.empty:
        candidates = with_net

    # Final tie-break: lowest MAE, then highest R2
    candidates = candidates.sort_values(["MAE_mean", "R2_mean"], ascending=[True, False])
    best = candidates.iloc[0]
    print(f"\nBest model for {crop}: {best['Model']} (FeatureSet={best['FeatureSet']}), "
          f"MAE_mean={best['MAE_mean']:.2f}")
    return best

def fit_and_plot_best_model(sup, crop, best_row, area_col_name=None):
    """Fit best model on full data (for ML models) or use baselines and
    save simple diagnostic plots: observed vs predicted and residuals histogram.
    """
    if best_row is None or sup.empty:
        return

    model_name = best_row["Model"]
    feature_set = best_row["FeatureSet"]

    data = sup.copy().sort_values("Year").reset_index(drop=True)
    base_features = ["Year", "DIVERSIFICATION_INDEX", "HERFINDAHL",
                     "Y_LAG1", "Y_LAG2", "Y_LAG3", "Y_ROLL_MEAN3"]
    if area_col_name is not None and area_col_name in data.columns:
        base_features.append(area_col_name)
    network_features = ["DIST_DEGREE", "DIST_STRENGTH", "DIST_BETWEENNESS"]

    if feature_set == "no_network":
        feature_cols = [c for c in base_features if c in data.columns]
    else:
        feature_cols = [c for c in base_features + network_features if c in data.columns]

    y = data["TARGET_YIELD"].values

    if model_name == "Naive":
        y_pred = data["Y_LAG1"].values
    elif model_name == "RollingMean3":
        y_pred = data["Y_ROLL_MEAN3"].values
    else:
        X = data[feature_cols].values
        if model_name == "Ridge":
            model = Ridge(alpha=1.0)
        elif model_name == "RandomForest":
            model = RandomForestRegressor(
                n_estimators=300,
                max_depth=None,
                min_samples_leaf=5,
                n_jobs=-1,
                random_state=RANDOM_STATE,
            )
        elif model_name == "GradientBoosting":
            model = GradientBoostingRegressor(
                n_estimators=300,
                learning_rate=0.05,
                max_depth=3,
                random_state=RANDOM_STATE,
            )
        else:
            return
        model.fit(X, y)
        y_pred = model.predict(X)

    # Drop NaNs
    mask = (~np.isnan(y)) & (~np.isnan(y_pred))
    y = y[mask]
    y_pred = y_pred[mask]
    if len(y) == 0:
        return

    # Scatter: observed vs predicted
    plt.figure(figsize=(5, 4))
    plt.scatter(y, y_pred, s=10, alpha=0.6)
    plt.xlabel("Observed yield (kg/ha)")
    plt.ylabel("Predicted yield (kg/ha)")
    plt.title(f"{crop} | {model_name} ({feature_set})")
    lim_min = min(y.min(), y_pred.min())
    lim_max = max(y.max(), y_pred.max())
    plt.plot([lim_min, lim_max], [lim_min, lim_max], linestyle="--")
    plt.tight_layout()
    out_scatter = os.path.join(
        OUTPUT_DIR,
        f"{crop.lower()}_{model_name.lower()}_{feature_set}_diag_scatter.pdf",
    )
    plt.savefig(out_scatter, dpi=PLOT_DPI)
    plt.close()

    # Residuals histogram
    residuals = y - y_pred
    plt.figure(figsize=(5, 4))
    plt.hist(residuals, bins=30)
    plt.xlabel("Residual (observed - predicted)")
    plt.ylabel("Frequency")
    plt.title(f"{crop} | {model_name} ({feature_set}) residuals")
    plt.tight_layout()
    out_resid = os.path.join(
        OUTPUT_DIR,
        f"{crop.lower()}_{model_name.lower()}_{feature_set}_residuals_hist.pdf",
    )
    plt.savefig(out_resid, dpi=PLOT_DPI)
    plt.close()

    print(f"Saved diagnostic plots for {crop} ({model_name}, feature set={feature_set}) to OUTPUT_DIR.")

# ----------------------------------------------------------------------
# MAIN PIPELINE
# ----------------------------------------------------------------------

def main():
    df = load_data(DATA_PATH)
    yield_cols, crops, crop_to_yield = basic_eda(df)

    # Clip yield outliers
    df = clip_yield_outliers(df, yield_cols, upper_q=0.99)

    # Area mapping & diversification
    crop_to_area = infer_area_columns(df, crop_to_yield)
    area_cols = list(crop_to_area.values())
    df = compute_diversification(df, area_cols)

    # Networks
    G_dist, dist_net_feat = build_district_similarity_network(
        df, area_cols, ref_start=2008, ref_end=2017, top_k=7
    )
    if not dist_net_feat.empty:
        df = df.merge(dist_net_feat, on="Dist Code", how="left")
    else:
        # If network features missing, fill with zeros
        df["DIST_DEGREE"] = 0.0
        df["DIST_STRENGTH"] = 0.0
        df["DIST_BETWEENNESS"] = 0.0

    G_crop = build_crop_cooccurrence_network(df, crop_to_area)

    cleaned_path = os.path.join(OUTPUT_DIR, "icrisat_cleaned_with_features.csv")
    df.to_csv(cleaned_path, index=False)
    print(f"\nSaved cleaned dataset with features to: {cleaned_path}")

    crops_to_model = ["RICE", "WHEAT", "MAIZE", "GROUNDNUT", "COTTON", "SUGARCANE"]

    all_summaries = []
    best_models_info = []

    for crop in crops_to_model:
        if crop not in crop_to_yield:
            print(f"\nCrop {crop} not found in yield columns; skipping modeling.")
            continue
        yield_col = crop_to_yield[crop]
        area_col = crop_to_area.get(crop, None)

        sup = build_supervised_panel_for_crop(df, crop, yield_col, area_col, min_year=1970)
        if sup.empty:
            print(f"\nSupervised panel for {crop} is empty; skipping.")
            continue

        panel_path = os.path.join(OUTPUT_DIR, f"supervised_panel_{crop.lower()}.csv")
        sup.to_csv(panel_path, index=False)
        print(f"\nSaved supervised panel for {crop} to: {panel_path}")

        summary = evaluate_models_for_crop(
            sup,
            crop,
            feature_sets=("no_network", "with_network"),
            area_col_name=area_col,
            random_state=RANDOM_STATE,
        )
        if summary.empty:
            continue

        all_summaries.append(summary)
        best = select_best_model(summary, crop)
        if best is not None:
            best_models_info.append(best)
            fit_and_plot_best_model(sup, crop, best, area_col_name=area_col)

    perf_path = None
    if all_summaries:
        perf_summary = pd.concat(all_summaries, ignore_index=True)
        perf_path = os.path.join(OUTPUT_DIR, "model_performance_summary.csv")
        perf_summary.to_csv(perf_path, index=False)
        print(f"\nSaved model performance summary to: {perf_path}")

    print("\n===== FINAL SUMMARY =====")
    print(f"Rows after cleaning: {len(df)}")
    print(f"Years: {df['Year'].min()} - {df['Year'].max()}")
    if "Dist Code" in df.columns:
        print(f"Number of districts: {df['Dist Code'].nunique()}")

    modeled_crops = [b["Crop"] for b in best_models_info] if best_models_info else []
    print(f"Crops modeled: {modeled_crops}")
    if best_models_info:
        print("Best model per crop (MAE-based with MAE-tolerance + R2/with-network preference):")
        for b in best_models_info:
            print(f"  {b['Crop']}: {b['Model']} (FeatureSet={b['FeatureSet']}), "
                  f"MAE_mean={b['MAE_mean']:.2f}")

    print(f"Cleaned dataset with features: {cleaned_path}")
    if perf_path is not None:
        print(f"Model performance summary: {perf_path}")
    print(f"Supervised panel CSV prefix: {os.path.join(OUTPUT_DIR, 'supervised_panel_')}")
    print("Key PDF plots saved in OUTPUT_DIR (diagnostic scatter + residuals for best models).")

if __name__ == "__main__":
    main()


Duplicate (Dist Code, Year) rows: 0
Key (Dist Code, Year) is unique.

===== BASIC EDA =====
Rows: 16146, Columns: 80
Time range: 1966 - 2017
Unique States: 20
Unique Districts: 311
Detected crops (count=23): ['BARLEY', 'CASTOR', 'CHICKPEA', 'COTTON', 'FINGER MILLET', 'GROUNDNUT', 'KHARIF SORGHUM', 'LINSEED', 'MAIZE', 'MINOR PULSES', 'OILSEEDS', 'PEARL MILLET', 'PIGEONPEA', 'RABI SORGHUM', 'RAPESEED AND MUSTARD', 'RICE', 'SAFFLOWER', 'SESAMUM', 'SORGHUM', 'SOYABEAN']...

Yield missingness (% of NaN):
                               YieldColumn  MissingPct
0                   RICE YIELD (Kg per ha)         0.0
1                  WHEAT YIELD (Kg per ha)         0.0
2         KHARIF SORGHUM YIELD (Kg per ha)         0.0
3           RABI SORGHUM YIELD (Kg per ha)         0.0
4                SORGHUM YIELD (Kg per ha)         0.0
5           PEARL MILLET YIELD (Kg per ha)         0.0
6                  MAIZE YIELD (Kg per ha)         0.0
7          FINGER MILLET YIELD (Kg per ha)         0.0


In [ ]:
import os
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    r2_score,
    median_absolute_error,
)

# -----------------------------------------------------------------------------
# CONFIG
# -----------------------------------------------------------------------------
RANDOM_STATE = 42

# IMPORTANT: set this to your actual raw ICRISAT CSV if you want to start from raw.
# Example:
# RAW_CSV_PATH = "/content/drive/MyDrive/Shiny/ICRISAT/ICRISAT_district_panel_1966_2017.csv"
RAW_CSV_PATH = "/content/drive/MyDrive/Shiny/ICRISAT/icrisat_raw.csv"

OUTPUT_DIR = "/content/drive/MyDrive/Shiny/ICRISAT/Results"
os.makedirs(OUTPUT_DIR, exist_ok=True)

CLEANED_CSV_PATH = os.path.join(OUTPUT_DIR, "icrisat_cleaned_with_features.csv")
MODEL_PERF_PATH = os.path.join(OUTPUT_DIR, "model_performance_summary.csv")
SUPERVISED_PANEL_PREFIX = os.path.join(OUTPUT_DIR, "supervised_panel_")

# Six main crops to model
CROP_YIELD_COLS = {
    "RICE": "RICE YIELD (Kg per ha)",
    "WHEAT": "WHEAT YIELD (Kg per ha)",
    "MAIZE": "MAIZE YIELD (Kg per ha)",
    "GROUNDNUT": "GROUNDNUT YIELD (Kg per ha)",
    "COTTON": "COTTON YIELD (Kg per ha)",
    "SUGARCANE": "SUGARCANE YIELD (Kg per ha)",
}

# -----------------------------------------------------------------------------
# DATA LOADER
# -----------------------------------------------------------------------------
def load_icrisat_dataframe():
    """
    Robust loader:
    1) Try RAW_CSV_PATH (for starting from raw data).
    2) If not found, try CLEANED_CSV_PATH as fallback (for re-running modeling).
    """
    if os.path.exists(RAW_CSV_PATH):
        print(f"Loading RAW dataset from: {RAW_CSV_PATH}")
        df = pd.read_csv(RAW_CSV_PATH)
        return df, "raw"
    elif os.path.exists(CLEANED_CSV_PATH):
        print(
            f"RAW_CSV_PATH not found.\n"
            f"Falling back to existing CLEANED dataset: {CLEANED_CSV_PATH}"
        )
        df = pd.read_csv(CLEANED_CSV_PATH)
        return df, "cleaned"
    else:
        raise FileNotFoundError(
            f"Neither RAW_CSV_PATH nor CLEANED_CSV_PATH exists.\n"
            f"RAW_CSV_PATH = {RAW_CSV_PATH}\n"
            f"CLEANED_CSV_PATH = {CLEANED_CSV_PATH}\n"
            f"Please update RAW_CSV_PATH to the correct CSV file."
        )

# -----------------------------------------------------------------------------
# UTILITIES
# -----------------------------------------------------------------------------
def safe_mape(y_true, y_pred):
    """MAPE computed only on strictly positive targets (in %)."""
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    mask = np.abs(y_true) > 1e-6
    if not np.any(mask):
        return np.nan
    return np.mean(np.abs(y_true[mask] - y_pred[mask]) / np.abs(y_true[mask])) * 100.0


def compute_metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    # sklearn version may not support squared=False, so compute RMSE manually
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    medae = median_absolute_error(y_true, y_pred)
    mape = safe_mape(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return mae, rmse, mape, medae, r2


def make_time_series_folds(df, time_col="Year", n_splits=3):
    """
    Expanding-window time-series CV on the *year* dimension.
    Years are split into (n_splits + 1) contiguous blocks.
    For fold k, train on blocks [0..k-1], test on block k.
    """
    years = np.sort(df[time_col].unique())
    blocks = np.array_split(years, n_splits + 1)  # e.g. 4 blocks for 3 folds
    folds = []
    for k in range(1, len(blocks)):
        train_years = np.concatenate(blocks[:k])
        test_years = blocks[k]
        folds.append((set(train_years), set(test_years)))
    return folds


def make_model(model_name):
    """Instantiate a fresh sklearn model object for the given name."""
    if model_name == "Ridge":
        model = Pipeline(
            [
                ("scaler", StandardScaler()),
                ("ridge", Ridge(alpha=1.0)),
            ]
        )
    elif model_name == "RandomForest":
        model = RandomForestRegressor(
            n_estimators=400,
            max_depth=None,
            min_samples_leaf=2,
            random_state=RANDOM_STATE,
            n_jobs=-1,
        )
    elif model_name == "GradientBoosting":
        model = GradientBoostingRegressor(
            n_estimators=500,
            learning_rate=0.05,
            max_depth=3,
            random_state=RANDOM_STATE,
        )
    else:
        raise ValueError(f"Unknown model name: {model_name}")
    return model

# -----------------------------------------------------------------------------
# BASIC EDA + CLEANING
# -----------------------------------------------------------------------------
def basic_eda(df):
    print("===== BASIC EDA =====")
    # key uniqueness
    dup_count = (
        df.groupby(["Dist Code", "Year"])
        .size()
        .reset_index(name="n")
        .query("n > 1")
        .shape[0]
    )
    print(f"Duplicate (Dist Code, Year) rows: {dup_count}")
    if dup_count == 0:
        print("Key (Dist Code, Year) is unique.\n")

    print(f"Rows: {df.shape[0]}, Columns: {df.shape[1]}")
    if "Year" in df.columns:
        print(f"Time range: {df['Year'].min()} - {df['Year'].max()}")
    if "State Name" in df.columns:
        print(f"Unique States: {df['State Name'].nunique()}")
    if "District Name" in df.columns:
        print(f"Unique Districts: {df['District Name'].nunique()}")

    # Yield columns
    yield_cols = [c for c in df.columns if "YIELD" in c]
    crops = [c.replace(" YIELD (Kg per ha)", "").strip() for c in yield_cols]
    print(f"Detected crops (count={len(crops)}): {sorted(set(crops))[:20]}...")

    # Missingness
    miss = df[yield_cols].isna().mean().sort_values() * 100
    miss_df = (
        miss.reset_index()
        .rename(columns={"index": "YieldColumn", 0: "MissingPct"})
    )
    print("\nYield missingness (% of NaN):")
    print(miss_df.to_string(index=False))

    # Descriptive stats for main crops (if present)
    for crop, ycol in CROP_YIELD_COLS.items():
        if ycol in df.columns:
            print(f"\nSummary for {crop} yield:")
            print(df[ycol].describe())


def clip_yield_outliers(df, upper_quantile=0.99):
    """Clip yield columns at [0, q] to remove extreme outliers."""
    print("\n===== CLEANING DATA =====")
    yield_cols = [c for c in df.columns if "YIELD" in c]
    for col in yield_cols:
        old_max = df[col].max()
        q = df[col].quantile(upper_quantile)
        df[col] = df[col].clip(lower=0, upper=q)
        new_max = df[col].max()
        crop_name = col.replace(" YIELD (Kg per ha)", "").strip()
        print(
            f"Clipped {crop_name} yield: old max={old_max:.2f}, new max={new_max:.2f}"
        )
    return df

# -----------------------------------------------------------------------------
# DIVERSIFICATION + NETWORKS
# -----------------------------------------------------------------------------
def compute_diversification(df):
    """Compute Herfindahl and diversification index using area columns."""
    print("\n===== COMPUTING DIVERSIFICATION =====")
    area_cols = [
        c
        for c in df.columns
        if "AREA" in c.upper() and "YIELD" not in c.upper()
    ]
    if not area_cols:
        print("No AREA columns found – diversification indices will be NaN.")
        df["HERFINDAHL"] = np.nan
        df["DIVERSIFICATION_INDEX"] = np.nan
        return df, area_cols

    area_df = df[["Dist Code", "Year"] + area_cols].copy()
    area_df[area_cols] = area_df[area_cols].clip(lower=0).fillna(0)

    total_area = area_df[area_cols].sum(axis=1)
    shares = area_df[area_cols].div(
        total_area.replace({0: np.nan}), axis=0
    )
    hhi = (shares ** 2).sum(axis=1)

    df["HERFINDAHL"] = hhi.values
    df["DIVERSIFICATION_INDEX"] = 1.0 - hhi.values

    return df, area_cols


def build_district_similarity_network(df, area_cols,
                                      start_year=2008, end_year=2017,
                                      corr_threshold=0.7):
    print("\n===== BUILDING DISTRICT SIMILARITY NETWORK =====")
    print(f"Network reference window: {start_year}-{end_year}")

    window = df[(df["Year"] >= start_year) & (df["Year"] <= end_year)].copy()
    if window.empty:
        raise ValueError("No data in the specified reference window.")

    valid_area_cols = [c for c in area_cols if c in window.columns]
    if not valid_area_cols:
        raise ValueError("No valid area columns found for network building.")

    window[valid_area_cols] = window[valid_area_cols].clip(lower=0).fillna(0)

    # Aggregate to district-level crop area shares over the window
    grouped = window.groupby("Dist Code")[valid_area_cols].sum()
    total = grouped.sum(axis=1)
    shares = grouped.div(total.replace({0: np.nan}), axis=0).fillna(0)

    corr = shares.T.corr()

    G = nx.Graph()
    for dist in shares.index:
        G.add_node(dist)

    dists = list(shares.index)
    for i in range(len(dists)):
        for j in range(i + 1, len(dists)):
            d1, d2 = dists[i], dists[j]
            w = corr.loc[d1, d2]
            if np.isfinite(w) and w >= corr_threshold:
                G.add_edge(d1, d2, weight=float(w))

    print(f"District network: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")
    return G


def build_crop_cooccurrence_network(df, area_cols, min_area=1e-6):
    print("\n===== BUILDING CROP-COOCCURRENCE NETWORK =====")
    valid_area_cols = [c for c in area_cols if c in df.columns]
    if not valid_area_cols:
        raise ValueError("No valid area columns for crop network.")

    crop_names = [
        c.replace("AREA", "")
        .replace("(000 ha)", "")
        .replace("(", "")
        .replace(")", "")
        .replace("  ", " ")
        .replace("  ", " ")
        .strip()
        for c in valid_area_cols
    ]

    G = nx.Graph()
    for crop in crop_names:
        G.add_node(crop)

    arr = df[valid_area_cols].fillna(0).values
    for row in arr:
        active = [
            crop
            for crop, val in zip(crop_names, row)
            if val > min_area
        ]
        for i in range(len(active)):
            for j in range(i + 1, len(active)):
                c1, c2 = active[i], active[j]
                if G.has_edge(c1, c2):
                    G[c1][c2]["weight"] += 1
                else:
                    G.add_edge(c1, c2, weight=1)

    print(f"Crop network: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")
    return G


def add_district_network_features(df, G):
    """Map district-level network metrics back to the full panel."""
    degree_dict = dict(G.degree(weight=None))
    strength_dict = dict(G.degree(weight="weight"))
    betw_dict = nx.betweenness_centrality(G, weight="weight", normalized=True)

    df["DIST_DEGREE"] = df["Dist Code"].map(degree_dict).fillna(0).astype(float)
    df["DIST_STRENGTH"] = df["Dist Code"].map(strength_dict).fillna(0).astype(float)
    df["DIST_BETWEENNESS"] = df["Dist Code"].map(betw_dict).fillna(0.0).astype(float)
    return df

# -----------------------------------------------------------------------------
# SUPERVISED PANEL CONSTRUCTION
# -----------------------------------------------------------------------------
def build_supervised_panel_for_crop(df, crop_name, yield_col,
                                    min_year=1970,
                                    save_prefix=SUPERVISED_PANEL_PREFIX):
    """
    Build a panel with:
      - TARGET_YIELD
      - lags: Y_LAG1, Y_LAG2, Y_LAG3
      - lagged rolling mean: Y_ROLL_MEAN3
      - diversification + network features (if present)
    """
    cols = ["Dist Code", "Year", yield_col]

    for extra_col in [
        "State Name",
        "District Name",
        "DIVERSIFICATION_INDEX",
        "HERFINDAHL",
        "DIST_DEGREE",
        "DIST_STRENGTH",
        "DIST_BETWEENNESS",
    ]:
        if extra_col in df.columns:
            cols.append(extra_col)

    cols = list(dict.fromkeys(cols))  # remove duplicates
    sub = df[cols].copy()
    sub = sub[sub["Year"] >= min_year]
    sub = sub.sort_values(["Dist Code", "Year"]).reset_index(drop=True)

    # Lags
    sub["Y_LAG1"] = sub.groupby("Dist Code")[yield_col].shift(1)
    sub["Y_LAG2"] = sub.groupby("Dist Code")[yield_col].shift(2)
    sub["Y_LAG3"] = sub.groupby("Dist Code")[yield_col].shift(3)

    # Rolling mean of the last 3 years, lagged by 1 year
    roll = (
        sub.groupby("Dist Code")[yield_col]
        .rolling(window=3, min_periods=3)
        .mean()
        .shift(1)
        .reset_index(level=0, drop=True)
    )
    sub["Y_ROLL_MEAN3"] = roll

    sub = sub.rename(columns={yield_col: "TARGET_YIELD"})

    # Require target + at least lag1 and rolling mean to exist
    sub = sub.dropna(
        subset=["TARGET_YIELD", "Y_LAG1", "Y_ROLL_MEAN3"]
    ).reset_index(drop=True)

    # Save to CSV
    out_path = f"{save_prefix}{crop_name.lower()}.csv"
    sub.to_csv(out_path, index=False)
    print(f"\nSaved supervised panel for {crop_name} to: {out_path}")

    return sub

# -----------------------------------------------------------------------------
# MODELING + EVALUATION
# -----------------------------------------------------------------------------
def evaluate_models_for_crop(crop_name, panel_df, records):
    print(f"\n===== MODELING {crop_name} =====")
    folds = make_time_series_folds(panel_df, time_col="Year", n_splits=3)

    base_features = [
        "Y_LAG1",
        "Y_LAG2",
        "Y_LAG3",
        "Y_ROLL_MEAN3",
        "DIVERSIFICATION_INDEX",
        "HERFINDAHL",
    ]
    base_features = [f for f in base_features if f in panel_df.columns]

    network_features = [
        "DIST_DEGREE",
        "DIST_STRENGTH",
        "DIST_BETWEENNESS",
    ]
    network_features = [f for f in network_features if f in panel_df.columns]

    models_to_run = ["Naive", "RollingMean3", "Ridge", "RandomForest", "GradientBoosting"]

    for feature_set in ["no_network", "with_network"]:
        if feature_set == "no_network":
            feature_cols = base_features
        else:
            feature_cols = base_features + network_features

        feature_cols = [c for c in feature_cols if c in panel_df.columns]
        if not feature_cols and feature_set == "with_network":
            # If network features missing, skip with_network
            continue

        for fold_idx, (train_years, test_years) in enumerate(folds, start=1):
            train_mask = panel_df["Year"].isin(train_years)
            test_mask = panel_df["Year"].isin(test_years)

            X_train = panel_df.loc[train_mask, feature_cols]
            X_test = panel_df.loc[test_mask, feature_cols]
            y_train = panel_df.loc[train_mask, "TARGET_YIELD"].values
            y_test = panel_df.loc[test_mask, "TARGET_YIELD"].values

            for model_name in models_to_run:
                if model_name == "Naive":
                    y_pred = panel_df.loc[test_mask, "Y_LAG1"].values
                elif model_name == "RollingMean3":
                    y_pred = panel_df.loc[test_mask, "Y_ROLL_MEAN3"].values
                else:
                    model = make_model(model_name)
                    model.fit(X_train, y_train)
                    y_pred = model.predict(X_test)

                mae, rmse, mape, medae, r2 = compute_metrics(y_test, y_pred)
                print(
                    f"{crop_name} | {model_name} | {feature_set} | Fold {fold_idx}: "
                    f"MAE={mae:.2f}, RMSE={rmse:.2f}, MAPE={mape:.2f}, "
                    f"MedAE={medae:.2f}, R2={r2:.3f}"
                )

                records.append(
                    {
                        "Crop": crop_name,
                        "Model": model_name,
                        "FeatureSet": feature_set,
                        "Fold": fold_idx,
                        "MAE": mae,
                        "RMSE": rmse,
                        "MAPE": mape,
                        "MedAE": medae,
                        "R2": r2,
                    }
                )


def summarise_performance(records_df):
    summary = (
        records_df.groupby(["Crop", "Model", "FeatureSet"])
        .agg(
            MAE_mean=("MAE", "mean"),
            MAE_std=("MAE", "std"),
            RMSE_mean=("RMSE", "mean"),
            RMSE_std=("RMSE", "std"),
            MAPE_mean=("MAPE", "mean"),
            MAPE_std=("MAPE", "std"),
            MedAE_mean=("MedAE", "mean"),
            MedAE_std=("MedAE", "std"),
            R2_mean=("R2", "mean"),
            R2_std=("R2", "std"),
        )
        .reset_index()
    )

    for crop in summary["Crop"].unique():
        for feat_set in ["no_network", "with_network"]:
            sub = summary[(summary["Crop"] == crop) &
                          (summary["FeatureSet"] == feat_set)]
            if not sub.empty:
                print(f"\nPerformance summary for {crop} ({feat_set}):")
                print(sub.to_string(index=False))

    return summary


def select_best_models(summary_df, mae_tol_frac=0.01):
    """
    For each crop, pick the best model as:
      - lowest MAE_mean
      - within 1% tolerance, prefer 'with_network'
      - within that, highest R2_mean
    """
    best_rows = []
    print("\n===== FINAL SUMMARY =====")
    crops = summary_df["Crop"].unique()
    for crop in crops:
        sub = summary_df[summary_df["Crop"] == crop].copy()
        best_mae = sub["MAE_mean"].min()
        tol = mae_tol_frac * best_mae
        candidates = sub[sub["MAE_mean"] <= best_mae + tol]

        # Prefer with_network if at least one candidate has it
        if (candidates["FeatureSet"] == "with_network").any():
            candidates = candidates[candidates["FeatureSet"] == "with_network"]

        # Among remaining, pick highest R2_mean
        idx = candidates["R2_mean"].idxmax()
        row = candidates.loc[idx]
        best_rows.append(row)

        print(
            f"  {crop}: {row['Model']} (FeatureSet={row['FeatureSet']}), "
            f"MAE_mean={row['MAE_mean']:.2f}, R2_mean={row['R2_mean']:.3f}"
        )

    best_df = pd.DataFrame(best_rows).reset_index(drop=True)
    return best_df


def get_oof_predictions_for_best(panel_df, crop_name, best_row):
    """Collect out-of-fold predictions for the best model for diagnostic plots."""
    model_name = best_row["Model"]
    feature_set = best_row["FeatureSet"]

    base_features = [
        "Y_LAG1",
        "Y_LAG2",
        "Y_LAG3",
        "Y_ROLL_MEAN3",
        "DIVERSIFICATION_INDEX",
        "HERFINDAHL",
    ]
    base_features = [f for f in base_features if f in panel_df.columns]

    network_features = [
        "DIST_DEGREE",
        "DIST_STRENGTH",
        "DIST_BETWEENNESS",
    ]
    network_features = [f for f in network_features if f in panel_df.columns]

    if feature_set == "no_network":
        feature_cols = base_features
    else:
        feature_cols = base_features + network_features

    feature_cols = [c for c in feature_cols if c in panel_df.columns]
    folds = make_time_series_folds(panel_df, time_col="Year", n_splits=3)

    all_y_true = []
    all_y_pred = []

    for train_years, test_years in folds:
        train_mask = panel_df["Year"].isin(train_years)
        test_mask = panel_df["Year"].isin(test_years)

        X_train = panel_df.loc[train_mask, feature_cols]
        X_test = panel_df.loc[test_mask, feature_cols]
        y_train = panel_df.loc[train_mask, "TARGET_YIELD"].values
        y_test = panel_df.loc[test_mask, "TARGET_YIELD"].values

        if model_name == "Naive":
            y_pred = panel_df.loc[test_mask, "Y_LAG1"].values
        elif model_name == "RollingMean3":
            y_pred = panel_df.loc[test_mask, "Y_ROLL_MEAN3"].values
        else:
            model = make_model(model_name)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

        all_y_true.append(y_test)
        all_y_pred.append(y_pred)

    all_y_true = np.concatenate(all_y_true)
    all_y_pred = np.concatenate(all_y_pred)
    return all_y_true, all_y_pred


def plot_diagnostics(y_true, y_pred, crop_name, model_name, feature_set):
    """Two-panel diagnostic: predicted vs observed and residuals vs predicted."""
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))

    # Scatter: observed vs predicted
    ax = axes[0]
    ax.scatter(y_true, y_pred, alpha=0.4, edgecolor="none")
    mn = min(np.min(y_true), np.min(y_pred))
    mx = max(np.max(y_true), np.max(y_pred))
    ax.plot([mn, mx], [mn, mx], "k--", linewidth=1)
    ax.set_xlabel("Observed yield (kg/ha)")
    ax.set_ylabel("Predicted yield (kg/ha)")
    ax.set_title(f"{crop_name} | {model_name} | {feature_set}")

    # Residuals vs predicted
    resid = y_true - y_pred
    ax = axes[1]
    ax.scatter(y_pred, resid, alpha=0.4, edgecolor="none")
    ax.axhline(0.0, color="k", linestyle="--", linewidth=1)
    ax.set_xlabel("Predicted yield (kg/ha)")
    ax.set_ylabel("Residual (kg/ha)")
    ax.set_title("Residuals vs predicted")

    fig.tight_layout()
    fname = os.path.join(
        OUTPUT_DIR,
        f"{crop_name.lower()}_{model_name.lower()}_{feature_set}_diagnostics.pdf",
    )
    fig.savefig(fname, bbox_inches="tight")
    plt.close(fig)
    print(
        f"Saved diagnostic plots for {crop_name} ({model_name}, "
        f"feature set={feature_set}) to {OUTPUT_DIR}."
    )

# -----------------------------------------------------------------------------
# MAIN
# -----------------------------------------------------------------------------
def main():
    # 1. Load data (raw or cleaned fallback)
    df, source_type = load_icrisat_dataframe()
    print(f"\nData source type: {source_type}")
    basic_eda(df)

    # 2. Clip yields / clean (idempotent if already clipped)
    df = clip_yield_outliers(df, upper_quantile=0.99)

    # 3. Diversification indices
    df, area_cols = compute_diversification(df)

    # 4. Build networks (district similarity + crop co-occurrence)
    dist_network = build_district_similarity_network(
        df, area_cols, start_year=2008, end_year=2017, corr_threshold=0.7
    )
    crop_network = build_crop_cooccurrence_network(df, area_cols)

    # 5. Add district-level network features
    df = add_district_network_features(df, dist_network)

    # 6. Save cleaned dataset with features
    df.to_csv(CLEANED_CSV_PATH, index=False)
    print(
        f"\nSaved cleaned dataset with features to: {CLEANED_CSV_PATH}"
    )

    # 7. Build supervised panels for each crop
    panels = {}
    for crop, ycol in CROP_YIELD_COLS.items():
        if ycol not in df.columns:
            print(f"\nWARNING: Yield column {ycol} not found, skipping {crop}.")
            continue
        panel = build_supervised_panel_for_crop(
            df, crop_name=crop, yield_col=ycol, min_year=1970
        )
        panels[crop] = panel

    # 8. Model evaluation
    records = []
    for crop, panel in panels.items():
        evaluate_models_for_crop(crop, panel, records)

    records_df = pd.DataFrame(records)
    records_df.to_csv(
        os.path.join(OUTPUT_DIR, "model_per_fold_raw.csv"), index=False
    )

    # 9. Summarise performance
    summary_df = summarise_performance(records_df)
    summary_df.to_csv(MODEL_PERF_PATH, index=False)
    print(f"\nSaved model performance summary to: {MODEL_PERF_PATH}")

    # 10. Select best model per crop
    best_df = select_best_models(summary_df)

    # 11. Diagnostics for best models
    for _, row in best_df.iterrows():
        crop = row["Crop"]
        model_name = row["Model"]
        feature_set = row["FeatureSet"]
        panel = panels[crop]
        y_true, y_pred = get_oof_predictions_for_best(panel, crop, row)
        plot_diagnostics(y_true, y_pred, crop, model_name, feature_set)

    print("\n===== PIPELINE COMPLETE =====")
    print(f"Rows after cleaning: {df.shape[0]}")
    if "Year" in df.columns:
        print(f"Years: {df['Year'].min()} - {df['Year'].max()}")
    if "District Name" in df.columns:
        print(f"Number of districts: {df['District Name'].nunique()}")
    elif "Dist Code" in df.columns:
        print(f"Number of districts (by Dist Code): {df['Dist Code'].nunique()}")
    print(f"Crops modeled: {list(panels.keys())}")
    print(f"Cleaned dataset with features: {CLEANED_CSV_PATH}")
    print(f"Model performance summary: {MODEL_PERF_PATH}")
    print(f"Supervised panel CSV prefix: {SUPERVISED_PANEL_PREFIX}")
    print("Key PDF plots saved in OUTPUT_DIR (diagnostic scatter + residuals for best models).")


if __name__ == "__main__":
    main()


RAW_CSV_PATH not found.
Falling back to existing CLEANED dataset: /content/drive/MyDrive/Shiny/ICRISAT/Results/icrisat_cleaned_with_features.csv

Data source type: cleaned
===== BASIC EDA =====
Duplicate (Dist Code, Year) rows: 0
Key (Dist Code, Year) is unique.

Rows: 16146, Columns: 85
Time range: 1966 - 2017
Unique States: 20
Detected crops (count=23): ['BARLEY', 'CASTOR', 'CHICKPEA', 'COTTON', 'FINGER MILLET', 'GROUNDNUT', 'KHARIF SORGHUM', 'LINSEED', 'MAIZE', 'MINOR PULSES', 'OILSEEDS', 'PEARL MILLET', 'PIGEONPEA', 'RABI SORGHUM', 'RAPESEED AND MUSTARD', 'RICE', 'SAFFLOWER', 'SESAMUM', 'SORGHUM', 'SOYABEAN']...

Yield missingness (% of NaN):
                           YieldColumn  MissingPct
                RICE YIELD (Kg per ha)         0.0
               WHEAT YIELD (Kg per ha)         0.0
      KHARIF SORGHUM YIELD (Kg per ha)         0.0
        RABI SORGHUM YIELD (Kg per ha)         0.0
             SORGHUM YIELD (Kg per ha)         0.0
        PEARL MILLET YIELD (Kg per ha) 

In [ ]:
import os
import warnings
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd
import matplotlib
matplotlib.use("Agg")  # for headless PDF saving
import matplotlib.pyplot as plt
import networkx as nx

from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    r2_score,
)
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# -------------------------------------------------------------------
# CONFIG
# -------------------------------------------------------------------

RAW_CSV_PATH = "/content/drive/MyDrive/Shiny/ICRISAT/Data/icrisat_raw.csv"
CLEANED_CSV_PATH = "/content/drive/MyDrive/Shiny/ICRISAT/Results/icrisat_cleaned_with_features.csv"
OUTPUT_DIR = "/content/drive/MyDrive/Shiny/ICRISAT/Results"

SUPERVISED_PANEL_PREFIX = os.path.join(OUTPUT_DIR, "supervised_panel_")
MODEL_PER_FOLD_CSV = os.path.join(OUTPUT_DIR, "model_per_fold_raw.csv")
MODEL_PERF_SUMMARY_CSV = os.path.join(OUTPUT_DIR, "model_performance_summary.csv")
BEST_MODELS_CSV = os.path.join(OUTPUT_DIR, "best_models_advanced_only.csv")

CROPS = ["RICE", "WHEAT", "MAIZE", "GROUNDNUT", "COTTON", "SUGARCANE"]
RANDOM_STATE = 42
N_FOLDS = 3

os.makedirs(OUTPUT_DIR, exist_ok=True)


# -------------------------------------------------------------------
# UTILS
# -------------------------------------------------------------------

def mape(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    """Mean Absolute Percentage Error in %, safely handling zeros."""
    y_true = np.array(y_true, dtype=float)
    y_pred = np.array(y_pred, dtype=float)
    denom = np.where(y_true == 0, np.nan, np.abs(y_true))
    return np.nanmean(np.abs(y_true - y_pred) / denom) * 100.0


def median_ae(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    y_true = np.array(y_true, dtype=float)
    y_pred = np.array(y_pred, dtype=float)
    return float(np.median(np.abs(y_true - y_pred)))


def safe_print(*args, **kwargs):
    """Wrapper to avoid crashing on weird encodings."""
    try:
        print(*args, **kwargs)
    except Exception:
        pass


# -------------------------------------------------------------------
# LOADING + BASIC EDA
# -------------------------------------------------------------------

def load_data() -> Tuple[pd.DataFrame, str]:
    """Load raw CSV if available, otherwise fall back to existing cleaned dataset."""
    if os.path.exists(RAW_CSV_PATH):
        safe_print(f"RAW_CSV_PATH found: {RAW_CSV_PATH}")
        df = pd.read_csv(RAW_CSV_PATH)
        source_type = "raw"
    elif os.path.exists(CLEANED_CSV_PATH):
        safe_print("RAW_CSV_PATH not found.")
        safe_print(f"Falling back to existing CLEANED dataset: {CLEANED_CSV_PATH}\n")
        df = pd.read_csv(CLEANED_CSV_PATH)
        source_type = "cleaned"
    else:
        raise FileNotFoundError(
            f"Neither raw ({RAW_CSV_PATH}) nor cleaned ({CLEANED_CSV_PATH}) dataset is available."
        )
    return df, source_type


def basic_eda(df: pd.DataFrame) -> None:
    safe_print("===== BASIC EDA =====")
    key_cols = ["Dist Code", "Year"]
    if all(k in df.columns for k in key_cols):
        dup = df.duplicated(subset=key_cols).sum()
        safe_print(f"Duplicate (Dist Code, Year) rows: {dup}")
        safe_print(f"Key (Dist Code, Year) is {'NOT ' if dup>0 else ''}unique.\n")
    else:
        safe_print("Warning: 'Dist Code' / 'Year' not found for uniqueness check.\n")

    safe_print(f"Rows: {len(df):d}, Columns: {df.shape[1]:d}")

    if "Year" in df.columns:
        safe_print(f"Time range: {int(df['Year'].min())} - {int(df['Year'].max())}")

    state_cols = [c for c in df.columns if "STATE" in c.upper() or "STATE NAME" in c.upper()]
    if state_cols:
        sc = state_cols[0]
        safe_print(f"Unique States: {df[sc].nunique()}")

    yield_cols = [c for c in df.columns if "YIELD (Kg per ha)" in c]
    crops = sorted(list({c.split(" YIELD")[0].strip() for c in yield_cols}))
    safe_print(f"Detected crops (count={len(crops)}): {crops[:23]}...\n")

    if yield_cols:
        miss_info = (
            pd.DataFrame(
                {
                    "YieldColumn": yield_cols,
                    "MissingPct": [df[col].isna().mean() * 100.0 for col in yield_cols],
                }
            )
            .sort_values("MissingPct")
            .reset_index(drop=True)
        )
        safe_print("Yield missingness (% of NaN):")
        safe_print(miss_info.to_string(index=False))
        safe_print()

        for col in ["RICE YIELD (Kg per ha)",
                    "WHEAT YIELD (Kg per ha)",
                    "MAIZE YIELD (Kg per ha)",
                    "GROUNDNUT YIELD (Kg per ha)",
                    "COTTON YIELD (Kg per ha)",
                    "SUGARCANE YIELD (Kg per ha)"]:
            if col in df.columns:
                safe_print(f"Summary for {col.split(' YIELD')[0]} yield:")
                safe_print(df[col].describe())
                safe_print()


# -------------------------------------------------------------------
# CLEANING YIELDS + DIVERSIFICATION
# -------------------------------------------------------------------

def clean_yields(df: pd.DataFrame) -> pd.DataFrame:
    """Clip extreme outliers and replace negatives for all yield columns."""
    safe_print("===== CLEANING DATA =====")
    df = df.copy()
    yield_cols = [c for c in df.columns if "YIELD (Kg per ha)" in c]

    for col in yield_cols:
        s = df[col].copy()
        old_max = s.max()
        s = s.clip(lower=0)
        if (s > 0).sum() > 0:
            q = s[s > 0].quantile(0.995)
            s = s.clip(upper=q)
            new_max = float(s.max())
            if new_max < old_max:
                safe_print(f"Clipped {col.split(' YIELD')[0]} yield: old max={old_max:.2f}, new max={new_max:.2f}")
        df[col] = s

    safe_print()
    return df


def compute_diversification_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Compute simple diversification features using AREA columns if available.
    Uses 1 - HHI (Simpson-like) and number of active crops.
    """
    safe_print("===== COMPUTING DIVERSIFICATION =====")
    df = df.copy()

    yield_cols = [c for c in df.columns if "YIELD (Kg per ha)" in c]
    crop_names = [c.split(" YIELD")[0].strip() for c in yield_cols]

    crop_area_cols = {}
    for crop in crop_names:
        candidates = [c for c in df.columns if crop in c and "AREA" in c.upper()]
        if candidates:
            crop_area_cols[crop] = candidates[0]

    if not crop_area_cols:
        safe_print("No AREA columns detected – diversification features will be zeros.\n")
        df["diversity_simpson"] = 0.0
        df["num_active_crops"] = 0
        return df

    area_mat = df[[crop_area_cols[c] for c in crop_area_cols]].fillna(0.0).values
    total_area = area_mat.sum(axis=1)
    total_area_safe = np.where(total_area == 0, 1.0, total_area)
    shares = area_mat / total_area_safe[:, None]
    hhi = (shares ** 2).sum(axis=1)
    simpson = 1.0 - hhi
    num_active = (area_mat > 0).sum(axis=1)

    df["diversity_simpson"] = simpson
    df["num_active_crops"] = num_active

    return df


# -------------------------------------------------------------------
# NETWORKS
# -------------------------------------------------------------------

def build_district_network(df: pd.DataFrame) -> Tuple[nx.Graph, pd.DataFrame]:
    """
    Build a simple district similarity network using yield profiles
    in a reference window (2008–2017).
    Returns:
        G: NetworkX graph
        dist_features: DataFrame with ['Dist Code', 'net_degree', 'net_strength', 'net_closeness', 'net_betweenness']
    """
    safe_print("===== BUILDING DISTRICT SIMILARITY NETWORK =====")
    df_ref = df.copy()
    if "Year" in df_ref.columns:
        df_ref = df_ref[(df_ref["Year"] >= 2008) & (df_ref["Year"] <= 2017)]

    if "Dist Code" not in df_ref.columns:
        safe_print("No 'Dist Code' column: cannot build district network.")
        return nx.Graph(), pd.DataFrame(columns=["Dist Code"])

    yield_cols = [c for c in df_ref.columns if "YIELD (Kg per ha)" in c]
    if not yield_cols:
        safe_print("No YIELD columns found: cannot build district network.")
        return nx.Graph(), pd.DataFrame(columns=["Dist Code"])

    prof = (
        df_ref.groupby("Dist Code")[yield_cols]
        .mean()
        .fillna(0.0)
    )

    if prof.shape[0] == 0:
        safe_print("No district profiles after filtering.")
        return nx.Graph(), pd.DataFrame(columns=["Dist Code"])

    arr = prof.values
    norms = np.linalg.norm(arr, axis=1)
    norms_safe = np.where(norms == 0, 1.0, norms)
    arr_norm = arr / norms_safe[:, None]

    sim = arr_norm @ arr_norm.T

    dist_codes = prof.index.tolist()
    G = nx.Graph()
    for di in dist_codes:
        G.add_node(di)

    thr = 0.6
    n = sim.shape[0]
    for i in range(n):
        for j in range(i + 1, n):
            w = float(sim[i, j])
            if w >= thr:
                G.add_edge(dist_codes[i], dist_codes[j], weight=w)

    safe_print(f"Network reference window: 2008-2017")
    safe_print(f"District network: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges\n")

    if G.number_of_nodes() == 0:
        return G, pd.DataFrame(columns=["Dist Code"])

    deg = dict(G.degree())
    strength = {n: sum(d["weight"] for _, _, d in G.edges(n, data=True)) for n in G.nodes}
    closeness = nx.closeness_centrality(G)
    betw = nx.betweenness_centrality(G, normalized=True)

    dist_features = pd.DataFrame({
        "Dist Code": list(G.nodes),
        "net_degree": [deg[d] for d in G.nodes],
        "net_strength": [strength[d] for d in G.nodes],
        "net_closeness": [closeness[d] for d in G.nodes],
        "net_betweenness": [betw[d] for d in G.nodes],
    })

    return G, dist_features


def build_crop_cooccurrence_network(df: pd.DataFrame) -> nx.Graph:
    """
    Simple crop co-occurrence network (for descriptive use; not fed into models here).
    Nodes: crops; edge weight = number of district-years where both are active.
    """
    safe_print("===== BUILDING CROP-COOCCURRENCE NETWORK =====")
    yield_cols = [c for c in df.columns if "YIELD (Kg per ha)" in c]
    if not yield_cols:
        safe_print("No YIELD columns, skipping crop network.\n")
        return nx.Graph()

    crop_names = [c.split(" YIELD")[0].strip() for c in yield_cols]
    crop_idx = {col: crop_names[i] for i, col in enumerate(yield_cols)}

    G = nx.Graph()
    for crop in crop_names:
        G.add_node(crop)

    key_cols = [c for c in ["Dist Code", "Year"] if c in df.columns]
    if not key_cols:
        rows = [df]
    else:
        rows = [g for _, g in df.groupby(key_cols)]

    from itertools import combinations

    for sub in rows:
        vals = sub[yield_cols].fillna(0.0)
        active_mask = (vals > 0).any(axis=0)
        active_cols = [c for c, m in zip(yield_cols, active_mask) if m]
        active_crops = [crop_idx[c] for c in active_cols]
        for c1, c2 in combinations(sorted(set(active_crops)), 2):
            if G.has_edge(c1, c2):
                G[c1][c2]["weight"] += 1
            else:
                G.add_edge(c1, c2, weight=1)

    safe_print(f"Crop network: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges\n")
    return G


# -------------------------------------------------------------------
# SUPERVISED PANEL PREPARATION
# -------------------------------------------------------------------

def prepare_panel_for_crop(
    df: pd.DataFrame,
    crop: str
) -> Tuple[pd.DataFrame, str, List[str], List[str]]:
    """
    Build supervised panel for a given crop:
      - adds lags & rolling mean
      - identifies feature columns and network feature subset.
    Returns:
      panel, yield_col, feature_cols, network_feature_cols
    """
    yield_candidates = [c for c in df.columns if c.startswith(crop) and "YIELD (Kg per ha)" in c]
    if not yield_candidates:
        raise ValueError(f"Yield column for crop '{crop}' not found.")
    yield_col = yield_candidates[0]

    if "Dist Code" not in df.columns or "Year" not in df.columns:
        raise ValueError("Both 'Dist Code' and 'Year' must be present in the dataset.")

    panel = df.copy()
    panel = panel.sort_values(["Dist Code", "Year"])
    grp = panel.groupby("Dist Code")[yield_col]

    panel[f"{crop}_lag1"] = grp.shift(1)
    panel[f"{crop}_lag2"] = grp.shift(2)
    panel[f"{crop}_lag3"] = grp.shift(3)
    panel[f"{crop}_roll3"] = grp.shift(1).rolling(window=3, min_periods=1).mean()

    excluded = {"Dist Code", "Year", yield_col}
    feature_cols = []
    for col in panel.columns:
        if col in excluded:
            continue
        if np.issubdtype(panel[col].dtype, np.number):
            feature_cols.append(col)

    network_feature_cols = [
        c for c in feature_cols
        if c.lower().startswith("net_")
        or "network" in c.lower()
        or "centrality" in c.lower()
    ]

    return panel, yield_col, feature_cols, network_feature_cols


# -------------------------------------------------------------------
# TIME-SERIES CROSS-VALIDATION SPLITS
# -------------------------------------------------------------------

def time_series_folds(years: np.ndarray, n_folds: int = N_FOLDS) -> List[Tuple[np.ndarray, np.ndarray]]:
    """
    Make simple expanding-window time-based folds.
    Returns list of (train_years, test_years).
    """
    years_unique = np.sort(np.unique(years))
    n_years = len(years_unique)
    if n_years < n_folds + 1:
        warnings.warn(f"Too few years ({n_years}) for {n_folds} folds; using single split.")
        mid = n_years // 2
        return [(years_unique[:mid], years_unique[mid:])]

    split_indices = [
        int((i + 1) * n_years / (n_folds + 1))
        for i in range(n_folds)
    ]

    folds = []
    for i in range(n_folds):
        train_end_idx = split_indices[i]
        test_start_idx = train_end_idx
        test_end_idx = split_indices[i + 1] if i + 1 < len(split_indices) else n_years

        train_years = years_unique[:train_end_idx]
        test_years = years_unique[test_start_idx:test_end_idx]
        if len(train_years) == 0 or len(test_years) == 0:
            continue
        folds.append((train_years, test_years))

    return folds


# -------------------------------------------------------------------
# MODEL TRAINING + EVALUATION
# -------------------------------------------------------------------

def evaluate_models_for_crop(
    crop: str,
    panel: pd.DataFrame,
    yield_col: str,
    feature_cols: List[str],
    network_feature_cols: List[str],
) -> List[Dict]:
    """
    Evaluate Naive, RollingMean3, Ridge, RandomForest, GradientBoosting
    under no_network / with_network feature sets.
    Returns list of per-fold metric dicts.
    """
    records = []

    mask_valid = panel[yield_col].notna() & (panel[yield_col] > 0)
    panel_valid = panel.loc[mask_valid].copy()

    years = panel_valid["Year"].values
    folds = time_series_folds(years, n_folds=N_FOLDS)
    if not folds:
        warnings.warn(f"No valid folds for crop {crop}; skipping.")
        return records

    non_network_feature_cols = [c for c in feature_cols if c not in network_feature_cols]
    if not non_network_feature_cols:
        non_network_feature_cols = feature_cols
        network_feature_cols = []

    ridge_no_net = Ridge(alpha=1.0, random_state=RANDOM_STATE)
    ridge_with_net = Ridge(alpha=1.0, random_state=RANDOM_STATE)

    rf_no_net = RandomForestRegressor(
        n_estimators=300,
        max_depth=None,
        min_samples_leaf=1,
        random_state=RANDOM_STATE,
        n_jobs=-1,
    )
    rf_with_net = RandomForestRegressor(
        n_estimators=300,
        max_depth=None,
        min_samples_leaf=1,
        random_state=RANDOM_STATE,
        n_jobs=-1,
    )

    gb_no_net = GradientBoostingRegressor(random_state=RANDOM_STATE)
    gb_with_net = GradientBoostingRegressor(random_state=RANDOM_STATE)

    fold_id = 0
    for train_years, test_years in folds:
        fold_id += 1
        train_mask = panel_valid["Year"].isin(train_years)
        test_mask = panel_valid["Year"].isin(test_years)

        train_df = panel_valid.loc[train_mask]
        test_df = panel_valid.loc[test_mask]

        if len(train_df) == 0 or len(test_df) == 0:
            continue

        y_train = train_df[yield_col].values
        y_test = test_df[yield_col].values

        lag_col = f"{crop}_lag1"
        roll3_col = f"{crop}_roll3"

        # -----------------------------
        # Baseline 1: Naive (lag-1)
        # -----------------------------
        if lag_col in test_df.columns:
            y_pred_naive = test_df[lag_col].values.astype(float)
            mean_train = y_train.mean()
            y_pred_naive = np.where(np.isnan(y_pred_naive), mean_train, y_pred_naive)
        else:
            y_pred_naive = np.full_like(y_test, y_train.mean(), dtype=float)

        mae = mean_absolute_error(y_test, y_pred_naive)
        rmse = mean_squared_error(y_test, y_pred_naive) ** 0.5
        mape_val = mape(y_test, y_pred_naive)
        medae = median_ae(y_test, y_pred_naive)
        r2 = r2_score(y_test, y_pred_naive)

        safe_print(f"{crop} | Naive | no_network | Fold {fold_id}: "
                   f"MAE={mae:.2f}, RMSE={rmse:.2f}, MAPE={mape_val:.2f}, MedAE={medae:.2f}, R2={r2:.3f}")

        records.append({
            "Crop": crop,
            "Model": "Naive",
            "FeatureSet": "no_network",
            "Fold": fold_id,
            "MAE": mae,
            "RMSE": rmse,
            "MAPE": mape_val,
            "MedAE": medae,
            "R2": r2,
        })

        # -----------------------------
        # Baseline 2: RollingMean3
        # -----------------------------
        if roll3_col in test_df.columns:
            y_pred_roll3 = test_df[roll3_col].values.astype(float)
            mean_train = y_train.mean()
            y_pred_roll3 = np.where(np.isnan(y_pred_roll3), mean_train, y_pred_roll3)
        else:
            y_pred_roll3 = np.full_like(y_test, y_train.mean(), dtype=float)

        mae = mean_absolute_error(y_test, y_pred_roll3)
        rmse = mean_squared_error(y_test, y_pred_roll3) ** 0.5
        mape_val = mape(y_test, y_pred_roll3)
        medae = median_ae(y_test, y_pred_roll3)
        r2 = r2_score(y_test, y_pred_roll3)

        safe_print(f"{crop} | RollingMean3 | no_network | Fold {fold_id}: "
                   f"MAE={mae:.2f}, RMSE={rmse:.2f}, MAPE={mape_val:.2f}, MedAE={medae:.2f}, R2={r2:.3f}")

        records.append({
            "Crop": crop,
            "Model": "RollingMean3",
            "FeatureSet": "no_network",
            "Fold": fold_id,
            "MAE": mae,
            "RMSE": rmse,
            "MAPE": mape_val,
            "MedAE": medae,
            "R2": r2,
        })

        # -----------------------------
        # Advanced models: no_network
        # -----------------------------
        X_train_no_net = train_df[non_network_feature_cols].fillna(0.0).values
        X_test_no_net = test_df[non_network_feature_cols].fillna(0.0).values

        # Ridge no_network
        ridge_no_net.fit(X_train_no_net, y_train)
        y_pred = ridge_no_net.predict(X_test_no_net)
        mae = mean_absolute_error(y_test, y_pred)
        rmse = mean_squared_error(y_test, y_pred) ** 0.5
        mape_val = mape(y_test, y_pred)
        medae = median_ae(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        safe_print(f"{crop} | Ridge | no_network | Fold {fold_id}: "
                   f"MAE={mae:.2f}, RMSE={rmse:.2f}, MAPE={mape_val:.2f}, MedAE={medae:.2f}, R2={r2:.3f}")
        records.append({
            "Crop": crop,
            "Model": "Ridge",
            "FeatureSet": "no_network",
            "Fold": fold_id,
            "MAE": mae,
            "RMSE": rmse,
            "MAPE": mape_val,
            "MedAE": medae,
            "R2": r2,
        })

        # RandomForest no_network
        rf_no_net.fit(X_train_no_net, y_train)
        y_pred = rf_no_net.predict(X_test_no_net)
        mae = mean_absolute_error(y_test, y_pred)
        rmse = mean_squared_error(y_test, y_pred) ** 0.5
        mape_val = mape(y_test, y_pred)
        medae = median_ae(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        safe_print(f"{crop} | RandomForest | no_network | Fold {fold_id}: "
                   f"MAE={mae:.2f}, RMSE={rmse:.2f}, MAPE={mape_val:.2f}, MedAE={medae:.2f}, R2={r2:.3f}")
        records.append({
            "Crop": crop,
            "Model": "RandomForest",
            "FeatureSet": "no_network",
            "Fold": fold_id,
            "MAE": mae,
            "RMSE": rmse,
            "MAPE": mape_val,
            "MedAE": medae,
            "R2": r2,
        })

        # GradientBoosting no_network
        gb_no_net.fit(X_train_no_net, y_train)
        y_pred = gb_no_net.predict(X_test_no_net)
        mae = mean_absolute_error(y_test, y_pred)
        rmse = mean_squared_error(y_test, y_pred) ** 0.5
        mape_val = mape(y_test, y_pred)
        medae = median_ae(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        safe_print(f"{crop} | GradientBoosting | no_network | Fold {fold_id}: "
                   f"MAE={mae:.2f}, RMSE={rmse:.2f}, MAPE={mape_val:.2f}, MedAE={medae:.2f}, R2={r2:.3f}")
        records.append({
            "Crop": crop,
            "Model": "GradientBoosting",
            "FeatureSet": "no_network",
            "Fold": fold_id,
            "MAE": mae,
            "RMSE": rmse,
            "MAPE": mape_val,
            "MedAE": medae,
            "R2": r2,
        })

        # -----------------------------
        # Advanced models: with_network
        # -----------------------------
        if network_feature_cols:
            feature_cols_with_net = non_network_feature_cols + network_feature_cols
        else:
            feature_cols_with_net = non_network_feature_cols

        X_train_with_net = train_df[feature_cols_with_net].fillna(0.0).values
        X_test_with_net = test_df[feature_cols_with_net].fillna(0.0).values

        # Ridge with_network
        ridge_with_net.fit(X_train_with_net, y_train)
        y_pred = ridge_with_net.predict(X_test_with_net)
        mae = mean_absolute_error(y_test, y_pred)
        rmse = mean_squared_error(y_test, y_pred) ** 0.5
        mape_val = mape(y_test, y_pred)
        medae = median_ae(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        safe_print(f"{crop} | Ridge | with_network | Fold {fold_id}: "
                   f"MAE={mae:.2f}, RMSE={rmse:.2f}, MAPE={mape_val:.2f}, MedAE={medae:.2f}, R2={r2:.3f}")
        records.append({
            "Crop": crop,
            "Model": "Ridge",
            "FeatureSet": "with_network",
            "Fold": fold_id,
            "MAE": mae,
            "RMSE": rmse,
            "MAPE": mape_val,
            "MedAE": medae,
            "R2": r2,
        })

        # RandomForest with_network
        rf_with_net.fit(X_train_with_net, y_train)
        y_pred = rf_with_net.predict(X_test_with_net)
        mae = mean_absolute_error(y_test, y_pred)
        rmse = mean_squared_error(y_test, y_pred) ** 0.5
        mape_val = mape(y_test, y_pred)
        medae = median_ae(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        safe_print(f"{crop} | RandomForest | with_network | Fold {fold_id}: "
                   f"MAE={mae:.2f}, RMSE={rmse:.2f}, MAPE={mape_val:.2f}, MedAE={medae:.2f}, R2={r2:.3f}")
        records.append({
            "Crop": crop,
            "Model": "RandomForest",
            "FeatureSet": "with_network",
            "Fold": fold_id,
            "MAE": mae,
            "RMSE": rmse,
            "MAPE": mape_val,
            "MedAE": medae,
            "R2": r2,
        })

        # GradientBoosting with_network
        gb_with_net.fit(X_train_with_net, y_train)
        y_pred = gb_with_net.predict(X_test_with_net)
        mae = mean_absolute_error(y_test, y_pred)
        rmse = mean_squared_error(y_test, y_pred) ** 0.5
        mape_val = mape(y_test, y_pred)
        medae = median_ae(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        safe_print(f"{crop} | GradientBoosting | with_network | Fold {fold_id}: "
                   f"MAE={mae:.2f}, RMSE={rmse:.2f}, MAPE={mape_val:.2f}, MedAE={medae:.2f}, R2={r2:.3f}")
        records.append({
            "Crop": crop,
            "Model": "GradientBoosting",
            "FeatureSet": "with_network",
            "Fold": fold_id,
            "MAE": mae,
            "RMSE": rmse,
            "MAPE": mape_val,
            "MedAE": medae,
            "R2": r2,
        })

    return records


# -------------------------------------------------------------------
# MODEL SELECTION (ADVANCED ONLY) + SUMMARY
# -------------------------------------------------------------------

def summarise_performance(per_fold_df: pd.DataFrame) -> pd.DataFrame:
    agg = (
        per_fold_df
        .groupby(["Crop", "Model", "FeatureSet"])
        .agg(
            MAE_mean=("MAE", "mean"),
            MAE_std=("MAE", "std"),
            RMSE_mean=("RMSE", "mean"),
            RMSE_std=("RMSE", "std"),
            MAPE_mean=("MAPE", "mean"),
            MAPE_std=("MAPE", "std"),
            MedAE_mean=("MedAE", "mean"),
            MedAE_std=("MedAE", "std"),
            R2_mean=("R2", "mean"),
            R2_std=("R2", "std"),
        )
        .reset_index()
    )
    return agg


def select_best_advanced_models(perf_summary: pd.DataFrame) -> pd.DataFrame:
    """
    Restrict best-model selection to advanced models only:
    Ridge, RandomForest, GradientBoosting.
    Preference:
      1) lower MAE_mean
      2) within 1% of best MAE_mean, highest R2_mean
      3) prefer with_network when available
    """
    advanced_models = ["Ridge", "RandomForest", "GradientBoosting"]
    best_rows = []

    for crop in sorted(perf_summary["Crop"].unique()):
        sub = perf_summary[
            (perf_summary["Crop"] == crop)
            & (perf_summary["Model"].isin(advanced_models))
        ]
        if sub.empty:
            continue

        sub_with_net = sub[sub["FeatureSet"] == "with_network"]
        if not sub_with_net.empty:
            cand = sub_with_net
        else:
            cand = sub

        min_mae = cand["MAE_mean"].min()
        mae_tol = min_mae * 1.01  # 1% tolerance
        cand = cand[cand["MAE_mean"] <= mae_tol]

        best_idx = cand["R2_mean"].idxmax()
        best_row = cand.loc[best_idx].copy()
        best_rows.append(best_row)

    best_df = pd.DataFrame(best_rows).reset_index(drop=True)
    if not best_df.empty:
        best_df["ModelLabel"] = [
            f"Proposed_{m}_{fs}"
            for m, fs in zip(best_df["Model"], best_df["FeatureSet"])
        ]
    return best_df


# -------------------------------------------------------------------
# DIAGNOSTIC PLOTS FOR BEST MODELS
# -------------------------------------------------------------------

def fit_model_for_diagnostics(
    crop: str,
    panel: pd.DataFrame,
    yield_col: str,
    feature_cols: List[str],
    network_feature_cols: List[str],
    model_name: str,
    feature_set: str,
):
    """Fit advanced model on all available data for diagnostics (no CV here)."""
    mask_valid = panel[yield_col].notna() & (panel[yield_col] > 0)
    df = panel.loc[mask_valid].copy()
    y = df[yield_col].values

    non_network_feature_cols = [c for c in feature_cols if c not in network_feature_cols]
    if feature_set == "no_network" or not network_feature_cols:
        X_cols = non_network_feature_cols
    else:
        X_cols = non_network_feature_cols + network_feature_cols

    X = df[X_cols].fillna(0.0).values

    if model_name == "Ridge":
        model = Ridge(alpha=1.0, random_state=RANDOM_STATE)
    elif model_name == "RandomForest":
        model = RandomForestRegressor(
            n_estimators=300,
            max_depth=None,
            min_samples_leaf=1,
            n_jobs=-1,
            random_state=RANDOM_STATE,
        )
    elif model_name == "GradientBoosting":
        model = GradientBoostingRegressor(random_state=RANDOM_STATE)
    else:
        raise ValueError(f"Diagnostics only implemented for advanced models, got {model_name}.")

    model.fit(X, y)
    y_pred = model.predict(X)

    return df, y, y_pred, X_cols


def save_diagnostic_plots(
    crop: str,
    model_name: str,
    feature_set: str,
    df: pd.DataFrame,
    y_true: np.ndarray,
    y_pred: np.ndarray,
):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred) ** 0.5
    r2 = r2_score(y_true, y_pred)

    plt.figure(figsize=(6, 6))
    plt.scatter(y_true, y_pred, alpha=0.3)
    lims = [
        min(y_true.min(), y_pred.min()),
        max(y_true.max(), y_pred.max()),
    ]
    plt.plot(lims, lims)
    plt.xlabel("Observed yield (kg/ha)")
    plt.ylabel("Predicted yield (kg/ha)")
    plt.title(f"{crop} – {model_name} ({feature_set})\nMAE={mae:.1f}, RMSE={rmse:.1f}, R²={r2:.3f}")
    plt.tight_layout()
    scatter_path = os.path.join(
        OUTPUT_DIR,
        f"{crop}_{model_name}_{feature_set}_diagnostic_scatter.pdf",
    )
    plt.savefig(scatter_path)
    plt.close()

    residuals = y_true - y_pred
    plt.figure(figsize=(6, 4))
    plt.scatter(y_pred, residuals, alpha=0.3)
    plt.axhline(0.0, linestyle="--")
    plt.xlabel("Predicted yield (kg/ha)")
    plt.ylabel("Residual (observed - predicted)")
    plt.title(f"{crop} – {model_name} ({feature_set}) residuals")
    plt.tight_layout()
    resid_path = os.path.join(
        OUTPUT_DIR,
        f"{crop}_{model_name}_{feature_set}_diagnostic_residuals.pdf",
    )
    plt.savefig(resid_path)
    plt.close()

    safe_print(
        f"Saved diagnostic plots for {crop} ({model_name}, feature set={feature_set}) to {OUTPUT_DIR}."
    )


# -------------------------------------------------------------------
# MAIN PIPELINE
# -------------------------------------------------------------------

def main():
    df, source_type = load_data()
    safe_print(f"Data source type: {source_type}")
    basic_eda(df)

    df = clean_yields(df)
    df = compute_diversification_features(df)

    dist_graph, dist_feats = build_district_network(df)
    crop_graph = build_crop_cooccurrence_network(df)

    if not dist_feats.empty:
        df = df.merge(dist_feats, on="Dist Code", how="left")

    df.to_csv(CLEANED_CSV_PATH, index=False)
    safe_print(f"Saved cleaned dataset with features to: {CLEANED_CSV_PATH}\n")

    all_records = []
    panels_by_crop: Dict[str, Dict] = {}

    for crop in CROPS:
        panel, y_col, feat_cols, net_cols = prepare_panel_for_crop(df, crop)

        panel_path = f"{SUPERVISED_PANEL_PREFIX}{crop.lower()}.csv"
        panel.to_csv(panel_path, index=False)
        safe_print(f"Saved supervised panel for {crop} to: {panel_path}\n")

        panels_by_crop[crop] = {
            "panel": panel,
            "yield_col": y_col,
            "feature_cols": feat_cols,
            "network_feature_cols": net_cols,
        }

        crop_records = evaluate_models_for_crop(
            crop=crop,
            panel=panel,
            yield_col=y_col,
            feature_cols=feat_cols,
            network_feature_cols=net_cols,
        )
        all_records.extend(crop_records)

    per_fold_df = pd.DataFrame(all_records)
    per_fold_df.to_csv(MODEL_PER_FOLD_CSV, index=False)
    safe_print(f"\nSaved per-fold model results to: {MODEL_PER_FOLD_CSV}")

    perf_summary = summarise_performance(per_fold_df)
    perf_summary.to_csv(MODEL_PERF_SUMMARY_CSV, index=False)
    safe_print(f"Saved model performance summary to: {MODEL_PERF_SUMMARY_CSV}\n")

    for crop in CROPS:
        for fs in ["no_network", "with_network"]:
            sub = perf_summary[(perf_summary["Crop"] == crop) & (perf_summary["FeatureSet"] == fs)]
            if sub.empty:
                continue
            safe_print(f"Performance summary for {crop} ({fs}):")
            safe_print(sub.sort_values("MAE_mean").to_string(index=False))
            safe_print()

    best_df = select_best_advanced_models(perf_summary)
    best_df.to_csv(BEST_MODELS_CSV, index=False)

    safe_print("===== FINAL SUMMARY (ADVANCED PROPOSED MODELS ONLY) =====")
    for row in best_df.itertuples(index=False):
        safe_print(
            f"  {row.Crop}: Proposed model = {row.Model} (FeatureSet={row.FeatureSet}), "
            f"MAE_mean={row.MAE_mean:.2f}, R2_mean={row.R2_mean:.3f}"
        )

    for row in best_df.itertuples(index=False):
        crop = row.Crop
        model_name = row.Model
        feature_set = row.FeatureSet

        meta = panels_by_crop[crop]
        panel = meta["panel"]
        y_col = meta["yield_col"]
        feat_cols = meta["feature_cols"]
        net_cols = meta["network_feature_cols"]

        df_diag, y_true, y_pred, _ = fit_model_for_diagnostics(
            crop=crop,
            panel=panel,
            yield_col=y_col,
            feature_cols=feat_cols,
            network_feature_cols=net_cols,
            model_name=model_name,
            feature_set=feature_set,
        )

        save_diagnostic_plots(
            crop=crop,
            model_name=model_name,
            feature_set=feature_set,
            df=df_diag,
            y_true=y_true,
            y_pred=y_pred,
        )

    safe_print("\n===== PIPELINE COMPLETE =====")
    if "Year" in df.columns:
        safe_print(f"Rows after cleaning: {len(df)}")
        safe_print(f"Years: {int(df['Year'].min())} - {int(df['Year'].max())}")
    safe_print(f"Number of districts (by Dist Code): {df['Dist Code'].nunique() if 'Dist Code' in df.columns else 'N/A'}")
    safe_print(f"Crops modeled: {CROPS}")
    safe_print(f"Cleaned dataset with features: {CLEANED_CSV_PATH}")
    safe_print(f"Model performance summary: {MODEL_PERF_SUMMARY_CSV}")
    safe_print(f"Supervised panel CSV prefix: {SUPERVISED_PANEL_PREFIX}")
    safe_print("Key PDF plots saved in OUTPUT_DIR (diagnostic scatter + residuals for best advanced models).")


if __name__ == "__main__":
    warnings.filterwarnings("ignore")
    main()


RAW_CSV_PATH not found.
Falling back to existing CLEANED dataset: /content/drive/MyDrive/Shiny/ICRISAT/Results/icrisat_cleaned_with_features.csv

Data source type: cleaned
===== BASIC EDA =====
Duplicate (Dist Code, Year) rows: 0
Key (Dist Code, Year) is unique.

Rows: 16146, Columns: 91
Time range: 1966 - 2017
Unique States: 20
Detected crops (count=23): ['BARLEY', 'CASTOR', 'CHICKPEA', 'COTTON', 'FINGER MILLET', 'GROUNDNUT', 'KHARIF SORGHUM', 'LINSEED', 'MAIZE', 'MINOR PULSES', 'OILSEEDS', 'PEARL MILLET', 'PIGEONPEA', 'RABI SORGHUM', 'RAPESEED AND MUSTARD', 'RICE', 'SAFFLOWER', 'SESAMUM', 'SORGHUM', 'SOYABEAN', 'SUGARCANE', 'SUNFLOWER', 'WHEAT']...

Yield missingness (% of NaN):
                           YieldColumn  MissingPct
                RICE YIELD (Kg per ha)         0.0
               WHEAT YIELD (Kg per ha)         0.0
      KHARIF SORGHUM YIELD (Kg per ha)         0.0
        RABI SORGHUM YIELD (Kg per ha)         0.0
             SORGHUM YIELD (Kg per ha)         0.0
    

In [ ]:
import os
import warnings
from typing import Dict, List, Tuple, Optional

import numpy as np
import pandas as pd
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

from scipy import stats
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.feature_selection import VarianceThreshold

# -------------------------------------------------------------------
# CONFIG
# -------------------------------------------------------------------

RAW_CSV_PATH = "/content/drive/MyDrive/Shiny/ICRISAT/Data/icrisat_raw.csv"
CLEANED_CSV_PATH = "/content/drive/MyDrive/Shiny/ICRISAT/Results/icrisat_cleaned_with_features.csv"
OUTPUT_DIR = "/content/drive/MyDrive/Shiny/ICRISAT/Results"

SUPERVISED_PANEL_PREFIX = os.path.join(OUTPUT_DIR, "supervised_panel_")
MODEL_PER_FOLD_CSV = os.path.join(OUTPUT_DIR, "model_per_fold_raw.csv")
MODEL_PERF_SUMMARY_CSV = os.path.join(OUTPUT_DIR, "model_performance_summary.csv")
BEST_MODELS_CSV = os.path.join(OUTPUT_DIR, "best_models_advanced_only.csv")
FEATURE_IMPORTANCE_CSV = os.path.join(OUTPUT_DIR, "feature_importance_summary.csv")
STATISTICAL_TESTS_CSV = os.path.join(OUTPUT_DIR, "statistical_significance_tests.csv")
TEMPORAL_STABILITY_CSV = os.path.join(OUTPUT_DIR, "temporal_stability_analysis.csv")

CROPS = ["RICE", "WHEAT", "MAIZE", "GROUNDNUT", "COTTON", "SUGARCANE"]
RANDOM_STATE = 42
N_FOLDS = 3

os.makedirs(OUTPUT_DIR, exist_ok=True)

# -------------------------------------------------------------------
# UTILS
# -------------------------------------------------------------------

def safe_mape(y_true: np.ndarray, y_pred: np.ndarray, min_threshold: float = 100.0) -> float:
    """
    Mean Absolute Percentage Error in %, with minimum threshold to avoid
    division by near-zero values (especially important for cotton).
    """
    y_true = np.array(y_true, dtype=float)
    y_pred = np.array(y_pred, dtype=float)
    mask = y_true > min_threshold
    if not np.any(mask):
        return np.nan
    return np.mean(np.abs(y_true[mask] - y_pred[mask]) / y_true[mask]) * 100.0


def median_ae(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    return float(np.median(np.abs(np.array(y_true) - np.array(y_pred))))


def safe_print(*args, **kwargs):
    try:
        print(*args, **kwargs)
    except Exception:
        pass


# -------------------------------------------------------------------
# LOADING + BASIC EDA
# -------------------------------------------------------------------

def load_data() -> Tuple[pd.DataFrame, str]:
    if os.path.exists(RAW_CSV_PATH):
        safe_print(f"RAW_CSV_PATH found: {RAW_CSV_PATH}")
        df = pd.read_csv(RAW_CSV_PATH)
        source_type = "raw"
    elif os.path.exists(CLEANED_CSV_PATH):
        safe_print("RAW_CSV_PATH not found.")
        safe_print(f"Falling back to existing CLEANED dataset: {CLEANED_CSV_PATH}\n")
        df = pd.read_csv(CLEANED_CSV_PATH)
        source_type = "cleaned"
    else:
        raise FileNotFoundError(
            f"Neither raw ({RAW_CSV_PATH}) nor cleaned ({CLEANED_CSV_PATH}) dataset is available."
        )
    return df, source_type


def basic_eda(df: pd.DataFrame) -> None:
    safe_print("===== BASIC EDA =====")
    key_cols = ["Dist Code", "Year"]
    if all(k in df.columns for k in key_cols):
        dup = df.duplicated(subset=key_cols).sum()
        safe_print(f"Duplicate (Dist Code, Year) rows: {dup}")
        safe_print(f"Key (Dist Code, Year) is {'NOT ' if dup>0 else ''}unique.\n")

    safe_print(f"Rows: {len(df):d}, Columns: {df.shape[1]:d}")

    if "Year" in df.columns:
        safe_print(f"Time range: {int(df['Year'].min())} - {int(df['Year'].max())}")

    state_cols = [c for c in df.columns if "STATE" in c.upper() or "STATE NAME" in c.upper()]
    if state_cols:
        safe_print(f"Unique States: {df[state_cols[0]].nunique()}")

    yield_cols = [c for c in df.columns if "YIELD (Kg per ha)" in c]
    crops = sorted(list({c.split(" YIELD")[0].strip() for c in yield_cols}))
    safe_print(f"Detected crops (count={len(crops)}): {crops[:23]}...\n")

    if yield_cols:
        miss_info = pd.DataFrame({
            "YieldColumn": yield_cols,
            "MissingPct": [df[col].isna().mean() * 100.0 for col in yield_cols],
        }).sort_values("MissingPct").reset_index(drop=True)
        safe_print("Yield missingness (% of NaN):")
        safe_print(miss_info.to_string(index=False))
        safe_print()

        for col in ["RICE YIELD (Kg per ha)", "WHEAT YIELD (Kg per ha)",
                    "MAIZE YIELD (Kg per ha)", "GROUNDNUT YIELD (Kg per ha)",
                    "COTTON YIELD (Kg per ha)", "SUGARCANE YIELD (Kg per ha)"]:
            if col in df.columns:
                safe_print(f"Summary for {col.split(' YIELD')[0]} yield:")
                safe_print(df[col].describe())
                safe_print()


# -------------------------------------------------------------------
# CLEANING YIELDS + DIVERSIFICATION
# -------------------------------------------------------------------

def clean_yields(df: pd.DataFrame) -> pd.DataFrame:
    safe_print("===== CLEANING DATA =====")
    df = df.copy()
    yield_cols = [c for c in df.columns if "YIELD (Kg per ha)" in c]

    for col in yield_cols:
        s = df[col].copy()
        old_max = s.max()
        s = s.clip(lower=0)
        if (s > 0).sum() > 0:
            q = s[s > 0].quantile(0.995)
            s = s.clip(upper=q)
            new_max = float(s.max())
            if new_max < old_max:
                safe_print(f"Clipped {col.split(' YIELD')[0]} yield: "
                          f"old max={old_max:.2f}, new max={new_max:.2f}")
        df[col] = s

    safe_print()
    return df


def compute_diversification_features(df: pd.DataFrame) -> pd.DataFrame:
    safe_print("===== COMPUTING DIVERSIFICATION =====")
    df = df.copy()

    yield_cols = [c for c in df.columns if "YIELD (Kg per ha)" in c]
    crop_names = [c.split(" YIELD")[0].strip() for c in yield_cols]

    crop_area_cols = {}
    for crop in crop_names:
        candidates = [c for c in df.columns if crop in c and "AREA" in c.upper()]
        if candidates:
            crop_area_cols[crop] = candidates[0]

    if not crop_area_cols:
        safe_print("No AREA columns detected – diversification features will be zeros.\n")
        df["diversity_simpson"] = 0.0
        df["num_active_crops"] = 0
        return df

    area_mat = df[[crop_area_cols[c] for c in crop_area_cols]].fillna(0.0).values
    total_area = area_mat.sum(axis=1)
    total_area_safe = np.where(total_area == 0, 1.0, total_area)
    shares = area_mat / total_area_safe[:, None]
    hhi = (shares ** 2).sum(axis=1)
    simpson = 1.0 - hhi
    num_active = (area_mat > 0).sum(axis=1)

    df["diversity_simpson"] = simpson
    df["num_active_crops"] = num_active

    return df


# -------------------------------------------------------------------
# ENHANCED NETWORK CONSTRUCTION
# -------------------------------------------------------------------

def build_district_network(df: pd.DataFrame) -> Tuple[nx.Graph, pd.DataFrame]:
    """
    IMPROVED: Build district similarity network with:
    - Higher correlation threshold (0.80) for meaningful connections
    - Top-k pruning to maintain sparse, interpretable network
    - Additional centrality measures
    """
    safe_print("===== BUILDING ENHANCED DISTRICT SIMILARITY NETWORK =====")
    df_ref = df.copy()
    if "Year" in df_ref.columns:
        df_ref = df_ref[(df_ref["Year"] >= 2008) & (df_ref["Year"] <= 2017)]

    if "Dist Code" not in df_ref.columns:
        safe_print("No 'Dist Code' column: cannot build district network.")
        return nx.Graph(), pd.DataFrame(columns=["Dist Code"])

    yield_cols = [c for c in df_ref.columns if "YIELD (Kg per ha)" in c]
    if not yield_cols:
        safe_print("No YIELD columns found: cannot build district network.")
        return nx.Graph(), pd.DataFrame(columns=["Dist Code"])

    # Aggregate to district-level mean yields
    prof = df_ref.groupby("Dist Code")[yield_cols].mean().fillna(0.0)

    if prof.shape[0] == 0:
        safe_print("No district profiles after filtering.")
        return nx.Graph(), pd.DataFrame(columns=["Dist Code"])

    # Compute cosine similarity
    arr = prof.values
    norms = np.linalg.norm(arr, axis=1)
    norms_safe = np.where(norms == 0, 1.0, norms)
    arr_norm = arr / norms_safe[:, None]
    sim = arr_norm @ arr_norm.T

    dist_codes = prof.index.tolist()
    G = nx.Graph()
    for di in dist_codes:
        G.add_node(di)

    # IMPROVED: Higher threshold for more meaningful connections
    threshold = 0.80
    n = sim.shape[0]
    edges_added = []

    for i in range(n):
        for j in range(i + 1, n):
            w = float(sim[i, j])
            if w >= threshold:
                edges_added.append((dist_codes[i], dist_codes[j], w))

    # IMPROVED: Keep only top-k strongest edges per node
    max_edges_per_node = 15
    node_edges = {node: [] for node in dist_codes}

    for d1, d2, w in edges_added:
        node_edges[d1].append((d2, w))
        node_edges[d2].append((d1, w))

    for node in dist_codes:
        edges = sorted(node_edges[node], key=lambda x: x[1], reverse=True)[:max_edges_per_node]
        for neighbor, weight in edges:
            if not G.has_edge(node, neighbor):
                G.add_edge(node, neighbor, weight=weight)

    density = nx.density(G) if G.number_of_nodes() > 0 else 0.0
    safe_print(f"Network reference window: 2008-2017")
    safe_print(f"District network: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")
    safe_print(f"Network density: {density:.1%} (target: 5-20%)\n")

    if G.number_of_nodes() == 0:
        return G, pd.DataFrame(columns=["Dist Code"])

    # IMPROVED: Multiple centrality measures
    deg = dict(G.degree())
    strength = {n: sum(d["weight"] for _, _, d in G.edges(n, data=True)) for n in G.nodes}

    # Only compute closeness/betweenness if network is connected enough
    if nx.is_connected(G):
        closeness = nx.closeness_centrality(G)
        betw = nx.betweenness_centrality(G, normalized=True)
    else:
        # For disconnected graphs, compute per component
        closeness = {}
        betw = {}
        for component in nx.connected_components(G):
            subG = G.subgraph(component)
            comp_close = nx.closeness_centrality(subG)
            comp_betw = nx.betweenness_centrality(subG, normalized=True)
            closeness.update(comp_close)
            betw.update(comp_betw)

    eigenvector = nx.eigenvector_centrality(G, max_iter=1000, weight='weight')
    clustering = nx.clustering(G, weight='weight')

    dist_features = pd.DataFrame({
        "Dist Code": list(G.nodes),
        "net_degree": [deg[d] for d in G.nodes],
        "net_strength": [strength[d] for d in G.nodes],
        "net_closeness": [closeness[d] for d in G.nodes],
        "net_betweenness": [betw[d] for d in G.nodes],
        "net_eigenvector": [eigenvector[d] for d in G.nodes],
        "net_clustering": [clustering[d] for d in G.nodes],
    })

    return G, dist_features


def build_crop_cooccurrence_network(df: pd.DataFrame) -> nx.Graph:
    safe_print("===== BUILDING CROP-COOCCURRENCE NETWORK =====")
    yield_cols = [c for c in df.columns if "YIELD (Kg per ha)" in c]
    if not yield_cols:
        safe_print("No YIELD columns, skipping crop network.\n")
        return nx.Graph()

    crop_names = [c.split(" YIELD")[0].strip() for c in yield_cols]
    crop_idx = {col: crop_names[i] for i, col in enumerate(yield_cols)}

    G = nx.Graph()
    for crop in crop_names:
        G.add_node(crop)

    key_cols = [c for c in ["Dist Code", "Year"] if c in df.columns]
    if not key_cols:
        rows = [df]
    else:
        rows = [g for _, g in df.groupby(key_cols)]

    from itertools import combinations

    for sub in rows:
        vals = sub[yield_cols].fillna(0.0)
        active_mask = (vals > 0).any(axis=0)
        active_cols = [c for c, m in zip(yield_cols, active_mask) if m]
        active_crops = [crop_idx[c] for c in active_cols]
        for c1, c2 in combinations(sorted(set(active_crops)), 2):
            if G.has_edge(c1, c2):
                G[c1][c2]["weight"] += 1
            else:
                G.add_edge(c1, c2, weight=1)

    safe_print(f"Crop network: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges\n")
    return G


# -------------------------------------------------------------------
# SUPERVISED PANEL PREPARATION WITH FEATURE ENGINEERING
# -------------------------------------------------------------------

def prepare_panel_for_crop(df: pd.DataFrame, crop: str) -> Tuple[pd.DataFrame, str, List[str], List[str]]:
    """
    IMPROVED: Enhanced feature engineering with:
    - Robust handling of multicollinearity
    - Removal of near-zero variance features
    - Proper scaling indicators
    """
    yield_candidates = [c for c in df.columns if c.startswith(crop) and "YIELD (Kg per ha)" in c]
    if not yield_candidates:
        raise ValueError(f"Yield column for crop '{crop}' not found.")
    yield_col = yield_candidates[0]

    if "Dist Code" not in df.columns or "Year" not in df.columns:
        raise ValueError("Both 'Dist Code' and 'Year' must be present in the dataset.")

    panel = df.copy()
    panel = panel.sort_values(["Dist Code", "Year"])
    grp = panel.groupby("Dist Code")[yield_col]

    # Temporal features
    panel[f"{crop}_lag1"] = grp.shift(1)
    panel[f"{crop}_lag2"] = grp.shift(2)
    panel[f"{crop}_lag3"] = grp.shift(3)
    panel[f"{crop}_roll3"] = grp.shift(1).rolling(window=3, min_periods=1).mean()

    # IMPROVED: Year-over-year change
    panel[f"{crop}_yoy_change"] = grp.diff(1)

    # IMPROVED: Volatility measure (rolling std)
    panel[f"{crop}_roll3_std"] = grp.shift(1).rolling(window=3, min_periods=2).std()

    excluded = {"Dist Code", "Year", yield_col}
    feature_cols = []
    for col in panel.columns:
        if col in excluded:
            continue
        if np.issubdtype(panel[col].dtype, np.number):
            # IMPROVED: Check for near-zero variance
            if panel[col].std() > 1e-6:
                feature_cols.append(col)
            else:
                safe_print(f"  Excluding near-zero variance feature: {col}")

    # IMPROVED: Check for perfect multicollinearity
    numeric_panel = panel[feature_cols].fillna(0.0)
    corr_matrix = numeric_panel.corr().abs()

    # Find highly correlated pairs
    upper_tri = corr_matrix.where(
        np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
    )
    to_drop = [col for col in upper_tri.columns if any(upper_tri[col] > 0.99)]

    if to_drop:
        safe_print(f"  Removing highly correlated features: {to_drop}")
        feature_cols = [c for c in feature_cols if c not in to_drop]

    network_feature_cols = [
        c for c in feature_cols
        if c.lower().startswith("net_")
        or "network" in c.lower()
        or "centrality" in c.lower()
    ]

    safe_print(f"  Total features: {len(feature_cols)}, Network features: {len(network_feature_cols)}")

    return panel, yield_col, feature_cols, network_feature_cols


# -------------------------------------------------------------------
# TIME-SERIES CROSS-VALIDATION SPLITS
# -------------------------------------------------------------------

def time_series_folds(years: np.ndarray, n_folds: int = N_FOLDS) -> List[Tuple[np.ndarray, np.ndarray]]:
    years_unique = np.sort(np.unique(years))
    n_years = len(years_unique)
    if n_years < n_folds + 1:
        warnings.warn(f"Too few years ({n_years}) for {n_folds} folds; using single split.")
        mid = n_years // 2
        return [(years_unique[:mid], years_unique[mid:])]

    split_indices = [int((i + 1) * n_years / (n_folds + 1)) for i in range(n_folds)]

    folds = []
    for i in range(n_folds):
        train_end_idx = split_indices[i]
        test_start_idx = train_end_idx
        test_end_idx = split_indices[i + 1] if i + 1 < len(split_indices) else n_years

        train_years = years_unique[:train_end_idx]
        test_years = years_unique[test_start_idx:test_end_idx]
        if len(train_years) == 0 or len(test_years) == 0:
            continue
        folds.append((train_years, test_years))

    return folds


# -------------------------------------------------------------------
# IMPROVED MODEL TRAINING WITH ROBUST RIDGE
# -------------------------------------------------------------------

def create_robust_ridge(alpha: float = 10.0) -> Ridge:
    """IMPROVED: Ridge with higher regularization to handle multicollinearity"""
    return Ridge(alpha=alpha, max_iter=10000, random_state=RANDOM_STATE)


def evaluate_models_for_crop(
    crop: str,
    panel: pd.DataFrame,
    yield_col: str,
    feature_cols: List[str],
    network_feature_cols: List[str],
) -> Tuple[List[Dict], Dict]:
    """
    IMPROVED: Returns both records and feature importance info
    """
    records = []
    feature_importance_data = {}

    mask_valid = panel[yield_col].notna() & (panel[yield_col] > 0)
    panel_valid = panel.loc[mask_valid].copy()

    years = panel_valid["Year"].values
    folds = time_series_folds(years, n_folds=N_FOLDS)
    if not folds:
        warnings.warn(f"No valid folds for crop {crop}; skipping.")
        return records, feature_importance_data

    non_network_feature_cols = [c for c in feature_cols if c not in network_feature_cols]
    if not non_network_feature_cols:
        non_network_feature_cols = feature_cols
        network_feature_cols = []

    fold_id = 0
    all_importances_gb_no_net = []
    all_importances_gb_with_net = []

    for train_years, test_years in folds:
        fold_id += 1
        train_mask = panel_valid["Year"].isin(train_years)
        test_mask = panel_valid["Year"].isin(test_years)

        train_df = panel_valid.loc[train_mask]
        test_df = panel_valid.loc[test_mask]

        if len(train_df) == 0 or len(test_df) == 0:
            continue

        y_train = train_df[yield_col].values
        y_test = test_df[yield_col].values

        lag_col = f"{crop}_lag1"
        roll3_col = f"{crop}_roll3"

        # Baseline 1: Naive
        if lag_col in test_df.columns:
            y_pred_naive = test_df[lag_col].values.astype(float)
            mean_train = y_train.mean()
            y_pred_naive = np.where(np.isnan(y_pred_naive), mean_train, y_pred_naive)
        else:
            y_pred_naive = np.full_like(y_test, y_train.mean(), dtype=float)

        mae = mean_absolute_error(y_test, y_pred_naive)
        rmse = mean_squared_error(y_test, y_pred_naive) ** 0.5
        mape_val = safe_mape(y_test, y_pred_naive)
        medae = median_ae(y_test, y_pred_naive)
        r2 = r2_score(y_test, y_pred_naive)

        safe_print(f"{crop} | Naive | no_network | Fold {fold_id}: "
                   f"MAE={mae:.2f}, RMSE={rmse:.2f}, MAPE={mape_val:.2f}, MedAE={medae:.2f}, R2={r2:.3f}")

        records.append({
            "Crop": crop, "Model": "Naive", "FeatureSet": "no_network", "Fold": fold_id,
            "MAE": mae, "RMSE": rmse, "MAPE": mape_val, "MedAE": medae, "R2": r2,
        })

        # Baseline 2: RollingMean3
        if roll3_col in test_df.columns:
            y_pred_roll3 = test_df[roll3_col].values.astype(float)
            mean_train = y_train.mean()
            y_pred_roll3 = np.where(np.isnan(y_pred_roll3), mean_train, y_pred_roll3)
        else:
            y_pred_roll3 = np.full_like(y_test, y_train.mean(), dtype=float)

        mae = mean_absolute_error(y_test, y_pred_roll3)
        rmse = mean_squared_error(y_test, y_pred_roll3) ** 0.5
        mape_val = safe_mape(y_test, y_pred_roll3)
        medae = median_ae(y_test, y_pred_roll3)
        r2 = r2_score(y_test, y_pred_roll3)

        safe_print(f"{crop} | RollingMean3 | no_network | Fold {fold_id}: "
                   f"MAE={mae:.2f}, RMSE={rmse:.2f}, MAPE={mape_val:.2f}, MedAE={medae:.2f}, R2={r2:.3f}")

        records.append({
            "Crop": crop, "Model": "RollingMean3", "FeatureSet": "no_network", "Fold": fold_id,
            "MAE": mae, "RMSE": rmse, "MAPE": mape_val, "MedAE": medae, "R2": r2,
        })

        # Advanced models: no_network
        X_train_no_net = train_df[non_network_feature_cols].fillna(0.0).values
        X_test_no_net = test_df[non_network_feature_cols].fillna(0.0).values

        # IMPROVED: Robust scaling for Ridge
        scaler_no_net = RobustScaler()
        X_train_no_net_scaled = scaler_no_net.fit_transform(X_train_no_net)
        X_test_no_net_scaled = scaler_no_net.transform(X_test_no_net)

        # Ridge no_network
        ridge_no_net = create_robust_ridge(alpha=10.0)
        ridge_no_net.fit(X_train_no_net_scaled, y_train)
        y_pred = ridge_no_net.predict(X_test_no_net_scaled)

        mae = mean_absolute_error(y_test, y_pred)
        rmse = mean_squared_error(y_test, y_pred) ** 0.5
        mape_val = safe_mape(y_test, y_pred)
        medae = median_ae(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        safe_print(f"{crop} | Ridge | no_network | Fold {fold_id}: "
                   f"MAE={mae:.2f}, RMSE={rmse:.2f}, MAPE={mape_val:.2f}, MedAE={medae:.2f}, R2={r2:.3f}")
        records.append({
            "Crop": crop, "Model": "Ridge", "FeatureSet": "no_network", "Fold": fold_id,
            "MAE": mae, "RMSE": rmse, "MAPE": mape_val, "MedAE": medae, "R2": r2,
        })

        # RandomForest no_network
        rf_no_net = RandomForestRegressor(
            n_estimators=300, max_depth=None, min_samples_leaf=1,
            random_state=RANDOM_STATE, n_jobs=-1
        )
        rf_no_net.fit(X_train_no_net, y_train)
        y_pred = rf_no_net.predict(X_test_no_net)

        mae = mean_absolute_error(y_test, y_pred)
        rmse = mean_squared_error(y_test, y_pred) ** 0.5
        mape_val = safe_mape(y_test, y_pred)
        medae = median_ae(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        safe_print(f"{crop} | RandomForest | no_network | Fold {fold_id}: "
                   f"MAE={mae:.2f}, RMSE={rmse:.2f}, MAPE={mape_val:.2f}, MedAE={medae:.2f}, R2={r2:.3f}")
        records.append({
            "Crop": crop, "Model": "RandomForest", "FeatureSet": "no_network", "Fold": fold_id,
            "MAE": mae, "RMSE": rmse, "MAPE": mape_val, "MedAE": medae, "R2": r2,
        })

        # GradientBoosting no_network
        gb_no_net = GradientBoostingRegressor(random_state=RANDOM_STATE)
        gb_no_net.fit(X_train_no_net, y_train)
        y_pred = gb_no_net.predict(X_test_no_net)

        mae = mean_absolute_error(y_test, y_pred)
        rmse = mean_squared_error(y_test, y_pred) ** 0.5
        mape_val = safe_mape(y_test, y_pred)
        medae = median_ae(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        safe_print(f"{crop} | GradientBoosting | no_network | Fold {fold_id}: "
                   f"MAE={mae:.2f}, RMSE={rmse:.2f}, MAPE={mape_val:.2f}, MedAE={medae:.2f}, R2={r2:.3f}")
        records.append({
            "Crop": crop, "Model": "GradientBoosting", "FeatureSet": "no_network", "Fold": fold_id,
            "MAE": mae, "RMSE": rmse, "MAPE": mape_val, "MedAE": medae, "R2": r2,
        })

        # Store feature importance
        all_importances_gb_no_net.append(gb_no_net.feature_importances_)

        # Advanced models: with_network
        if network_feature_cols:
            feature_cols_with_net = non_network_feature_cols + network_feature_cols
        else:
            feature_cols_with_net = non_network_feature_cols

        X_train_with_net = train_df[feature_cols_with_net].fillna(0.0).values
        X_test_with_net = test_df[feature_cols_with_net].fillna(0.0).values

        # IMPROVED: Robust scaling for Ridge
        scaler_with_net = RobustScaler()
        X_train_with_net_scaled = scaler_with_net.fit_transform(X_train_with_net)
        X_test_with_net_scaled = scaler_with_net.transform(X_test_with_net)

        # Ridge with_network
        ridge_with_net = create_robust_ridge(alpha=10.0)
        ridge_with_net.fit(X_train_with_net_scaled, y_train)
        y_pred = ridge_with_net.predict(X_test_with_net_scaled)

        mae = mean_absolute_error(y_test, y_pred)
        rmse = mean_squared_error(y_test, y_pred) ** 0.5
        mape_val = safe_mape(y_test, y_pred)
        medae = median_ae(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        safe_print(f"{crop} | Ridge | with_network | Fold {fold_id}: "
                   f"MAE={mae:.2f}, RMSE={rmse:.2f}, MAPE={mape_val:.2f}, MedAE={medae:.2f}, R2={r2:.3f}")
        records.append({
            "Crop": crop, "Model": "Ridge", "FeatureSet": "with_network", "Fold": fold_id,
            "MAE": mae, "RMSE": rmse, "MAPE": mape_val, "MedAE": medae, "R2": r2,
        })

        # RandomForest with_network
        rf_with_net = RandomForestRegressor(
            n_estimators=300, max_depth=None, min_samples_leaf=1,
            random_state=RANDOM_STATE, n_jobs=-1
        )
        rf_with_net.fit(X_train_with_net, y_train)
        y_pred = rf_with_net.predict(X_test_with_net)

        mae = mean_absolute_error(y_test, y_pred)
        rmse = mean_squared_error(y_test, y_pred) ** 0.5
        mape_val = safe_mape(y_test, y_pred)
        medae = median_ae(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        safe_print(f"{crop} | RandomForest | with_network | Fold {fold_id}: "
                   f"MAE={mae:.2f}, RMSE={rmse:.2f}, MAPE={mape_val:.2f}, MedAE={medae:.2f}, R2={r2:.3f}")
        records.append({
            "Crop": crop, "Model": "RandomForest", "FeatureSet": "with_network", "Fold": fold_id,
            "MAE": mae, "RMSE": rmse, "MAPE": mape_val, "MedAE": medae, "R2": r2,
        })

        # GradientBoosting with_network
        gb_with_net = GradientBoostingRegressor(random_state=RANDOM_STATE)
        gb_with_net.fit(X_train_with_net, y_train)
        y_pred = gb_with_net.predict(X_test_with_net)

        mae = mean_absolute_error(y_test, y_pred)
        rmse = mean_squared_error(y_test, y_pred) ** 0.5
        mape_val = safe_mape(y_test, y_pred)
        medae = median_ae(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        safe_print(f"{crop} | GradientBoosting | with_network | Fold {fold_id}: "
                   f"MAE={mae:.2f}, RMSE={rmse:.2f}, MAPE={mape_val:.2f}, MedAE={medae:.2f}, R2={r2:.3f}")
        records.append({
            "Crop": crop, "Model": "GradientBoosting", "FeatureSet": "with_network", "Fold": fold_id,
            "MAE": mae, "RMSE": rmse, "MAPE": mape_val, "MedAE": medae, "R2": r2,
        })

        # Store feature importance
        all_importances_gb_with_net.append(gb_with_net.feature_importances_)

    # Compute average feature importance across folds
    if all_importances_gb_no_net:
        avg_imp_no_net = np.mean(all_importances_gb_no_net, axis=0)
        feature_importance_data['no_network'] = {
            'features': non_network_feature_cols,
            'importance': avg_imp_no_net
        }

    if all_importances_gb_with_net:
        avg_imp_with_net = np.mean(all_importances_gb_with_net, axis=0)
        feature_importance_data['with_network'] = {
            'features': feature_cols_with_net,
            'importance': avg_imp_with_net
        }

    return records, feature_importance_data


# -------------------------------------------------------------------
# STATISTICAL SIGNIFICANCE TESTING
# -------------------------------------------------------------------

def perform_statistical_tests(per_fold_df: pd.DataFrame) -> pd.DataFrame:
    """
    IMPROVED: Test if GradientBoosting with_network significantly outperforms
    GradientBoosting no_network and baselines using paired t-tests.
    """
    safe_print("\n===== STATISTICAL SIGNIFICANCE TESTS =====")
    test_results = []

    for crop in CROPS:
        crop_data = per_fold_df[per_fold_df["Crop"] == crop]

        # Get GradientBoosting scores
        gb_with_net = crop_data[
            (crop_data["Model"] == "GradientBoosting") &
            (crop_data["FeatureSet"] == "with_network")
        ]["MAE"].values

        gb_no_net = crop_data[
            (crop_data["Model"] == "GradientBoosting") &
            (crop_data["FeatureSet"] == "no_network")
        ]["MAE"].values

        naive = crop_data[crop_data["Model"] == "Naive"]["MAE"].values
        rolling = crop_data[crop_data["Model"] == "RollingMean3"]["MAE"].values

        if len(gb_with_net) > 0 and len(gb_no_net) > 0 and len(gb_with_net) == len(gb_no_net):
            # Test: GB with_network vs GB no_network
            t_stat, p_val = stats.ttest_rel(gb_with_net, gb_no_net)
            improvement = ((gb_no_net.mean() - gb_with_net.mean()) / gb_no_net.mean()) * 100

            test_results.append({
                "Crop": crop,
                "Comparison": "GB_with_net vs GB_no_net",
                "Mean_MAE_1": gb_with_net.mean(),
                "Mean_MAE_2": gb_no_net.mean(),
                "Improvement_%": improvement,
                "t_statistic": t_stat,
                "p_value": p_val,
                "Significant_05": "Yes" if p_val < 0.05 else "No"
            })

            safe_print(f"{crop}: GB with_network vs no_network: "
                      f"Improvement={improvement:.2f}%, p={p_val:.4f}")

        if len(gb_with_net) > 0 and len(naive) > 0 and len(gb_with_net) == len(naive):
            # Test: GB with_network vs Naive
            t_stat, p_val = stats.ttest_rel(gb_with_net, naive)
            improvement = ((naive.mean() - gb_with_net.mean()) / naive.mean()) * 100

            test_results.append({
                "Crop": crop,
                "Comparison": "GB_with_net vs Naive",
                "Mean_MAE_1": gb_with_net.mean(),
                "Mean_MAE_2": naive.mean(),
                "Improvement_%": improvement,
                "t_statistic": t_stat,
                "p_value": p_val,
                "Significant_05": "Yes" if p_val < 0.05 else "No"
            })

            safe_print(f"{crop}: GB with_network vs Naive: "
                      f"Improvement={improvement:.2f}%, p={p_val:.4f}")

    safe_print()
    return pd.DataFrame(test_results)


# -------------------------------------------------------------------
# TEMPORAL STABILITY ANALYSIS
# -------------------------------------------------------------------

def analyze_temporal_stability(
    per_fold_df: pd.DataFrame,
    panels_by_crop: Dict
) -> pd.DataFrame:
    """
    IMPROVED: Analyze how model performance varies over time
    to detect potential concept drift or temporal instability.
    """
    safe_print("\n===== TEMPORAL STABILITY ANALYSIS =====")
    stability_results = []

    for crop in CROPS:
        if crop not in panels_by_crop:
            continue

        panel = panels_by_crop[crop]["panel"]
        yield_col = panels_by_crop[crop]["yield_col"]

        # Get per-fold results
        crop_folds = per_fold_df[
            (per_fold_df["Crop"] == crop) &
            (per_fold_df["Model"] == "GradientBoosting") &
            (per_fold_df["FeatureSet"] == "with_network")
        ]

        if len(crop_folds) == 0:
            continue

        # Check if performance degrades over folds (time)
        mae_by_fold = crop_folds.sort_values("Fold")["MAE"].values
        r2_by_fold = crop_folds.sort_values("Fold")["R2"].values

        if len(mae_by_fold) >= 2:
            # Compute trend (positive slope = degrading performance)
            fold_nums = np.arange(len(mae_by_fold))
            mae_slope, mae_intercept, mae_r, _, _ = stats.linregress(fold_nums, mae_by_fold)
            r2_slope, r2_intercept, r2_r, _, _ = stats.linregress(fold_nums, r2_by_fold)

            stability_results.append({
                "Crop": crop,
                "MAE_Fold1": mae_by_fold[0],
                "MAE_Fold_Last": mae_by_fold[-1],
                "MAE_Slope": mae_slope,
                "MAE_Trend_R2": mae_r**2,
                "R2_Fold1": r2_by_fold[0],
                "R2_Fold_Last": r2_by_fold[-1],
                "R2_Slope": r2_slope,
                "Performance_Stable": "Yes" if abs(mae_slope) < 10 else "No"
            })

            safe_print(f"{crop}: MAE slope={mae_slope:.2f} (trend R²={mae_r**2:.3f}), "
                      f"R² slope={r2_slope:.4f}")

    safe_print()
    return pd.DataFrame(stability_results)


# -------------------------------------------------------------------
# FEATURE IMPORTANCE ANALYSIS
# -------------------------------------------------------------------

def analyze_feature_importance(
    feature_importance_by_crop: Dict,
    output_path: str
) -> None:
    """
    IMPROVED: Comprehensive feature importance analysis with network contribution.
    """
    safe_print("\n===== FEATURE IMPORTANCE ANALYSIS =====")
    all_importance_records = []

    for crop, imp_data in feature_importance_by_crop.items():
        for feature_set in ["no_network", "with_network"]:
            if feature_set not in imp_data:
                continue

            features = imp_data[feature_set]['features']
            importances = imp_data[feature_set]['importance']

            # Sort by importance
            sorted_idx = np.argsort(importances)[::-1]

            for rank, idx in enumerate(sorted_idx, 1):
                is_network = any(x in features[idx].lower()
                                for x in ['net_', 'network', 'centrality'])
                is_temporal = any(x in features[idx].lower()
                                 for x in ['lag', 'roll', 'yoy'])
                is_diversity = any(x in features[idx].lower()
                                  for x in ['diversity', 'num_active'])

                all_importance_records.append({
                    "Crop": crop,
                    "FeatureSet": feature_set,
                    "Feature": features[idx],
                    "Importance": importances[idx],
                    "Rank": rank,
                    "IsNetwork": is_network,
                    "IsTemporal": is_temporal,
                    "IsDiversity": is_diversity
                })

        # Calculate network contribution
        if "with_network" in imp_data:
            features = imp_data["with_network"]['features']
            importances = imp_data["with_network"]['importance']

            network_mask = [any(x in f.lower() for x in ['net_', 'network', 'centrality'])
                           for f in features]
            network_importance = importances[network_mask].sum() if any(network_mask) else 0.0
            total_importance = importances.sum()
            network_pct = (network_importance / total_importance * 100) if total_importance > 0 else 0.0

            safe_print(f"{crop}: Network features contribute {network_pct:.1f}% of total importance")

    importance_df = pd.DataFrame(all_importance_records)
    importance_df.to_csv(output_path, index=False)
    safe_print(f"\nSaved feature importance to: {output_path}\n")


# -------------------------------------------------------------------
# MODEL SELECTION + SUMMARY
# -------------------------------------------------------------------

def summarise_performance(per_fold_df: pd.DataFrame) -> pd.DataFrame:
    agg = (
        per_fold_df
        .groupby(["Crop", "Model", "FeatureSet"])
        .agg(
            MAE_mean=("MAE", "mean"),
            MAE_std=("MAE", "std"),
            RMSE_mean=("RMSE", "mean"),
            RMSE_std=("RMSE", "std"),
            MAPE_mean=("MAPE", "mean"),
            MAPE_std=("MAPE", "std"),
            MedAE_mean=("MedAE", "mean"),
            MedAE_std=("MedAE", "std"),
            R2_mean=("R2", "mean"),
            R2_std=("R2", "std"),
        )
        .reset_index()
    )
    return agg


def select_best_advanced_models(perf_summary: pd.DataFrame) -> pd.DataFrame:
    advanced_models = ["Ridge", "RandomForest", "GradientBoosting"]
    best_rows = []

    for crop in sorted(perf_summary["Crop"].unique()):
        sub = perf_summary[
            (perf_summary["Crop"] == crop)
            & (perf_summary["Model"].isin(advanced_models))
        ]
        if sub.empty:
            continue

        sub_with_net = sub[sub["FeatureSet"] == "with_network"]
        if not sub_with_net.empty:
            cand = sub_with_net
        else:
            cand = sub

        min_mae = cand["MAE_mean"].min()
        mae_tol = min_mae * 1.01
        cand = cand[cand["MAE_mean"] <= mae_tol]

        best_idx = cand["R2_mean"].idxmax()
        best_row = cand.loc[best_idx].copy()
        best_rows.append(best_row)

    best_df = pd.DataFrame(best_rows).reset_index(drop=True)
    if not best_df.empty:
        best_df["ModelLabel"] = [
            f"Proposed_{m}_{fs}"
            for m, fs in zip(best_df["Model"], best_df["FeatureSet"])
        ]
    return best_df


# -------------------------------------------------------------------
# ENHANCED DIAGNOSTIC PLOTS
# -------------------------------------------------------------------

def save_enhanced_diagnostic_plots(
    crop: str,
    model_name: str,
    feature_set: str,
    y_true: np.ndarray,
    y_pred: np.ndarray,
    output_dir: str
):
    """
    IMPROVED: Enhanced diagnostic plots with publication-quality visualizations
    """
    mae = mean_absolute_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred) ** 0.5
    r2 = r2_score(y_true, y_pred)
    residuals = y_true - y_pred

    # Create 2x2 subplot figure
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))

    # 1. Predicted vs Observed
    ax = axes[0, 0]
    ax.scatter(y_true, y_pred, alpha=0.4, s=20, edgecolors='none')
    lims = [min(y_true.min(), y_pred.min()), max(y_true.max(), y_pred.max())]
    ax.plot(lims, lims, 'r--', linewidth=2, label='Perfect prediction')
    ax.set_xlabel('Observed Yield (kg/ha)', fontsize=11)
    ax.set_ylabel('Predicted Yield (kg/ha)', fontsize=11)
    ax.set_title(f'{crop} - Predicted vs Observed\nMAE={mae:.1f}, RMSE={rmse:.1f}, R²={r2:.3f}',
                 fontsize=12)
    ax.legend()
    ax.grid(alpha=0.3)

    # 2. Residuals vs Predicted
    ax = axes[0, 1]
    ax.scatter(y_pred, residuals, alpha=0.4, s=20, edgecolors='none')
    ax.axhline(0, color='r', linestyle='--', linewidth=2)
    ax.set_xlabel('Predicted Yield (kg/ha)', fontsize=11)
    ax.set_ylabel('Residual (kg/ha)', fontsize=11)
    ax.set_title('Residual Plot', fontsize=12)
    ax.grid(alpha=0.3)

    # 3. Residual Distribution
    ax = axes[1, 0]
    ax.hist(residuals, bins=50, edgecolor='black', alpha=0.7)
    ax.axvline(0, color='r', linestyle='--', linewidth=2)
    ax.set_xlabel('Residual (kg/ha)', fontsize=11)
    ax.set_ylabel('Frequency', fontsize=11)
    ax.set_title(f'Residual Distribution\nMean={residuals.mean():.1f}, Std={residuals.std():.1f}',
                 fontsize=12)
    ax.grid(alpha=0.3)

    # 4. Q-Q Plot
    ax = axes[1, 1]
    stats.probplot(residuals, dist="norm", plot=ax)
    ax.set_title('Q-Q Plot (Normality Check)', fontsize=12)
    ax.grid(alpha=0.3)

    plt.tight_layout()
    plot_path = os.path.join(
        output_dir,
        f"{crop}_{model_name}_{feature_set}_comprehensive_diagnostics.pdf"
    )
    plt.savefig(plot_path, dpi=300, bbox_inches='tight')
    plt.close()

    safe_print(f"Saved enhanced diagnostic plots for {crop} ({model_name}, {feature_set}) to {output_dir}.")


# -------------------------------------------------------------------
# MAIN PIPELINE
# -------------------------------------------------------------------

def main():
    safe_print("="*70)
    safe_print("ENHANCED AGRICULTURAL YIELD PREDICTION PIPELINE")
    safe_print("="*70 + "\n")

    df, source_type = load_data()
    safe_print(f"Data source type: {source_type}")
    basic_eda(df)

    df = clean_yields(df)
    df = compute_diversification_features(df)

    dist_graph, dist_feats = build_district_network(df)
    crop_graph = build_crop_cooccurrence_network(df)

    if not dist_feats.empty:
        df = df.merge(dist_feats, on="Dist Code", how="left")

    df.to_csv(CLEANED_CSV_PATH, index=False)
    safe_print(f"Saved cleaned dataset with features to: {CLEANED_CSV_PATH}\n")

    all_records = []
    panels_by_crop: Dict[str, Dict] = {}
    feature_importance_by_crop: Dict[str, Dict] = {}

    for crop in CROPS:
        safe_print(f"\n{'='*70}")
        safe_print(f"PROCESSING {crop}")
        safe_print(f"{'='*70}")

        panel, y_col, feat_cols, net_cols = prepare_panel_for_crop(df, crop)

        panel_path = f"{SUPERVISED_PANEL_PREFIX}{crop.lower()}.csv"
        panel.to_csv(panel_path, index=False)
        safe_print(f"Saved supervised panel for {crop} to: {panel_path}\n")

        panels_by_crop[crop] = {
            "panel": panel,
            "yield_col": y_col,
            "feature_cols": feat_cols,
            "network_feature_cols": net_cols,
        }

        crop_records, feat_imp = evaluate_models_for_crop(
            crop=crop,
            panel=panel,
            yield_col=y_col,
            feature_cols=feat_cols,
            network_feature_cols=net_cols,
        )
        all_records.extend(crop_records)
        feature_importance_by_crop[crop] = feat_imp

    per_fold_df = pd.DataFrame(all_records)
    per_fold_df.to_csv(MODEL_PER_FOLD_CSV, index=False)
    safe_print(f"\nSaved per-fold model results to: {MODEL_PER_FOLD_CSV}")

    perf_summary = summarise_performance(per_fold_df)
    perf_summary.to_csv(MODEL_PERF_SUMMARY_CSV, index=False)
    safe_print(f"Saved model performance summary to: {MODEL_PERF_SUMMARY_CSV}\n")

    # NEW: Statistical significance tests
    stat_tests = perform_statistical_tests(per_fold_df)
    stat_tests.to_csv(STATISTICAL_TESTS_CSV, index=False)
    safe_print(f"Saved statistical tests to: {STATISTICAL_TESTS_CSV}\n")

    # NEW: Temporal stability analysis
    stability_analysis = analyze_temporal_stability(per_fold_df, panels_by_crop)
    stability_analysis.to_csv(TEMPORAL_STABILITY_CSV, index=False)
    safe_print(f"Saved temporal stability analysis to: {TEMPORAL_STABILITY_CSV}\n")

    # NEW: Feature importance analysis
    analyze_feature_importance(feature_importance_by_crop, FEATURE_IMPORTANCE_CSV)

    for crop in CROPS:
        for fs in ["no_network", "with_network"]:
            sub = perf_summary[(perf_summary["Crop"] == crop) & (perf_summary["FeatureSet"] == fs)]
            if sub.empty:
                continue
            safe_print(f"\nPerformance summary for {crop} ({fs}):")
            safe_print(sub.sort_values("MAE_mean").to_string(index=False))

    best_df = select_best_advanced_models(perf_summary)
    best_df.to_csv(BEST_MODELS_CSV, index=False)

    safe_print("\n" + "="*70)
    safe_print("FINAL SUMMARY (BEST ADVANCED MODELS)")
    safe_print("="*70)
    for row in best_df.itertuples(index=False):
        safe_print(
            f"  {row.Crop}: {row.Model} ({row.FeatureSet}) | "
            f"MAE={row.MAE_mean:.2f} (±{row.MAE_std:.2f}), R²={row.R2_mean:.3f} (±{row.R2_std:.3f})"
        )

    # Generate enhanced diagnostic plots
    safe_print("\n" + "="*70)
    safe_print("GENERATING ENHANCED DIAGNOSTIC PLOTS")
    safe_print("="*70)

    for row in best_df.itertuples(index=False):
        crop = row.Crop
        meta = panels_by_crop[crop]
        panel = meta["panel"]
        y_col = meta["yield_col"]
        feat_cols = meta["feature_cols"]
        net_cols = meta["network_feature_cols"]

        # Fit on full data for diagnostics
        mask_valid = panel[y_col].notna() & (panel[y_col] > 0)
        panel_valid = panel.loc[mask_valid].copy()

        non_net_cols = [c for c in feat_cols if c not in net_cols]
        if row.FeatureSet == "with_network" and net_cols:
            use_cols = non_net_cols + net_cols
        else:
            use_cols = non_net_cols

        X = panel_valid[use_cols].fillna(0.0).values
        y = panel_valid[y_col].values

        if row.Model == "GradientBoosting":
            model = GradientBoostingRegressor(random_state=RANDOM_STATE)
        elif row.Model == "RandomForest":
            model = RandomForestRegressor(n_estimators=300, random_state=RANDOM_STATE, n_jobs=-1)
        elif row.Model == "Ridge":
            scaler = RobustScaler()
            X = scaler.fit_transform(X)
            model = create_robust_ridge(alpha=10.0)
        else:
            continue

        model.fit(X, y)
        y_pred = model.predict(X)

        save_enhanced_diagnostic_plots(
            crop=crop,
            model_name=row.Model,
            feature_set=row.FeatureSet,
            y_true=y,
            y_pred=y_pred,
            output_dir=OUTPUT_DIR
        )

    safe_print("\n" + "="*70)
    safe_print("PIPELINE COMPLETE")
    safe_print("="*70)
    safe_print(f"Rows after cleaning: {len(df)}")
    if "Year" in df.columns:
        safe_print(f"Years: {int(df['Year'].min())} - {int(df['Year'].max())}")
    safe_print(f"Districts: {df['Dist Code'].nunique() if 'Dist Code' in df.columns else 'N/A'}")
    safe_print(f"Crops modeled: {CROPS}")
    safe_print(f"\nKey outputs:")
    safe_print(f"  - Cleaned dataset: {CLEANED_CSV_PATH}")
    safe_print(f"  - Performance summary: {MODEL_PERF_SUMMARY_CSV}")
    safe_print(f"  - Statistical tests: {STATISTICAL_TESTS_CSV}")
    safe_print(f"  - Temporal stability: {TEMPORAL_STABILITY_CSV}")
    safe_print(f"  - Feature importance: {FEATURE_IMPORTANCE_CSV}")
    safe_print(f"  - Best models: {BEST_MODELS_CSV}")
    safe_print(f"  - Diagnostic plots: {OUTPUT_DIR}/*.pdf")


if __name__ == "__main__":
    warnings.filterwarnings("ignore")
    main()

ENHANCED AGRICULTURAL YIELD PREDICTION PIPELINE

RAW_CSV_PATH not found.
Falling back to existing CLEANED dataset: /content/drive/MyDrive/Shiny/ICRISAT/Results/icrisat_cleaned_with_features.csv

Data source type: cleaned
===== BASIC EDA =====
Duplicate (Dist Code, Year) rows: 0
Key (Dist Code, Year) is unique.

Rows: 16146, Columns: 95
Time range: 1966 - 2017
Unique States: 20
Detected crops (count=23): ['BARLEY', 'CASTOR', 'CHICKPEA', 'COTTON', 'FINGER MILLET', 'GROUNDNUT', 'KHARIF SORGHUM', 'LINSEED', 'MAIZE', 'MINOR PULSES', 'OILSEEDS', 'PEARL MILLET', 'PIGEONPEA', 'RABI SORGHUM', 'RAPESEED AND MUSTARD', 'RICE', 'SAFFLOWER', 'SESAMUM', 'SORGHUM', 'SOYABEAN', 'SUGARCANE', 'SUNFLOWER', 'WHEAT']...

Yield missingness (% of NaN):
                           YieldColumn  MissingPct
                RICE YIELD (Kg per ha)         0.0
               WHEAT YIELD (Kg per ha)         0.0
      KHARIF SORGHUM YIELD (Kg per ha)         0.0
        RABI SORGHUM YIELD (Kg per ha)         0.0
      

In [ ]:
import os
import warnings
from typing import Dict, List, Tuple, Optional

import numpy as np
import pandas as pd
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

from scipy import stats
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.feature_selection import VarianceThreshold

# -------------------------------------------------------------------
# CONFIG
# -------------------------------------------------------------------

RAW_CSV_PATH = "/content/drive/MyDrive/Shiny/ICRISAT/Data/icrisat_raw.csv"
CLEANED_CSV_PATH = "/content/drive/MyDrive/Shiny/ICRISAT/Results1/icrisat_cleaned_with_features.csv"
OUTPUT_DIR = "/content/drive/MyDrive/Shiny/ICRISAT/Results1"

SUPERVISED_PANEL_PREFIX = os.path.join(OUTPUT_DIR, "supervised_panel_")
MODEL_PER_FOLD_CSV = os.path.join(OUTPUT_DIR, "model_per_fold_raw.csv")
MODEL_PERF_SUMMARY_CSV = os.path.join(OUTPUT_DIR, "model_performance_summary.csv")
BEST_MODELS_CSV = os.path.join(OUTPUT_DIR, "best_models_advanced_only.csv")
FEATURE_IMPORTANCE_CSV = os.path.join(OUTPUT_DIR, "feature_importance_summary.csv")
STATISTICAL_TESTS_CSV = os.path.join(OUTPUT_DIR, "statistical_significance_tests.csv")
TEMPORAL_STABILITY_CSV = os.path.join(OUTPUT_DIR, "temporal_stability_analysis.csv")

CROPS = ["RICE", "WHEAT", "MAIZE", "GROUNDNUT", "COTTON", "SUGARCANE"]
RANDOM_STATE = 42
N_FOLDS = 3

os.makedirs(OUTPUT_DIR, exist_ok=True)

# -------------------------------------------------------------------
# UTILS
# -------------------------------------------------------------------

def safe_mape(y_true: np.ndarray, y_pred: np.ndarray, min_threshold: float = 100.0) -> float:
    """
    Mean Absolute Percentage Error in %, with minimum threshold to avoid
    division by near-zero values (especially important for cotton).
    """
    y_true = np.array(y_true, dtype=float)
    y_pred = np.array(y_pred, dtype=float)
    mask = y_true > min_threshold
    if not np.any(mask):
        return np.nan
    return np.mean(np.abs(y_true[mask] - y_pred[mask]) / y_true[mask]) * 100.0


def median_ae(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    return float(np.median(np.abs(np.array(y_true) - np.array(y_pred))))


def safe_print(*args, **kwargs):
    try:
        print(*args, **kwargs)
    except Exception:
        pass


# -------------------------------------------------------------------
# LOADING + BASIC EDA
# -------------------------------------------------------------------

def load_data() -> Tuple[pd.DataFrame, str]:
    if os.path.exists(RAW_CSV_PATH):
        safe_print(f"RAW_CSV_PATH found: {RAW_CSV_PATH}")
        df = pd.read_csv(RAW_CSV_PATH)
        source_type = "raw"
    elif os.path.exists(CLEANED_CSV_PATH):
        safe_print("RAW_CSV_PATH not found.")
        safe_print(f"Falling back to existing CLEANED dataset: {CLEANED_CSV_PATH}\n")
        df = pd.read_csv(CLEANED_CSV_PATH)
        source_type = "cleaned"
    else:
        raise FileNotFoundError(
            f"Neither raw ({RAW_CSV_PATH}) nor cleaned ({CLEANED_CSV_PATH}) dataset is available."
        )
    return df, source_type


def basic_eda(df: pd.DataFrame) -> None:
    safe_print("===== BASIC EDA =====")
    key_cols = ["Dist Code", "Year"]
    if all(k in df.columns for k in key_cols):
        dup = df.duplicated(subset=key_cols).sum()
        safe_print(f"Duplicate (Dist Code, Year) rows: {dup}")
        safe_print(f"Key (Dist Code, Year) is {'NOT ' if dup>0 else ''}unique.\n")

    safe_print(f"Rows: {len(df):d}, Columns: {df.shape[1]:d}")

    if "Year" in df.columns:
        safe_print(f"Time range: {int(df['Year'].min())} - {int(df['Year'].max())}")

    state_cols = [c for c in df.columns if "STATE" in c.upper() or "STATE NAME" in c.upper()]
    if state_cols:
        safe_print(f"Unique States: {df[state_cols[0]].nunique()}")

    yield_cols = [c for c in df.columns if "YIELD (Kg per ha)" in c]
    crops = sorted(list({c.split(" YIELD")[0].strip() for c in yield_cols}))
    safe_print(f"Detected crops (count={len(crops)}): {crops[:23]}...\n")

    if yield_cols:
        miss_info = pd.DataFrame({
            "YieldColumn": yield_cols,
            "MissingPct": [df[col].isna().mean() * 100.0 for col in yield_cols],
        }).sort_values("MissingPct").reset_index(drop=True)
        safe_print("Yield missingness (% of NaN):")
        safe_print(miss_info.to_string(index=False))
        safe_print()

        for col in ["RICE YIELD (Kg per ha)", "WHEAT YIELD (Kg per ha)",
                    "MAIZE YIELD (Kg per ha)", "GROUNDNUT YIELD (Kg per ha)",
                    "COTTON YIELD (Kg per ha)", "SUGARCANE YIELD (Kg per ha)"]:
            if col in df.columns:
                safe_print(f"Summary for {col.split(' YIELD')[0]} yield:")
                safe_print(df[col].describe())
                safe_print()


# -------------------------------------------------------------------
# CLEANING YIELDS + DIVERSIFICATION
# -------------------------------------------------------------------

def clean_yields(df: pd.DataFrame) -> pd.DataFrame:
    safe_print("===== CLEANING DATA =====")
    df = df.copy()
    yield_cols = [c for c in df.columns if "YIELD (Kg per ha)" in c]

    for col in yield_cols:
        s = df[col].copy()
        old_max = s.max()
        s = s.clip(lower=0)
        if (s > 0).sum() > 0:
            q = s[s > 0].quantile(0.995)
            s = s.clip(upper=q)
            new_max = float(s.max())
            if new_max < old_max:
                safe_print(f"Clipped {col.split(' YIELD')[0]} yield: "
                          f"old max={old_max:.2f}, new max={new_max:.2f}")
        df[col] = s

    safe_print()
    return df


def compute_diversification_features(df: pd.DataFrame) -> pd.DataFrame:
    safe_print("===== COMPUTING DIVERSIFICATION =====")
    df = df.copy()

    yield_cols = [c for c in df.columns if "YIELD (Kg per ha)" in c]
    crop_names = [c.split(" YIELD")[0].strip() for c in yield_cols]

    crop_area_cols = {}
    for crop in crop_names:
        candidates = [c for c in df.columns if crop in c and "AREA" in c.upper()]
        if candidates:
            crop_area_cols[crop] = candidates[0]

    if not crop_area_cols:
        safe_print("No AREA columns detected – diversification features will be zeros.\n")
        df["diversity_simpson"] = 0.0
        df["num_active_crops"] = 0
        return df

    area_mat = df[[crop_area_cols[c] for c in crop_area_cols]].fillna(0.0).values
    total_area = area_mat.sum(axis=1)
    total_area_safe = np.where(total_area == 0, 1.0, total_area)
    shares = area_mat / total_area_safe[:, None]
    hhi = (shares ** 2).sum(axis=1)
    simpson = 1.0 - hhi
    num_active = (area_mat > 0).sum(axis=1)

    df["diversity_simpson"] = simpson
    df["num_active_crops"] = num_active

    return df


# -------------------------------------------------------------------
# ENHANCED NETWORK CONSTRUCTION
# -------------------------------------------------------------------

def build_district_network(df: pd.DataFrame) -> Tuple[nx.Graph, pd.DataFrame]:
    """
    IMPROVED: Build district similarity network with:
    - Higher correlation threshold (0.80) for meaningful connections
    - Top-k pruning to maintain sparse, interpretable network
    - Additional centrality measures
    """
    safe_print("===== BUILDING ENHANCED DISTRICT SIMILARITY NETWORK =====")
    df_ref = df.copy()
    if "Year" in df_ref.columns:
        df_ref = df_ref[(df_ref["Year"] >= 2008) & (df_ref["Year"] <= 2017)]

    if "Dist Code" not in df_ref.columns:
        safe_print("No 'Dist Code' column: cannot build district network.")
        return nx.Graph(), pd.DataFrame(columns=["Dist Code"])

    yield_cols = [c for c in df_ref.columns if "YIELD (Kg per ha)" in c]
    if not yield_cols:
        safe_print("No YIELD columns found: cannot build district network.")
        return nx.Graph(), pd.DataFrame(columns=["Dist Code"])

    # Aggregate to district-level mean yields
    prof = df_ref.groupby("Dist Code")[yield_cols].mean().fillna(0.0)

    if prof.shape[0] == 0:
        safe_print("No district profiles after filtering.")
        return nx.Graph(), pd.DataFrame(columns=["Dist Code"])

    # Compute cosine similarity
    arr = prof.values
    norms = np.linalg.norm(arr, axis=1)
    norms_safe = np.where(norms == 0, 1.0, norms)
    arr_norm = arr / norms_safe[:, None]
    sim = arr_norm @ arr_norm.T

    dist_codes = prof.index.tolist()
    G = nx.Graph()
    for di in dist_codes:
        G.add_node(di)

    # IMPROVED: Higher threshold for more meaningful connections
    threshold = 0.80
    n = sim.shape[0]
    edges_added = []

    for i in range(n):
        for j in range(i + 1, n):
            w = float(sim[i, j])
            if w >= threshold:
                edges_added.append((dist_codes[i], dist_codes[j], w))

    # IMPROVED: Keep only top-k strongest edges per node
    max_edges_per_node = 15
    node_edges = {node: [] for node in dist_codes}

    for d1, d2, w in edges_added:
        node_edges[d1].append((d2, w))
        node_edges[d2].append((d1, w))

    for node in dist_codes:
        edges = sorted(node_edges[node], key=lambda x: x[1], reverse=True)[:max_edges_per_node]
        for neighbor, weight in edges:
            if not G.has_edge(node, neighbor):
                G.add_edge(node, neighbor, weight=weight)

    density = nx.density(G) if G.number_of_nodes() > 0 else 0.0
    safe_print(f"Network reference window: 2008-2017")
    safe_print(f"District network: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")
    safe_print(f"Network density: {density:.1%} (target: 5-20%)\n")

    if G.number_of_nodes() == 0:
        return G, pd.DataFrame(columns=["Dist Code"])

    # IMPROVED: Multiple centrality measures
    deg = dict(G.degree())
    strength = {n: sum(d["weight"] for _, _, d in G.edges(n, data=True)) for n in G.nodes}

    # Only compute closeness/betweenness if network is connected enough
    if nx.is_connected(G):
        closeness = nx.closeness_centrality(G)
        betw = nx.betweenness_centrality(G, normalized=True)
    else:
        # For disconnected graphs, compute per component
        closeness = {}
        betw = {}
        for component in nx.connected_components(G):
            subG = G.subgraph(component)
            comp_close = nx.closeness_centrality(subG)
            comp_betw = nx.betweenness_centrality(subG, normalized=True)
            closeness.update(comp_close)
            betw.update(comp_betw)

    eigenvector = nx.eigenvector_centrality(G, max_iter=1000, weight='weight')
    clustering = nx.clustering(G, weight='weight')

    dist_features = pd.DataFrame({
        "Dist Code": list(G.nodes),
        "net_degree": [deg[d] for d in G.nodes],
        "net_strength": [strength[d] for d in G.nodes],
        "net_closeness": [closeness[d] for d in G.nodes],
        "net_betweenness": [betw[d] for d in G.nodes],
        "net_eigenvector": [eigenvector[d] for d in G.nodes],
        "net_clustering": [clustering[d] for d in G.nodes],
    })

    return G, dist_features


def build_crop_cooccurrence_network(df: pd.DataFrame) -> nx.Graph:
    safe_print("===== BUILDING CROP-COOCCURRENCE NETWORK =====")
    yield_cols = [c for c in df.columns if "YIELD (Kg per ha)" in c]
    if not yield_cols:
        safe_print("No YIELD columns, skipping crop network.\n")
        return nx.Graph()

    crop_names = [c.split(" YIELD")[0].strip() for c in yield_cols]
    crop_idx = {col: crop_names[i] for i, col in enumerate(yield_cols)}

    G = nx.Graph()
    for crop in crop_names:
        G.add_node(crop)

    key_cols = [c for c in ["Dist Code", "Year"] if c in df.columns]
    if not key_cols:
        rows = [df]
    else:
        rows = [g for _, g in df.groupby(key_cols)]

    from itertools import combinations

    for sub in rows:
        vals = sub[yield_cols].fillna(0.0)
        active_mask = (vals > 0).any(axis=0)
        active_cols = [c for c, m in zip(yield_cols, active_mask) if m]
        active_crops = [crop_idx[c] for c in active_cols]
        for c1, c2 in combinations(sorted(set(active_crops)), 2):
            if G.has_edge(c1, c2):
                G[c1][c2]["weight"] += 1
            else:
                G.add_edge(c1, c2, weight=1)

    safe_print(f"Crop network: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges\n")
    return G


# -------------------------------------------------------------------
# SUPERVISED PANEL PREPARATION WITH FEATURE ENGINEERING
# -------------------------------------------------------------------

def prepare_panel_for_crop(df: pd.DataFrame, crop: str) -> Tuple[pd.DataFrame, str, List[str], List[str]]:
    """
    IMPROVED: Enhanced feature engineering with:
    - Robust handling of multicollinearity
    - Removal of near-zero variance features
    - Proper scaling indicators
    """
    yield_candidates = [c for c in df.columns if c.startswith(crop) and "YIELD (Kg per ha)" in c]
    if not yield_candidates:
        raise ValueError(f"Yield column for crop '{crop}' not found.")
    yield_col = yield_candidates[0]

    if "Dist Code" not in df.columns or "Year" not in df.columns:
        raise ValueError("Both 'Dist Code' and 'Year' must be present in the dataset.")

    panel = df.copy()
    panel = panel.sort_values(["Dist Code", "Year"])
    grp = panel.groupby("Dist Code")[yield_col]

    # Temporal features
    panel[f"{crop}_lag1"] = grp.shift(1)
    panel[f"{crop}_lag2"] = grp.shift(2)
    panel[f"{crop}_lag3"] = grp.shift(3)
    panel[f"{crop}_roll3"] = grp.shift(1).rolling(window=3, min_periods=1).mean()

    # IMPROVED: Year-over-year change
    panel[f"{crop}_yoy_change"] = grp.diff(1)

    # IMPROVED: Volatility measure (rolling std)
    panel[f"{crop}_roll3_std"] = grp.shift(1).rolling(window=3, min_periods=2).std()

    excluded = {"Dist Code", "Year", yield_col}
    feature_cols = []
    for col in panel.columns:
        if col in excluded:
            continue
        if np.issubdtype(panel[col].dtype, np.number):
            # IMPROVED: Check for near-zero variance
            if panel[col].std() > 1e-6:
                feature_cols.append(col)
            else:
                safe_print(f"  Excluding near-zero variance feature: {col}")

    # IMPROVED: Check for perfect multicollinearity
    numeric_panel = panel[feature_cols].fillna(0.0)
    corr_matrix = numeric_panel.corr().abs()

    # Find highly correlated pairs
    upper_tri = corr_matrix.where(
        np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
    )
    to_drop = [col for col in upper_tri.columns if any(upper_tri[col] > 0.99)]

    if to_drop:
        safe_print(f"  Removing highly correlated features: {to_drop}")
        feature_cols = [c for c in feature_cols if c not in to_drop]

    network_feature_cols = [
        c for c in feature_cols
        if c.lower().startswith("net_")
        or "network" in c.lower()
        or "centrality" in c.lower()
    ]

    safe_print(f"  Total features: {len(feature_cols)}, Network features: {len(network_feature_cols)}")

    return panel, yield_col, feature_cols, network_feature_cols


# -------------------------------------------------------------------
# TIME-SERIES CROSS-VALIDATION SPLITS
# -------------------------------------------------------------------

def time_series_folds(years: np.ndarray, n_folds: int = N_FOLDS) -> List[Tuple[np.ndarray, np.ndarray]]:
    years_unique = np.sort(np.unique(years))
    n_years = len(years_unique)
    if n_years < n_folds + 1:
        warnings.warn(f"Too few years ({n_years}) for {n_folds} folds; using single split.")
        mid = n_years // 2
        return [(years_unique[:mid], years_unique[mid:])]

    split_indices = [int((i + 1) * n_years / (n_folds + 1)) for i in range(n_folds)]

    folds = []
    for i in range(n_folds):
        train_end_idx = split_indices[i]
        test_start_idx = train_end_idx
        test_end_idx = split_indices[i + 1] if i + 1 < len(split_indices) else n_years

        train_years = years_unique[:train_end_idx]
        test_years = years_unique[test_start_idx:test_end_idx]
        if len(train_years) == 0 or len(test_years) == 0:
            continue
        folds.append((train_years, test_years))

    return folds


# -------------------------------------------------------------------
# IMPROVED MODEL TRAINING WITH ROBUST RIDGE
# -------------------------------------------------------------------

def create_robust_ridge(alpha: float = 10.0) -> Ridge:
    """IMPROVED: Ridge with higher regularization to handle multicollinearity"""
    return Ridge(alpha=alpha, max_iter=10000, random_state=RANDOM_STATE)


def evaluate_models_for_crop(
    crop: str,
    panel: pd.DataFrame,
    yield_col: str,
    feature_cols: List[str],
    network_feature_cols: List[str],
) -> Tuple[List[Dict], Dict]:
    """
    IMPROVED: Returns both records and feature importance info
    """
    records = []
    feature_importance_data = {}

    mask_valid = panel[yield_col].notna() & (panel[yield_col] > 0)
    panel_valid = panel.loc[mask_valid].copy()

    years = panel_valid["Year"].values
    folds = time_series_folds(years, n_folds=N_FOLDS)
    if not folds:
        warnings.warn(f"No valid folds for crop {crop}; skipping.")
        return records, feature_importance_data

    non_network_feature_cols = [c for c in feature_cols if c not in network_feature_cols]
    if not non_network_feature_cols:
        non_network_feature_cols = feature_cols
        network_feature_cols = []

    fold_id = 0
    all_importances_gb_no_net = []
    all_importances_gb_with_net = []

    for train_years, test_years in folds:
        fold_id += 1
        train_mask = panel_valid["Year"].isin(train_years)
        test_mask = panel_valid["Year"].isin(test_years)

        train_df = panel_valid.loc[train_mask]
        test_df = panel_valid.loc[test_mask]

        if len(train_df) == 0 or len(test_df) == 0:
            continue

        y_train = train_df[yield_col].values
        y_test = test_df[yield_col].values

        lag_col = f"{crop}_lag1"
        roll3_col = f"{crop}_roll3"

        # Baseline 1: Naive
        if lag_col in test_df.columns:
            y_pred_naive = test_df[lag_col].values.astype(float)
            mean_train = y_train.mean()
            y_pred_naive = np.where(np.isnan(y_pred_naive), mean_train, y_pred_naive)
        else:
            y_pred_naive = np.full_like(y_test, y_train.mean(), dtype=float)

        mae = mean_absolute_error(y_test, y_pred_naive)
        rmse = mean_squared_error(y_test, y_pred_naive) ** 0.5
        mape_val = safe_mape(y_test, y_pred_naive)
        medae = median_ae(y_test, y_pred_naive)
        r2 = r2_score(y_test, y_pred_naive)

        safe_print(f"{crop} | Naive | no_network | Fold {fold_id}: "
                   f"MAE={mae:.2f}, RMSE={rmse:.2f}, MAPE={mape_val:.2f}, MedAE={medae:.2f}, R2={r2:.3f}")

        records.append({
            "Crop": crop, "Model": "Naive", "FeatureSet": "no_network", "Fold": fold_id,
            "MAE": mae, "RMSE": rmse, "MAPE": mape_val, "MedAE": medae, "R2": r2,
        })

        # Baseline 2: RollingMean3
        if roll3_col in test_df.columns:
            y_pred_roll3 = test_df[roll3_col].values.astype(float)
            mean_train = y_train.mean()
            y_pred_roll3 = np.where(np.isnan(y_pred_roll3), mean_train, y_pred_roll3)
        else:
            y_pred_roll3 = np.full_like(y_test, y_train.mean(), dtype=float)

        mae = mean_absolute_error(y_test, y_pred_roll3)
        rmse = mean_squared_error(y_test, y_pred_roll3) ** 0.5
        mape_val = safe_mape(y_test, y_pred_roll3)
        medae = median_ae(y_test, y_pred_roll3)
        r2 = r2_score(y_test, y_pred_roll3)

        safe_print(f"{crop} | RollingMean3 | no_network | Fold {fold_id}: "
                   f"MAE={mae:.2f}, RMSE={rmse:.2f}, MAPE={mape_val:.2f}, MedAE={medae:.2f}, R2={r2:.3f}")

        records.append({
            "Crop": crop, "Model": "RollingMean3", "FeatureSet": "no_network", "Fold": fold_id,
            "MAE": mae, "RMSE": rmse, "MAPE": mape_val, "MedAE": medae, "R2": r2,
        })

        # Advanced models: no_network
        X_train_no_net = train_df[non_network_feature_cols].fillna(0.0).values
        X_test_no_net = test_df[non_network_feature_cols].fillna(0.0).values

        # IMPROVED: Robust scaling for Ridge
        scaler_no_net = RobustScaler()
        X_train_no_net_scaled = scaler_no_net.fit_transform(X_train_no_net)
        X_test_no_net_scaled = scaler_no_net.transform(X_test_no_net)

        # Ridge no_network
        ridge_no_net = create_robust_ridge(alpha=10.0)
        ridge_no_net.fit(X_train_no_net_scaled, y_train)
        y_pred = ridge_no_net.predict(X_test_no_net_scaled)

        mae = mean_absolute_error(y_test, y_pred)
        rmse = mean_squared_error(y_test, y_pred) ** 0.5
        mape_val = safe_mape(y_test, y_pred)
        medae = median_ae(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        safe_print(f"{crop} | Ridge | no_network | Fold {fold_id}: "
                   f"MAE={mae:.2f}, RMSE={rmse:.2f}, MAPE={mape_val:.2f}, MedAE={medae:.2f}, R2={r2:.3f}")
        records.append({
            "Crop": crop, "Model": "Ridge", "FeatureSet": "no_network", "Fold": fold_id,
            "MAE": mae, "RMSE": rmse, "MAPE": mape_val, "MedAE": medae, "R2": r2,
        })

        # RandomForest no_network
        rf_no_net = RandomForestRegressor(
            n_estimators=300, max_depth=None, min_samples_leaf=1,
            random_state=RANDOM_STATE, n_jobs=-1
        )
        rf_no_net.fit(X_train_no_net, y_train)
        y_pred = rf_no_net.predict(X_test_no_net)

        mae = mean_absolute_error(y_test, y_pred)
        rmse = mean_squared_error(y_test, y_pred) ** 0.5
        mape_val = safe_mape(y_test, y_pred)
        medae = median_ae(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        safe_print(f"{crop} | RandomForest | no_network | Fold {fold_id}: "
                   f"MAE={mae:.2f}, RMSE={rmse:.2f}, MAPE={mape_val:.2f}, MedAE={medae:.2f}, R2={r2:.3f}")
        records.append({
            "Crop": crop, "Model": "RandomForest", "FeatureSet": "no_network", "Fold": fold_id,
            "MAE": mae, "RMSE": rmse, "MAPE": mape_val, "MedAE": medae, "R2": r2,
        })

        # GradientBoosting no_network
        gb_no_net = GradientBoostingRegressor(random_state=RANDOM_STATE)
        gb_no_net.fit(X_train_no_net, y_train)
        y_pred = gb_no_net.predict(X_test_no_net)

        mae = mean_absolute_error(y_test, y_pred)
        rmse = mean_squared_error(y_test, y_pred) ** 0.5
        mape_val = safe_mape(y_test, y_pred)
        medae = median_ae(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        safe_print(f"{crop} | GradientBoosting | no_network | Fold {fold_id}: "
                   f"MAE={mae:.2f}, RMSE={rmse:.2f}, MAPE={mape_val:.2f}, MedAE={medae:.2f}, R2={r2:.3f}")
        records.append({
            "Crop": crop, "Model": "GradientBoosting", "FeatureSet": "no_network", "Fold": fold_id,
            "MAE": mae, "RMSE": rmse, "MAPE": mape_val, "MedAE": medae, "R2": r2,
        })

        # Store feature importance
        all_importances_gb_no_net.append(gb_no_net.feature_importances_)

        # Advanced models: with_network
        if network_feature_cols:
            feature_cols_with_net = non_network_feature_cols + network_feature_cols
        else:
            feature_cols_with_net = non_network_feature_cols

        X_train_with_net = train_df[feature_cols_with_net].fillna(0.0).values
        X_test_with_net = test_df[feature_cols_with_net].fillna(0.0).values

        # IMPROVED: Robust scaling for Ridge
        scaler_with_net = RobustScaler()
        X_train_with_net_scaled = scaler_with_net.fit_transform(X_train_with_net)
        X_test_with_net_scaled = scaler_with_net.transform(X_test_with_net)

        # Ridge with_network
        ridge_with_net = create_robust_ridge(alpha=10.0)
        ridge_with_net.fit(X_train_with_net_scaled, y_train)
        y_pred = ridge_with_net.predict(X_test_with_net_scaled)

        mae = mean_absolute_error(y_test, y_pred)
        rmse = mean_squared_error(y_test, y_pred) ** 0.5
        mape_val = safe_mape(y_test, y_pred)
        medae = median_ae(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        safe_print(f"{crop} | Ridge | with_network | Fold {fold_id}: "
                   f"MAE={mae:.2f}, RMSE={rmse:.2f}, MAPE={mape_val:.2f}, MedAE={medae:.2f}, R2={r2:.3f}")
        records.append({
            "Crop": crop, "Model": "Ridge", "FeatureSet": "with_network", "Fold": fold_id,
            "MAE": mae, "RMSE": rmse, "MAPE": mape_val, "MedAE": medae, "R2": r2,
        })

        # RandomForest with_network
        rf_with_net = RandomForestRegressor(
            n_estimators=300, max_depth=None, min_samples_leaf=1,
            random_state=RANDOM_STATE, n_jobs=-1
        )
        rf_with_net.fit(X_train_with_net, y_train)
        y_pred = rf_with_net.predict(X_test_with_net)

        mae = mean_absolute_error(y_test, y_pred)
        rmse = mean_squared_error(y_test, y_pred) ** 0.5
        mape_val = safe_mape(y_test, y_pred)
        medae = median_ae(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        safe_print(f"{crop} | RandomForest | with_network | Fold {fold_id}: "
                   f"MAE={mae:.2f}, RMSE={rmse:.2f}, MAPE={mape_val:.2f}, MedAE={medae:.2f}, R2={r2:.3f}")
        records.append({
            "Crop": crop, "Model": "RandomForest", "FeatureSet": "with_network", "Fold": fold_id,
            "MAE": mae, "RMSE": rmse, "MAPE": mape_val, "MedAE": medae, "R2": r2,
        })

        # GradientBoosting with_network
        gb_with_net = GradientBoostingRegressor(random_state=RANDOM_STATE)
        gb_with_net.fit(X_train_with_net, y_train)
        y_pred = gb_with_net.predict(X_test_with_net)

        mae = mean_absolute_error(y_test, y_pred)
        rmse = mean_squared_error(y_test, y_pred) ** 0.5
        mape_val = safe_mape(y_test, y_pred)
        medae = median_ae(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        safe_print(f"{crop} | GradientBoosting | with_network | Fold {fold_id}: "
                   f"MAE={mae:.2f}, RMSE={rmse:.2f}, MAPE={mape_val:.2f}, MedAE={medae:.2f}, R2={r2:.3f}")
        records.append({
            "Crop": crop, "Model": "GradientBoosting", "FeatureSet": "with_network", "Fold": fold_id,
            "MAE": mae, "RMSE": rmse, "MAPE": mape_val, "MedAE": medae, "R2": r2,
        })

        # Store feature importance
        all_importances_gb_with_net.append(gb_with_net.feature_importances_)

    # Compute average feature importance across folds
    if all_importances_gb_no_net:
        avg_imp_no_net = np.mean(all_importances_gb_no_net, axis=0)
        feature_importance_data['no_network'] = {
            'features': non_network_feature_cols,
            'importance': avg_imp_no_net
        }

    if all_importances_gb_with_net:
        avg_imp_with_net = np.mean(all_importances_gb_with_net, axis=0)
        feature_importance_data['with_network'] = {
            'features': feature_cols_with_net,
            'importance': avg_imp_with_net
        }

    return records, feature_importance_data


# -------------------------------------------------------------------
# STATISTICAL SIGNIFICANCE TESTING
# -------------------------------------------------------------------

def perform_statistical_tests(per_fold_df: pd.DataFrame) -> pd.DataFrame:
    """
    IMPROVED: Test if GradientBoosting with_network significantly outperforms
    GradientBoosting no_network and baselines using paired t-tests.
    """
    safe_print("\n===== STATISTICAL SIGNIFICANCE TESTS =====")
    test_results = []

    for crop in CROPS:
        crop_data = per_fold_df[per_fold_df["Crop"] == crop]

        # Get GradientBoosting scores
        gb_with_net = crop_data[
            (crop_data["Model"] == "GradientBoosting") &
            (crop_data["FeatureSet"] == "with_network")
        ]["MAE"].values

        gb_no_net = crop_data[
            (crop_data["Model"] == "GradientBoosting") &
            (crop_data["FeatureSet"] == "no_network")
        ]["MAE"].values

        naive = crop_data[crop_data["Model"] == "Naive"]["MAE"].values
        rolling = crop_data[crop_data["Model"] == "RollingMean3"]["MAE"].values

        if len(gb_with_net) > 0 and len(gb_no_net) > 0 and len(gb_with_net) == len(gb_no_net):
            # Test: GB with_network vs GB no_network
            t_stat, p_val = stats.ttest_rel(gb_with_net, gb_no_net)
            improvement = ((gb_no_net.mean() - gb_with_net.mean()) / gb_no_net.mean()) * 100

            test_results.append({
                "Crop": crop,
                "Comparison": "GB_with_net vs GB_no_net",
                "Mean_MAE_1": gb_with_net.mean(),
                "Mean_MAE_2": gb_no_net.mean(),
                "Improvement_%": improvement,
                "t_statistic": t_stat,
                "p_value": p_val,
                "Significant_05": "Yes" if p_val < 0.05 else "No"
            })

            safe_print(f"{crop}: GB with_network vs no_network: "
                      f"Improvement={improvement:.2f}%, p={p_val:.4f}")

        if len(gb_with_net) > 0 and len(naive) > 0 and len(gb_with_net) == len(naive):
            # Test: GB with_network vs Naive
            t_stat, p_val = stats.ttest_rel(gb_with_net, naive)
            improvement = ((naive.mean() - gb_with_net.mean()) / naive.mean()) * 100

            test_results.append({
                "Crop": crop,
                "Comparison": "GB_with_net vs Naive",
                "Mean_MAE_1": gb_with_net.mean(),
                "Mean_MAE_2": naive.mean(),
                "Improvement_%": improvement,
                "t_statistic": t_stat,
                "p_value": p_val,
                "Significant_05": "Yes" if p_val < 0.05 else "No"
            })

            safe_print(f"{crop}: GB with_network vs Naive: "
                      f"Improvement={improvement:.2f}%, p={p_val:.4f}")

    safe_print()
    return pd.DataFrame(test_results)


# -------------------------------------------------------------------
# TEMPORAL STABILITY ANALYSIS
# -------------------------------------------------------------------

def analyze_temporal_stability(
    per_fold_df: pd.DataFrame,
    panels_by_crop: Dict
) -> pd.DataFrame:
    """
    IMPROVED: Analyze how model performance varies over time
    to detect potential concept drift or temporal instability.
    """
    safe_print("\n===== TEMPORAL STABILITY ANALYSIS =====")
    stability_results = []

    for crop in CROPS:
        if crop not in panels_by_crop:
            continue

        panel = panels_by_crop[crop]["panel"]
        yield_col = panels_by_crop[crop]["yield_col"]

        # Get per-fold results
        crop_folds = per_fold_df[
            (per_fold_df["Crop"] == crop) &
            (per_fold_df["Model"] == "GradientBoosting") &
            (per_fold_df["FeatureSet"] == "with_network")
        ]

        if len(crop_folds) == 0:
            continue

        # Check if performance degrades over folds (time)
        mae_by_fold = crop_folds.sort_values("Fold")["MAE"].values
        r2_by_fold = crop_folds.sort_values("Fold")["R2"].values

        if len(mae_by_fold) >= 2:
            # Compute trend (positive slope = degrading performance)
            fold_nums = np.arange(len(mae_by_fold))
            mae_slope, mae_intercept, mae_r, _, _ = stats.linregress(fold_nums, mae_by_fold)
            r2_slope, r2_intercept, r2_r, _, _ = stats.linregress(fold_nums, r2_by_fold)

            stability_results.append({
                "Crop": crop,
                "MAE_Fold1": mae_by_fold[0],
                "MAE_Fold_Last": mae_by_fold[-1],
                "MAE_Slope": mae_slope,
                "MAE_Trend_R2": mae_r**2,
                "R2_Fold1": r2_by_fold[0],
                "R2_Fold_Last": r2_by_fold[-1],
                "R2_Slope": r2_slope,
                "Performance_Stable": "Yes" if abs(mae_slope) < 10 else "No"
            })

            safe_print(f"{crop}: MAE slope={mae_slope:.2f} (trend R²={mae_r**2:.3f}), "
                      f"R² slope={r2_slope:.4f}")

    safe_print()
    return pd.DataFrame(stability_results)


# -------------------------------------------------------------------
# FEATURE IMPORTANCE ANALYSIS
# -------------------------------------------------------------------

def analyze_feature_importance(
    feature_importance_by_crop: Dict,
    output_path: str
) -> None:
    """
    IMPROVED: Comprehensive feature importance analysis with network contribution.
    """
    safe_print("\n===== FEATURE IMPORTANCE ANALYSIS =====")
    all_importance_records = []

    for crop, imp_data in feature_importance_by_crop.items():
        for feature_set in ["no_network", "with_network"]:
            if feature_set not in imp_data:
                continue

            features = imp_data[feature_set]['features']
            importances = imp_data[feature_set]['importance']

            # Sort by importance
            sorted_idx = np.argsort(importances)[::-1]

            for rank, idx in enumerate(sorted_idx, 1):
                is_network = any(x in features[idx].lower()
                                for x in ['net_', 'network', 'centrality'])
                is_temporal = any(x in features[idx].lower()
                                 for x in ['lag', 'roll', 'yoy'])
                is_diversity = any(x in features[idx].lower()
                                  for x in ['diversity', 'num_active'])

                all_importance_records.append({
                    "Crop": crop,
                    "FeatureSet": feature_set,
                    "Feature": features[idx],
                    "Importance": importances[idx],
                    "Rank": rank,
                    "IsNetwork": is_network,
                    "IsTemporal": is_temporal,
                    "IsDiversity": is_diversity
                })

        # Calculate network contribution
        if "with_network" in imp_data:
            features = imp_data["with_network"]['features']
            importances = imp_data["with_network"]['importance']

            network_mask = [any(x in f.lower() for x in ['net_', 'network', 'centrality'])
                           for f in features]
            network_importance = importances[network_mask].sum() if any(network_mask) else 0.0
            total_importance = importances.sum()
            network_pct = (network_importance / total_importance * 100) if total_importance > 0 else 0.0

            safe_print(f"{crop}: Network features contribute {network_pct:.1f}% of total importance")

    importance_df = pd.DataFrame(all_importance_records)
    importance_df.to_csv(output_path, index=False)
    safe_print(f"\nSaved feature importance to: {output_path}\n")


# -------------------------------------------------------------------
# MODEL SELECTION + SUMMARY
# -------------------------------------------------------------------

def summarise_performance(per_fold_df: pd.DataFrame) -> pd.DataFrame:
    agg = (
        per_fold_df
        .groupby(["Crop", "Model", "FeatureSet"])
        .agg(
            MAE_mean=("MAE", "mean"),
            MAE_std=("MAE", "std"),
            RMSE_mean=("RMSE", "mean"),
            RMSE_std=("RMSE", "std"),
            MAPE_mean=("MAPE", "mean"),
            MAPE_std=("MAPE", "std"),
            MedAE_mean=("MedAE", "mean"),
            MedAE_std=("MedAE", "std"),
            R2_mean=("R2", "mean"),
            R2_std=("R2", "std"),
        )
        .reset_index()
    )
    return agg


def select_best_advanced_models(perf_summary: pd.DataFrame) -> pd.DataFrame:
    advanced_models = ["Ridge", "RandomForest", "GradientBoosting"]
    best_rows = []

    for crop in sorted(perf_summary["Crop"].unique()):
        sub = perf_summary[
            (perf_summary["Crop"] == crop)
            & (perf_summary["Model"].isin(advanced_models))
        ]
        if sub.empty:
            continue

        sub_with_net = sub[sub["FeatureSet"] == "with_network"]
        if not sub_with_net.empty:
            cand = sub_with_net
        else:
            cand = sub

        min_mae = cand["MAE_mean"].min()
        mae_tol = min_mae * 1.01
        cand = cand[cand["MAE_mean"] <= mae_tol]

        best_idx = cand["R2_mean"].idxmax()
        best_row = cand.loc[best_idx].copy()
        best_rows.append(best_row)

    best_df = pd.DataFrame(best_rows).reset_index(drop=True)
    if not best_df.empty:
        best_df["ModelLabel"] = [
            f"Proposed_{m}_{fs}"
            for m, fs in zip(best_df["Model"], best_df["FeatureSet"])
        ]
    return best_df


# -------------------------------------------------------------------
# ENHANCED DIAGNOSTIC PLOTS
# -------------------------------------------------------------------

def save_enhanced_diagnostic_plots(
    crop: str,
    model_name: str,
    feature_set: str,
    y_true: np.ndarray,
    y_pred: np.ndarray,
    output_dir: str
):
    """
    IMPROVED: Enhanced diagnostic plots with publication-quality visualizations
    """
    mae = mean_absolute_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred) ** 0.5
    r2 = r2_score(y_true, y_pred)
    residuals = y_true - y_pred

    # Create 2x2 subplot figure
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))

    # 1. Predicted vs Observed
    ax = axes[0, 0]
    ax.scatter(y_true, y_pred, alpha=0.4, s=20, edgecolors='none')
    lims = [min(y_true.min(), y_pred.min()), max(y_true.max(), y_pred.max())]
    ax.plot(lims, lims, 'r--', linewidth=2, label='Perfect prediction')
    ax.set_xlabel('Observed Yield (kg/ha)', fontsize=11)
    ax.set_ylabel('Predicted Yield (kg/ha)', fontsize=11)
    ax.set_title(f'{crop} - Predicted vs Observed\nMAE={mae:.1f}, RMSE={rmse:.1f}, R²={r2:.3f}',
                 fontsize=12)
    ax.legend()
    ax.grid(alpha=0.3)

    # 2. Residuals vs Predicted
    ax = axes[0, 1]
    ax.scatter(y_pred, residuals, alpha=0.4, s=20, edgecolors='none')
    ax.axhline(0, color='r', linestyle='--', linewidth=2)
    ax.set_xlabel('Predicted Yield (kg/ha)', fontsize=11)
    ax.set_ylabel('Residual (kg/ha)', fontsize=11)
    ax.set_title('Residual Plot', fontsize=12)
    ax.grid(alpha=0.3)

    # 3. Residual Distribution
    ax = axes[1, 0]
    ax.hist(residuals, bins=50, edgecolor='black', alpha=0.7)
    ax.axvline(0, color='r', linestyle='--', linewidth=2)
    ax.set_xlabel('Residual (kg/ha)', fontsize=11)
    ax.set_ylabel('Frequency', fontsize=11)
    ax.set_title(f'Residual Distribution\nMean={residuals.mean():.1f}, Std={residuals.std():.1f}',
                 fontsize=12)
    ax.grid(alpha=0.3)

    # 4. Q-Q Plot
    ax = axes[1, 1]
    stats.probplot(residuals, dist="norm", plot=ax)
    ax.set_title('Q-Q Plot (Normality Check)', fontsize=12)
    ax.grid(alpha=0.3)

    plt.tight_layout()
    plot_path = os.path.join(
        output_dir,
        f"{crop}_{model_name}_{feature_set}_comprehensive_diagnostics.pdf"
    )
    plt.savefig(plot_path, dpi=300, bbox_inches='tight')
    plt.close()

    safe_print(f"Saved enhanced diagnostic plots for {crop} ({model_name}, {feature_set}) to {output_dir}.")


# -------------------------------------------------------------------
# MAIN PIPELINE
# -------------------------------------------------------------------

def main():
    safe_print("="*70)
    safe_print("ENHANCED AGRICULTURAL YIELD PREDICTION PIPELINE")
    safe_print("="*70 + "\n")

    df, source_type = load_data()
    safe_print(f"Data source type: {source_type}")
    basic_eda(df)

    df = clean_yields(df)
    df = compute_diversification_features(df)

    dist_graph, dist_feats = build_district_network(df)
    crop_graph = build_crop_cooccurrence_network(df)

    if not dist_feats.empty:
        df = df.merge(dist_feats, on="Dist Code", how="left")

    df.to_csv(CLEANED_CSV_PATH, index=False)
    safe_print(f"Saved cleaned dataset with features to: {CLEANED_CSV_PATH}\n")

    all_records = []
    panels_by_crop: Dict[str, Dict] = {}
    feature_importance_by_crop: Dict[str, Dict] = {}

    for crop in CROPS:
        safe_print(f"\n{'='*70}")
        safe_print(f"PROCESSING {crop}")
        safe_print(f"{'='*70}")

        panel, y_col, feat_cols, net_cols = prepare_panel_for_crop(df, crop)

        panel_path = f"{SUPERVISED_PANEL_PREFIX}{crop.lower()}.csv"
        panel.to_csv(panel_path, index=False)
        safe_print(f"Saved supervised panel for {crop} to: {panel_path}\n")

        panels_by_crop[crop] = {
            "panel": panel,
            "yield_col": y_col,
            "feature_cols": feat_cols,
            "network_feature_cols": net_cols,
        }

        crop_records, feat_imp = evaluate_models_for_crop(
            crop=crop,
            panel=panel,
            yield_col=y_col,
            feature_cols=feat_cols,
            network_feature_cols=net_cols,
        )
        all_records.extend(crop_records)
        feature_importance_by_crop[crop] = feat_imp

    per_fold_df = pd.DataFrame(all_records)
    per_fold_df.to_csv(MODEL_PER_FOLD_CSV, index=False)
    safe_print(f"\nSaved per-fold model results to: {MODEL_PER_FOLD_CSV}")

    perf_summary = summarise_performance(per_fold_df)
    perf_summary.to_csv(MODEL_PERF_SUMMARY_CSV, index=False)
    safe_print(f"Saved model performance summary to: {MODEL_PERF_SUMMARY_CSV}\n")

    # NEW: Statistical significance tests
    stat_tests = perform_statistical_tests(per_fold_df)
    stat_tests.to_csv(STATISTICAL_TESTS_CSV, index=False)
    safe_print(f"Saved statistical tests to: {STATISTICAL_TESTS_CSV}\n")

    # NEW: Temporal stability analysis
    stability_analysis = analyze_temporal_stability(per_fold_df, panels_by_crop)
    stability_analysis.to_csv(TEMPORAL_STABILITY_CSV, index=False)
    safe_print(f"Saved temporal stability analysis to: {TEMPORAL_STABILITY_CSV}\n")

    # NEW: Feature importance analysis
    analyze_feature_importance(feature_importance_by_crop, FEATURE_IMPORTANCE_CSV)

    for crop in CROPS:
        for fs in ["no_network", "with_network"]:
            sub = perf_summary[(perf_summary["Crop"] == crop) & (perf_summary["FeatureSet"] == fs)]
            if sub.empty:
                continue
            safe_print(f"\nPerformance summary for {crop} ({fs}):")
            safe_print(sub.sort_values("MAE_mean").to_string(index=False))

    best_df = select_best_advanced_models(perf_summary)
    best_df.to_csv(BEST_MODELS_CSV, index=False)

    safe_print("\n" + "="*70)
    safe_print("FINAL SUMMARY (BEST ADVANCED MODELS)")
    safe_print("="*70)
    for row in best_df.itertuples(index=False):
        safe_print(
            f"  {row.Crop}: {row.Model} ({row.FeatureSet}) | "
            f"MAE={row.MAE_mean:.2f} (±{row.MAE_std:.2f}), R²={row.R2_mean:.3f} (±{row.R2_std:.3f})"
        )

    # Generate enhanced diagnostic plots
    safe_print("\n" + "="*70)
    safe_print("GENERATING ENHANCED DIAGNOSTIC PLOTS")
    safe_print("="*70)

    for row in best_df.itertuples(index=False):
        crop = row.Crop
        meta = panels_by_crop[crop]
        panel = meta["panel"]
        y_col = meta["yield_col"]
        feat_cols = meta["feature_cols"]
        net_cols = meta["network_feature_cols"]

        # Fit on full data for diagnostics
        mask_valid = panel[y_col].notna() & (panel[y_col] > 0)
        panel_valid = panel.loc[mask_valid].copy()

        non_net_cols = [c for c in feat_cols if c not in net_cols]
        if row.FeatureSet == "with_network" and net_cols:
            use_cols = non_net_cols + net_cols
        else:
            use_cols = non_net_cols

        X = panel_valid[use_cols].fillna(0.0).values
        y = panel_valid[y_col].values

        if row.Model == "GradientBoosting":
            model = GradientBoostingRegressor(random_state=RANDOM_STATE)
        elif row.Model == "RandomForest":
            model = RandomForestRegressor(n_estimators=300, random_state=RANDOM_STATE, n_jobs=-1)
        elif row.Model == "Ridge":
            scaler = RobustScaler()
            X = scaler.fit_transform(X)
            model = create_robust_ridge(alpha=10.0)
        else:
            continue

        model.fit(X, y)
        y_pred = model.predict(X)

        save_enhanced_diagnostic_plots(
            crop=crop,
            model_name=row.Model,
            feature_set=row.FeatureSet,
            y_true=y,
            y_pred=y_pred,
            output_dir=OUTPUT_DIR
        )

    safe_print("\n" + "="*70)
    safe_print("PIPELINE COMPLETE")
    safe_print("="*70)
    safe_print(f"Rows after cleaning: {len(df)}")
    if "Year" in df.columns:
        safe_print(f"Years: {int(df['Year'].min())} - {int(df['Year'].max())}")
    safe_print(f"Districts: {df['Dist Code'].nunique() if 'Dist Code' in df.columns else 'N/A'}")
    safe_print(f"Crops modeled: {CROPS}")
    safe_print(f"\nKey outputs:")
    safe_print(f"  - Cleaned dataset: {CLEANED_CSV_PATH}")
    safe_print(f"  - Performance summary: {MODEL_PERF_SUMMARY_CSV}")
    safe_print(f"  - Statistical tests: {STATISTICAL_TESTS_CSV}")
    safe_print(f"  - Temporal stability: {TEMPORAL_STABILITY_CSV}")
    safe_print(f"  - Feature importance: {FEATURE_IMPORTANCE_CSV}")
    safe_print(f"  - Best models: {BEST_MODELS_CSV}")
    safe_print(f"  - Diagnostic plots: {OUTPUT_DIR}/*.pdf")


if __name__ == "__main__":
    warnings.filterwarnings("ignore")
    main()

ENHANCED AGRICULTURAL YIELD PREDICTION PIPELINE

RAW_CSV_PATH not found.
Falling back to existing CLEANED dataset: /content/drive/MyDrive/Shiny/ICRISAT/Results1/icrisat_cleaned_with_features.csv

Data source type: cleaned
===== BASIC EDA =====
Duplicate (Dist Code, Year) rows: 0
Key (Dist Code, Year) is unique.

Rows: 16146, Columns: 85
Time range: 1966 - 2017
Unique States: 20
Detected crops (count=23): ['BARLEY', 'CASTOR', 'CHICKPEA', 'COTTON', 'FINGER MILLET', 'GROUNDNUT', 'KHARIF SORGHUM', 'LINSEED', 'MAIZE', 'MINOR PULSES', 'OILSEEDS', 'PEARL MILLET', 'PIGEONPEA', 'RABI SORGHUM', 'RAPESEED AND MUSTARD', 'RICE', 'SAFFLOWER', 'SESAMUM', 'SORGHUM', 'SOYABEAN', 'SUGARCANE', 'SUNFLOWER', 'WHEAT']...

Yield missingness (% of NaN):
                           YieldColumn  MissingPct
                RICE YIELD (Kg per ha)         0.0
               WHEAT YIELD (Kg per ha)         0.0
      KHARIF SORGHUM YIELD (Kg per ha)         0.0
        RABI SORGHUM YIELD (Kg per ha)         0.0
     